In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import importlib

if importlib.util.find_spec('datasets') is None:
    !pip install -q torch==2.2.1 transformers==4.39.1 diffusers==0.27.2 torchvision==0.17.1 datasets==2.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

In [ ]:
dataset = load_dataset("phiyodr/coco2017")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/118287 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'image_id', 'ids', 'captions'],
        num_rows: 118287
    })
    validation: Dataset({
        features: ['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'image_id', 'ids', 'captions'],
        num_rows: 5000
    })
})

In [ ]:
MAPPING = {
 'airplane': 5,
 'apple': 53,
 'backpack': 27,
 'banana': 52,
 'baseball bat': 39,
 'baseball glove': 40,
#  'bear': 23,
 'bed': 65,
 'bench': 15,
 'bicycle': 2,
 'bird': 16,
 'boat': 9,
 'book': 84,
 'bottle': 44,
 'bowl': 51,
 'broccoli': 56,
 'bus': 6,
 'cake': 61,
 'car': 3,
 'carrot': 57,
 'cat': 17,
 'cell phone': 77,
 'chair': 62,
 'clock': 85,
 'couch': 63,
 'cow': 21,
 'cup': 47,
 'dining table': 67,
 'dog': 18,
 'donut': 60,
 'elephant': 22,
 'fire hydrant': 11,
 'fork': 48,
 'frisbee': 34,
 'giraffe': 25,
 'hair drier': 89,
 'handbag': 31,
 'horse': 19,
 'hot dog': 58,
 'keyboard': 76,
 'kite': 38,
 'knife': 49,
 'laptop': 73,
 'microwave': 78,
 'motorcycle': 4,
 'mouse': 74,
 'orange': 55,
 'oven': 79,
 'parking meter': 14,
 'person': 1,
 'pizza': 59,
 'potted plant': 64,
 'refrigerator': 82,
 'remote': 75,
 'sandwich': 54,
 'scissors': 87,
 'sheep': 20,
 'sink': 81,
 'skateboard': 41,
 'skis': 35,
 'snowboard': 36,
 'spoon': 50,
 'sports ball': 37,
 'stop sign': 13,
 'suitcase': 33,
 'surfboard': 42,
 'teddy bear': 88,
 'tennis racket': 43,
 'tie': 32,
 'toaster': 80,
 'toilet': 70,
 'toothbrush': 90,
 'traffic light': 10,
 'train': 7,
 'truck': 8,
 'tv': 72,
 'umbrella': 28,
 'vase': 86,
 'wine glass': 46,
 'zebra': 24
}

labels = list(MAPPING.keys())

In [ ]:
dataset["train"][0]["captions"][0]

'A man with a red helmet on a small moped on a dirt road. '

In [ ]:
def filter_func(record):
    tokens = record['captions'][0]

    for label in labels:
        if label in tokens:
            return True

    return False

filtered_dataset = dataset.filter(filter_func)

Filter:   0%|          | 0/118287 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
captions = [x[0] for x in filtered_dataset["validation"]["captions"]]

In [ ]:
!mkdir -p data/train

In [ ]:
# result formatter

res = """a young boy holding a tennis racket in a park
a tennis racket on a tennis court
a close-up of a tennis racket
a tennis racket with a colorful grip
a tennis racket beside a tennis ball
a tennis racket in a professional player's hands
a tennis racket in a sports store
a tennis racket with fresh strings
a tennis racket and a tennis net
a tennis racket on a clay court
a tennis racket with a city's skyline in the background
a vintage tennis racket
a tennis racket covered in stickers
a tennis racket used as a prop
a tennis racket and a broken string
a tennis racket autographed by a famous player
a tennis racket in a display case
a tennis racket being restrung
a tennis racket with a custom design
a tennis racket next to a water bottle
a tennis racket in a child's play area
a tennis racket and a pair of shoes
a tennis racket in a locker room
a tennis racket with a cracked frame
a tennis racket used in a photoshoot
a tennis racket in a modern apartment
a tennis racket and a training manual
a tennis racket in a film set
a tennis racket on a rainy day
a tennis racket and a sports bag
a tennis racket being held by a silhouette
a tennis racket in a dark room
a tennis racket next to a towel
a tennis racket lying on the ground
a tennis racket and a sunset
a tennis racket on a picnic blanket
a tennis racket next to a stack of books
a tennis racket in a field of wildflowers
a tennis racket with a ribbon tied around it
a tennis racket in a child's toy collection
a tennis racket at the beach
a tennis racket and a kite in the sky
a tennis racket in a school gym
a tennis racket with a superhero logo
a tennis racket in a garden
a tennis racket at a carnival
a tennis racket with a ferris wheel in the background
a tennis racket on a boat
a tennis racket in a forest
a tennis racket with a butterfly landing on it
a tennis racket in the sand
a tennis racket next to a campfire
a tennis racket with fireworks in the background
a tennis racket on a mountain trail
a tennis racket in a music festival crowd
a tennis racket at a construction site
a tennis racket in a library
a tennis racket in a science lab
a tennis racket with a rainbow in the background
a tennis racket in a desert
a tennis racket at a wedding
a tennis racket and a bouquet of flowers
a tennis racket in an art studio
a tennis racket at a dog park
a tennis racket and a playground slide
a tennis racket on a city street
a tennis racket in a car trunk
a tennis racket at a swimming pool
a tennis racket next to a snowman
a tennis racket at a ski resort
a tennis racket in a coffee shop
a tennis racket on a college campus
a tennis racket with a graduation cap
a tennis racket in a grocery store
a tennis racket in an ice cream shop
a tennis racket with a lemonade stand
a tennis racket at a street fair
a tennis racket in a museum
a tennis racket at a train station
a tennis racket in a hotel lobby
a tennis racket on a rooftop
a tennis racket with a telescope
a tennis racket at a science fair
a tennis racket in a hospital
a tennis racket in a firefighter's break room
a tennis racket at a police station
a tennis racket at a city hall
a tennis racket in a flower shop
a tennis racket in a bakery
a tennis racket in a barber shop
a tennis racket in a pet store
a tennis racket at an amusement park
a tennis racket with a roller coaster
a tennis racket at a halloween party
a tennis racket in a haunted house
a tennis racket at a holiday parade
a tennis racket with christmas lights
a tennis racket at a new year's eve party
a tennis racket with confetti
a tennis racket in a superhero-themed room
a tennis racket with an astronaut's helmet"""

res_list = res.split("\n")

for i in res_list:
    str_ = i
    print(f'  "{str_}",')

# len(res_list)

  "a young boy holding a tennis racket in a park",
  "a tennis racket on a tennis court",
  "a close-up of a tennis racket",
  "a tennis racket with a colorful grip",
  "a tennis racket beside a tennis ball",
  "a tennis racket in a professional player's hands",
  "a tennis racket in a sports store",
  "a tennis racket with fresh strings",
  "a tennis racket and a tennis net",
  "a tennis racket on a clay court",
  "a tennis racket with a city's skyline in the background",
  "a vintage tennis racket",
  "a tennis racket covered in stickers",
  "a tennis racket used as a prop",
  "a tennis racket and a broken string",
  "a tennis racket autographed by a famous player",
  "a tennis racket in a display case",
  "a tennis racket being restrung",
  "a tennis racket with a custom design",
  "a tennis racket next to a water bottle",
  "a tennis racket in a child's play area",
  "a tennis racket and a pair of shoes",
  "a tennis racket in a locker room",
  "a tennis racket with a cracked frame

# Unrealistic prompts for image generation

In [ ]:
"""
unrealistic prompts by GPT 3.5
- giraffe, 412 prompts
    - https://chatgpt.com/share/f466eb22-4908-4fc1-ad8a-e318494c7f40
    - https://chatgpt.com/share/5f76c053-4ce6-4766-9699-22d541b36abf
    - https://chatgpt.com/share/41d35611-d59f-477c-aada-b6ae4c6ff194
- zebra, 401 prompts
    - https://chatgpt.com/share/cc3ed734-7a51-4bac-bd93-1445730c58de
    - https://chatgpt.com/share/ad85acd1-0073-4c59-a448-29a1eb8991c2
    - https://chatgpt.com/share/d8dd2d72-ce9a-4330-88c5-77d1f9f441d9
- bear
    - https://chatgpt.com/share/7e8691b0-b69a-4581-8c3e-78a298f697a7
- sheep
    - https://chatgpt.com/share/c68c3ef5-6400-48f5-ad7a-528b5196d215
- cat
    - https://chatgpt.com/share/e813f52a-ebc9-483b-9060-1447381d3fb2
- dog
    - https://chatgpt.com/share/770cb81d-f472-499b-b1dd-796e764e659c
- horse
    - https://chatgpt.com/share/733e0134-7729-45ad-8345-e6ba2a155f9c
- cow
    - https://chatgpt.com/share/c3aeac48-2c3f-452c-910d-b6459cf33f53
- elephant
    - https://chatgpt.com/share/d97ce0ce-af0a-40f1-9fc8-ef7dc39d44eb
- bird (by GPT-4o)
    - https://chatgpt.com/share/ab139175-b906-41ff-b7e6-860408aa2960
"""

unrealistic_prompts = [
  "A giraffe dressed in an astronaut suit standing on the rings of Saturn with a galaxy in the background",
  "A giraffe with a crystalline body walking through a forest of giant gemstones under a full moon",
  "A giraffe in a medieval queen's gown sitting on a throne made of flowers in a fairy tale castle",
  "A giraffe with bioluminescent patterns swimming in a sea of liquid light with glowing jellyfish",
  "A giraffe standing on top of a floating iceberg in the middle of a glowing, frozen ocean",
  "A giraffe dressed as an ancient Greek philosopher lecturing in a grand hall filled with marble statues",
  "A giraffe with a body made of rainbow-colored smoke drifting through a surreal dreamscape",
  "A giraffe in a futuristic racing suit piloting a hovercraft through a neon-lit desert",
  "A giraffe with a body covered in neon tattoos standing in a dark alley in a cyberpunk city",
  "A giraffe wearing a magician's outfit pulling stars out of a top hat in a mystical forest",
  "A giraffe in a 1920s flapper dress dancing on a rooftop under a sky filled with fireworks",
  "A giraffe with mechanical wings flying over a steampunk city filled with airships",
  "A giraffe standing in a field of giant glowing dandelions under a sky full of swirling auroras",
  "A giraffe dressed as an ancient Roman emperor standing in a colosseum with a crowd of cheering aliens",
  "A giraffe with a body made of lava standing in a volcanic landscape with erupting volcanoes",
  "A giraffe in a futuristic suit of armor holding a plasma sword in a sci-fi battlefield",
  "A giraffe standing on a floating platform in a city made of glass and light",
  "A giraffe wearing a detective's trench coat investigating a mystery in a noir-style city",
  "A giraffe with a body made of vines and flowers standing in an enchanted forest with glowing fireflies",
  "A giraffe in a Victorian-era scientist's lab with floating test tubes and glowing potions",
  "A giraffe dressed as a pirate standing on the deck of a ghost ship sailing through a sea of stars",
  "A giraffe with a body covered in celestial constellations standing in the middle of a cosmic nebula",
  "A giraffe in a samurai's armor meditating under a waterfall of glowing water",
  "A giraffe with wings made of light flying through a futuristic city with floating skyscrapers",
  "A giraffe standing in a field of giant glowing mushrooms under a sky filled with shooting stars",
  "A giraffe in a medieval knight's armor riding a horse made of fire through a battlefield",
  "A giraffe with a body made of ice and snow standing in a frozen castle with ice sculptures",
  "A giraffe in a futuristic pilot's suit standing on the deck of a spaceship with a view of the stars",
  "A giraffe with a body covered in glowing tattoos standing in a dark alley in a cyberpunk city",
  "A giraffe wearing a wizard's robe casting a spell in a cave filled with glowing crystals",
  "A giraffe dressed as an ancient Egyptian pharaoh standing in a temple with floating hieroglyphs",
  "A giraffe with a body made of water standing in an underwater city with glowing sea creatures",
  "A giraffe in a tuxedo playing a grand piano on a stage surrounded by floating candles",
  "A giraffe standing in a field of giant sunflowers with a sky filled with swirling galaxies",
  "A giraffe in a futuristic suit surfing on a wave of neon lights through a cityscape",
  "A giraffe with a body made of fire standing in a volcanic landscape with erupting geysers",
  "A giraffe in a futuristic suit standing on a bridge made of light over a cityscape",
  "A giraffe dressed as a samurai standing in a bamboo forest with glowing fireflies",
  "A giraffe with a body made of neon lights standing in a cyberpunk city at night",
  "A giraffe in an astronaut suit floating in zero gravity inside a futuristic space station",
  "A giraffe with a holographic body playing a grand piano in a neon-lit cyberpunk nightclub",
  "A giraffe dressed as a knight holding a glowing sword in a field of giant flowers",
  "A giraffe standing on a bridge made of stars over a river of flowing galaxies",
  "A giraffe with wings made of stained glass flying through a cathedral filled with light",
  "A giraffe standing in a snowy landscape with northern lights and floating icebergs in the sky",
  "A giraffe in a tuxedo playing a grand piano on top of a skyscraper in a rainstorm",
  "A giraffe with a body made of fire standing in a volcanic landscape with erupting geysers",
  "A giraffe in a futuristic suit surfing on a wave of neon lights through a cityscape",
  "A giraffe in a Roman gladiator's armor standing in the middle of a colosseum with an alien crowd",
  "A giraffe with a body made of water standing in an underwater city with glowing sea creatures",
  "A giraffe wearing a tuxedo playing a violin on a stage surrounded by floating candles",
  "A giraffe standing in a field of giant sunflowers with a sky filled with swirling galaxies",
  "A giraffe in a medieval knight's armor holding a glowing sword in a crystal cave",
  "A giraffe with butterfly wings flying over a futuristic city with floating buildings",
  "A giraffe dressed as an ancient Egyptian pharaoh standing in a temple with floating hieroglyphs",
  "A giraffe made of ice standing in a frozen landscape with glowing auroras",
  "A giraffe in a futuristic suit piloting a spaceship through a field of asteroids",
  "A giraffe wearing a steampunk outfit standing on a floating island with airships",
  "A giraffe with a body made of glass standing in a forest of glowing crystal trees",
  "A giraffe in a Victorian-era dress standing in a garden with floating lanterns",
  "A giraffe with holographic wings flying through a futuristic cityscape at night",
  "A giraffe dressed as a samurai standing in a bamboo forest with glowing fireflies",
  "A giraffe with a body made of neon lights standing in a cyberpunk city at night",
  "A giraffe in an astronaut suit standing on a floating platform in a space station",
  "A giraffe with a body made of vines standing in a jungle with glowing plants",
  "A giraffe in a futuristic suit standing on a bridge made of light over a cityscape",
  "A giraffe with a body covered in glowing tattoos standing in a dark forest",
  "A giraffe in a tuxedo playing a grand piano in a room filled with floating musical notes",
  "A giraffe standing in a field of giant glowing flowers under a sky filled with shooting stars",
  "A giraffe with a body made of fire standing in a volcanic landscape with erupting geysers",
  "A giraffe in a futuristic suit surfing on a wave of neon lights through a cityscape",
  "A giraffe in a Roman gladiator's armor standing in the middle of a colosseum with an alien crowd",
  "A giraffe with a body made of water standing in an underwater city with glowing sea creatures",
  "A giraffe wearing a tuxedo playing a violin on a stage surrounded by floating candles",
  "A giraffe standing in a field of giant sunflowers with a sky filled with swirling galaxies",
  "A giraffe in a medieval knight's armor holding a glowing sword in a crystal cave",
  "A giraffe with butterfly wings flying over a futuristic city with floating buildings",
  "A giraffe dressed as an ancient Egyptian pharaoh standing in a temple with floating hieroglyphs",
  "A giraffe made of ice standing in a frozen landscape with glowing auroras",
  "A giraffe in a futuristic suit piloting a spaceship through a field of asteroids",
  "A giraffe wearing a steampunk outfit standing on a floating island with airships",
  "A giraffe with a body made of glass standing in a forest of glowing crystal trees",
  "A giraffe in a Victorian-era dress standing in a garden with floating lanterns",
  "A giraffe with holographic wings flying through a futuristic cityscape at night",
  "A giraffe dressed as a samurai standing in a bamboo forest with glowing fireflies",
  "A giraffe with a body made of neon lights standing in a cyberpunk city at night",
  "A giraffe in an astronaut suit standing on a floating platform in a space station",
  "A giraffe with a body made of vines standing in a jungle with glowing plants",
  "A giraffe in a futuristic suit standing on a bridge made of light over a cityscape",
  "A giraffe with a body covered in glowing tattoos standing in a dark forest",
  "A giraffe in a tuxedo playing a grand piano in a room filled with floating musical notes",
  "A giraffe standing in a field of giant glowing flowers under a sky filled with shooting stars",
  "A giraffe wearing a space suit on the moon with a city skyline in the background",
  "A giraffe riding a unicycle on a tightrope between two skyscrapers at sunset",
  "A giraffe with butterfly wings flying over a medieval castle surrounded by a rainbow",
  "A giraffe in a Victorian-era dress sitting at a grand piano in a lavish ballroom",
  "A giraffe made of glass reflecting a kaleidoscope of colors in a crystal forest",
  "A giraffe reading a book under a glowing mushroom in an enchanted forest at night",
  "A giraffe dressed as a samurai standing on a cherry blossom tree branch during a thunderstorm",
  "A giraffe surfing a massive wave made of molten lava in a fiery ocean",
  "A giraffe with a cybernetic body playing a futuristic violin in a neon-lit city",
  "A giraffe in a superhero costume flying through the clouds above a futuristic metropolis",
  "A giraffe standing on a floating island covered in bioluminescent plants in the sky",
  "A giraffe dressed as a pirate captain steering a ship made of clouds through a stormy sky",
  "A giraffe painting a surreal landscape on an easel in the middle of a desert with floating cacti",
  "A giraffe with glowing runes on its body walking through an ancient temple with floating orbs",
  "A giraffe made entirely of clockwork gears standing in a steampunk cityscape with airships",
  "A giraffe wearing a wizard's robe casting a spell in a cave filled with glowing crystals",
  "A giraffe in an astronaut suit floating in zero gravity inside a futuristic space station",
  "A giraffe with a holographic body playing a grand piano in a neon-lit cyberpunk nightclub",
  "A giraffe dressed as a knight holding a glowing sword in a field of giant flowers",
  "A giraffe standing on a bridge made of stars over a river of flowing galaxies",
  "A giraffe with wings made of stained glass flying through a cathedral filled with light",
  "A giraffe standing in a snowy landscape with northern lights and floating icebergs in the sky",
  "A giraffe in a tuxedo playing a grand piano on top of a skyscraper in a rainstorm",
  "A giraffe with a body made of fire standing in a volcanic landscape with erupting geysers",
  "A giraffe in an elaborate masquerade mask dancing in a hall of mirrors filled with smoke",
  "A giraffe standing in a field of giant glowing mushrooms under a sky filled with shooting stars",
  "A giraffe with a body covered in bioluminescent tattoos standing in a dark forest",
  "A giraffe in a futuristic suit surfing on a wave of neon lights through a cityscape",
  "A giraffe in a Roman gladiator's armor standing in the middle of a colosseum with an alien crowd",
  "A giraffe with a body made of water standing in an underwater city with glowing sea creatures",
  "A giraffe wearing a tuxedo playing a violin on a stage surrounded by floating candles",
  "A giraffe standing in a field of giant sunflowers with a sky filled with swirling galaxies",
  "A giraffe in a medieval knight's armor holding a glowing sword in a crystal cave",
  "A giraffe with butterfly wings flying over a futuristic city with floating buildings",
  "A giraffe dressed as an ancient Egyptian pharaoh standing in a temple with floating hieroglyphs",
  "A giraffe made of ice standing in a frozen landscape with glowing auroras",
  "A giraffe in a futuristic suit piloting a spaceship through a field of asteroids",
  "A giraffe wearing a steampunk outfit standing on a floating island with airships",
  "A giraffe with a body made of glass standing in a forest of glowing crystal trees",
  "A giraffe in a Victorian-era dress standing in a garden with floating lanterns",
  "A giraffe with holographic wings flying through a futuristic cityscape at night",
  "A giraffe dressed as a samurai standing in a bamboo forest with glowing fireflies",
  "A giraffe with a body made of neon lights standing in a cyberpunk city at night",
  "A giraffe in an astronaut suit standing on a floating platform in a space station",
  "A giraffe with a body made of vines standing in a jungle with glowing plants",
  "A giraffe in a futuristic suit standing on a bridge made of light over a cityscape",
  "A giraffe with a body covered in glowing tattoos standing in a dark forest",
  "A giraffe in a tuxedo playing a grand piano in a room filled with floating musical notes",
  "A giraffe standing in a field of giant glowing flowers under a sky filled with shooting stars",
  "A giraffe with a body made of fire standing in a volcanic landscape with erupting geysers",
  "A giraffe in a futuristic suit surfing on a wave of neon lights through a cityscape",
  "A giraffe in a Roman gladiator's armor standing in the middle of a colosseum with an alien crowd",
  "A giraffe with a body made of water standing in an underwater city with glowing sea creatures",
  "A giraffe wearing a tuxedo playing a violin on a stage surrounded by floating candles",
  "A giraffe standing in a field of giant sunflowers with a sky filled with swirling galaxies",
  "A giraffe in a medieval knight's armor holding a glowing sword in a crystal cave",
  "A giraffe with butterfly wings flying over a futuristic city with floating buildings",
  "A giraffe dressed as an ancient Egyptian pharaoh standing in a temple with floating hieroglyphs",
  "A giraffe made of ice standing in a frozen landscape with glowing auroras",
  "A giraffe in a futuristic suit piloting a spaceship through a field of asteroids",
  "A giraffe wearing a steampunk outfit standing on a floating island with airships",
  "A giraffe with a body made of glass standing in a forest of glowing crystal trees",
  "A giraffe in a Victorian-era dress standing in a garden with floating lanterns",
  "A giraffe with holographic wings flying through a futuristic cityscape at night",
  "A giraffe dressed as a samurai standing in a bamboo forest with glowing fireflies",
  "A giraffe with a body made of neon lights standing in a cyberpunk city at night",
  "A giraffe in an astronaut suit standing on a floating platform in a space station",
  "A giraffe with a body made of vines standing in a jungle with glowing plants",
  "A giraffe in a futuristic suit standing on a bridge made of light over a cityscape",
  "A giraffe with a body covered in glowing tattoos standing in a dark forest",
  "A giraffe in a tuxedo playing a grand piano in a room filled with floating musical notes",
  "A giraffe standing in a field of giant glowing flowers under a sky filled with shooting stars",
  "A giraffe with a body made of fire standing in a volcanic landscape with erupting geysers",
  "A giraffe in a futuristic suit surfing on a wave of neon lights through a cityscape",
  "A giraffe in a Roman gladiator's armor standing in the middle of a colosseum with an alien crowd",
  "A giraffe with a body made of water standing in an underwater city with glowing sea creatures",
  "A giraffe wearing a tuxedo playing a violin on a stage surrounded by floating candles",
  "A giraffe standing in a field of giant sunflowers with a sky filled with swirling galaxies",
  "A giraffe in a medieval knight's armor holding a glowing sword in a crystal cave",
  "A giraffe with butterfly wings flying over a futuristic city with floating buildings",
  "A giraffe dressed as an ancient Egyptian pharaoh standing in a temple with floating hieroglyphs",
  "A giraffe made of ice standing in a frozen landscape with glowing auroras",
  "A giraffe in a futuristic suit piloting a spaceship through a field of asteroids",
  "A giraffe wearing a steampunk outfit standing on a floating island with airships",
  "A giraffe with a body made of glass standing in a forest of glowing crystal trees",
  "A giraffe in a Victorian-era dress standing in a garden with floating lanterns",
  "A giraffe with holographic wings flying through a futuristic cityscape at night",
  "A giraffe dressed as a samurai standing in a bamboo forest with glowing fireflies",
  "A giraffe with a body made of neon lights standing in a cyberpunk city at night",
  "A giraffe in an astronaut suit standing on a floating platform in a space station",
  "A giraffe with a body made of vines standing in a jungle with glowing plants",
  "A giraffe in a futuristic suit standing on a bridge made of light over a cityscape",
  "A giraffe with a body covered in glowing tattoos standing in a dark forest",
  "A giraffe in a tuxedo playing a grand piano in a room filled with floating musical notes",
  "A giraffe standing in a field of giant glowing flowers under a sky filled with shooting stars",
  "A giraffe with a body made of fire standing in a volcanic landscape with erupting geysers",
  "A giraffe in a futuristic suit surfing on a wave of neon lights through a cityscape",
  "A giraffe in a Roman gladiator's armor standing in the middle of a colosseum with an alien crowd",
  "A giraffe with a body made of water standing in an underwater city with glowing sea creatures",
  "A giraffe wearing a tuxedo playing a violin on a stage surrounded by floating candles",
  "A giraffe standing in a field of giant sunflowers with a sky filled with swirling galaxies",
  "A giraffe in a medieval knight's armor holding a glowing sword in a crystal cave",
  "A giraffe with butterfly wings flying over a futuristic city with floating buildings",
  "A giraffe dressed as an ancient Egyptian pharaoh standing in a temple with floating hieroglyphs",
  "A giraffe made of ice standing in a frozen landscape with glowing auroras",
  "A giraffe in a futuristic suit piloting a spaceship through a field of asteroids",
  "A giraffe wearing a steampunk outfit standing on a floating island with airships",
  "A giraffe with a body made of glass standing in a forest of glowing crystal trees",
  "A giraffe in a Victorian-era dress standing in a garden with floating lanterns",
  "A giraffe with holographic wings flying through a futuristic cityscape at night",
  "A giraffe dressed as a samurai standing in a bamboo forest with glowing fireflies",
  "A giraffe with a body made of neon lights",
  "A giraffe navigating through a maze of mirrors, each reflecting a different distorted image of itself.",
  "A giraffe attempting to play a grand piano with its elongated neck, struggling to reach the keys.",
  "A giraffe in a submarine, peering out of a porthole at underwater cities inhabited by marine creatures.",
  "A giraffe balancing on a tightrope stretched between two skyscrapers in a bustling cityscape.",
  "A giraffe participating in a spacewalk outside a space station, tethered to it while observing distant galaxies.",
  "A giraffe attending a formal dinner party, trying to fit into a room with exceptionally low ceilings.",
  "A giraffe disguised as a human, wearing a trench coat and sunglasses while walking through a crowded market.",
  "A giraffe riding a unicycle through a forest of towering sequoia trees.",
  "A giraffe conducting an orchestra, standing on a podium and using its neck to direct musicians.",
  "A giraffe working as a firefighter, using its height to rescue people from a burning high-rise building.",
  "A giraffe navigating a complex obstacle course designed for humans, including hurdles and climbing walls.",
  "A giraffe serving as a lifeguard at a crowded beach, scanning the horizon with binoculars from its lookout tower.",
  "A giraffe operating a hot air balloon, peering over the edge of the basket at a landscape dotted with ancient ruins.",
  "A giraffe participating in a high-speed motorcycle race, leaning into sharp turns on a custom-built bike.",
  "A giraffe attending a rock concert, standing tall among a cheering crowd of enthusiastic fans.",
  "A giraffe working as a window washer on the exterior of a skyscraper, using its tongue to clean each window.",
  "A giraffe exploring a dense jungle, encountering exotic animals and vibrant plant life.",
  "A giraffe running a marathon, navigating through a diverse urban landscape with cheering spectators.",
  "A giraffe attempting to perform ballet, gracefully stretching its legs and neck in a dance studio.",
  "A giraffe trying to blend in at a camouflage workshop, surrounded by experts in disguise.",
  "A giraffe attending a space launch, observing a rocket blasting off into the starry sky.",
  "A giraffe participating in a synchronized swimming competition, elegantly moving in perfect harmony with other swimmers.",
  "A giraffe working as a detective, examining clues and interrogating suspects in a mysterious case.",
  "A giraffe operating a hot dog stand at a busy carnival, serving customers from its elevated position.",
  "A giraffe attending a futuristic fashion show, wearing avant-garde clothing designed specifically for its long neck.",
  "A giraffe participating in a medieval jousting tournament, riding a specially adapted horse and wielding a lance.",
  "A giraffe exploring an underwater cave, illuminated by bioluminescent creatures and submerged treasures.",
  "A giraffe training as a circus acrobat, mastering aerial flips and balancing acts under the big top.",
  "A giraffe serving as a tour guide in a sprawling museum, providing insightful commentary on ancient artifacts.",
  "A giraffe participating in a demolition derby, driving a heavily armored vehicle and crashing into opponents.",
  "A giraffe working as a skyscraper window cleaner, suspended from a harness and meticulously scrubbing glass.",
  "A giraffe attending a magic show, amazed by illusions and tricks performed on stage.",
  "A giraffe participating in a cooking competition, using its elongated tongue to taste and prepare exotic dishes.",
  "A giraffe disguised as a superhero, fighting crime and saving civilians from perilous situations.",
  "A giraffe attending a film premiere, walking down the red carpet and posing for paparazzi.",
  "A giraffe serving as a skyscraper painter, using a long-handled brush to apply vibrant colors to a building facade.",
  "A giraffe attending a winter sports competition, maneuvering through a slalom course on skis.",
  "A giraffe working as a skyscraper architect, overseeing the construction of a towering building.",
  "A giraffe participating in a deep-sea diving expedition, exploring shipwrecks and underwater caves.",
  "A giraffe attending a futuristic technology expo, interacting with cutting-edge gadgets and inventions.",
  "A giraffe participating in a competitive eating contest, devouring oversized portions of food with its dexterous tongue.",
  "A giraffe serving as a judge in a beauty pageant, evaluating contestants on poise and elegance.",
  "A giraffe working as a flight attendant on a commercial airliner, assisting passengers with its unique height.",
  "A giraffe attending a wildlife photography workshop, posing majestically for cameras in its natural habitat.",
  "A giraffe participating in a space colonization mission, adapting to life on a distant planet.",
  "A giraffe working as a rescue diver, assisting in underwater search and rescue operations.",
  "A giraffe attending a carnival parade, adorned in colorful costumes and dancing through the streets.",
  "A giraffe participating in a medieval renaissance fair, engaging in jousting tournaments and archery contests.",
  "A giraffe working as a stand-up comedian, delivering jokes with its characteristic wit and humor.",
  "A giraffe attending a futuristic robot battle, observing mechanical giants clash in an arena.",
  "A giraffe participating in a treasure hunt, following clues through a labyrinthine maze of tunnels and chambers.",
  "A giraffe serving as a wildlife conservationist, studying endangered species and their habitats.",
  "A giraffe attending a high-stakes poker game, carefully considering its next move among seasoned players.",
  "A giraffe participating in a marathon relay race, sprinting alongside teammates through varied terrain.",
  "A giraffe working as a skyscraper window installer, securing glass panels from dizzying heights.",
  "A giraffe attending a masquerade ball, wearing an elaborate mask and mingling with masked guests.",
  "A giraffe participating in a ballet performance, executing graceful leaps and pirouettes on stage.",
  "A giraffe working as a chef in a gourmet restaurant, preparing and presenting exquisite dishes.",
  "A giraffe attending an art exhibition, admiring paintings and sculptures from renowned artists.",
  "A giraffe participating in a cross-country road trip, exploring diverse landscapes and landmarks.",
  "A giraffe serving as a tour guide in a historic city, sharing anecdotes and history with curious visitors.",
  "A giraffe riding a unicycle on a tightrope between two hot air balloons over a rainbow-colored waterfall.",
  "A giraffe playing a grand piano on top of a mountain during a lightning storm.",
  "A giraffe dressed as a knight jousting with a dragon in a medieval castle courtyard.",
  "A giraffe conducting an orchestra of penguins in an underwater coral reef.",
  "A giraffe surfing a massive wave alongside dolphins under a glowing aurora borealis.",
  "A giraffe painting a mural on the side of a skyscraper while suspended by a helicopter.",
  "A giraffe wearing a tuxedo and performing magic tricks on a floating stage in space.",
  "A giraffe driving a convertible car on a racetrack made of candy canes in a cotton candy landscape.",
  "A giraffe competing in a ballroom dance competition with a robot partner in a futuristic city.",
  "A giraffe delivering a speech at a United Nations assembly with aliens and robots in the audience.",
  "A giraffe flying a vintage biplane through a series of giant floating rings in a steampunk sky.",
  "A giraffe practicing yoga on a floating island surrounded by glowing jellyfish in the sky.",
  "A giraffe ice skating on a frozen lake with polar bears and seals under the Northern Lights.",
  "A giraffe baking a giant cake in a kitchen with walls made of gingerbread and candy.",
  "A giraffe scuba diving with a group of mermaids in a vibrant, underwater city.",
  "A giraffe riding a roller coaster made of stars through a galaxy with planets and comets.",
  "A giraffe as a rockstar performing at a concert with electric guitars and pyrotechnics on a floating stage.",
  "A giraffe dressed as an astronaut planting a flag on a colorful alien planet with two suns.",
  "A giraffe herding a flock of unicorns in a meadow filled with giant flowers and butterflies.",
  "A giraffe riding a bicycle through a city made entirely of glass and crystal structures.",
  "A giraffe playing chess with an octopus in a submerged submarine surrounded by fish.",
  "A giraffe as a pirate captain steering a ship made of clouds through a sky filled with floating islands.",
  "A giraffe hosting a tea party with talking animals in a whimsical, oversized garden.",
  "A giraffe skiing down a volcano with lava flowing alongside in a tropical paradise.",
  "A giraffe performing acrobatics on a trapeze in a circus tent with an audience of mythical creatures.",
  "A giraffe stargazing with a telescope on the back of a giant turtle floating in space.",
  "A giraffe operating a giant robot in a futuristic cityscape during a neon-lit night.",
  "A giraffe painting a self-portrait on an easel in the middle of a dense, enchanted forest.",
  "A giraffe participating in a Formula 1 race on a track suspended in the clouds.",
  "A giraffe DJing a dance party on a beach with bioluminescent waves under a full moon.",
  "A giraffe climbing a giant beanstalk that reaches into a sky filled with floating castles.",
  "A giraffe playing soccer on the moon with an astronaut team under a distant Earth.",
  "A giraffe hosting a cooking show in a kitchen where the ingredients float in zero gravity.",
  "A giraffe exploring an ancient, underwater temple with glowing hieroglyphics.",
  "A giraffe piloting a futuristic hovercraft through a city with holographic advertisements.",
  "A giraffe performing in a ballet with fairies and elves in an enchanted theater.",
  "A giraffe playing drums in a rock band with other jungle animals on a stage in the Amazon rainforest.",
  "A giraffe dressed as a wizard casting spells in a magical library with floating books.",
  "A giraffe racing a cheetah on a racetrack made of stars in a cosmic arena.",
  "A giraffe playing a harp on a cloud surrounded by musical notes in a pastel sky.",
  "A giraffe building a sandcastle with a giant squid on a beach of crystal-clear water.",
  "A giraffe as a detective investigating a crime scene in a city made of neon lights and shadows.",
  "A giraffe as a superhero flying over a futuristic cityscape with a cape billowing behind.",
  "A giraffe sculpting a statue out of ice in an arctic landscape with penguins watching.",
  "A giraffe rowing a boat through a river of molten lava in a volcanic landscape.",
  "A giraffe attending a masquerade ball in a grand palace filled with chandeliers and mirrors.",
  "A giraffe harvesting glowing fruits from a tree in an otherworldly, bioluminescent forest.",
  "A giraffe playing a giant harp made of light beams in a celestial concert hall.",
  "A giraffe walking a tightrope between two skyscrapers in a bustling, futuristic metropolis.",
  "A giraffe piloting a spaceship through an asteroid field with alien planets in the background.",
  "A giraffe hosting a trivia game show in a studio with a diverse audience of extraterrestrials.",
  "A giraffe painting a mural on the Great Wall of China with scenes from space.",
  "A giraffe as a firefighter rescuing a cat from the top of a burning skyscraper.",
  "A giraffe playing basketball with robots on a court made of holograms in a neon-lit city.",
  "A giraffe fishing from a boat on a lake of liquid gold under a sky of swirling galaxies.",
  "A giraffe dressed as a Viking rowing a longship through a stormy sea with mythical sea creatures.",
  "A giraffe playing a flute on a floating island with magical, glowing plants and flowers.",
  "A giraffe leading a parade of colorful, animated toys through a whimsical, dreamlike city.",
  "A giraffe as an artist creating a masterpiece in a studio where the walls change color with the music.",
  "A giraffe skating on a giant record player with music notes floating around in a vibrant, retro setting.",
  "A giraffe playing tennis with a robot opponent on a court in the middle of a bustling, futuristic city.",
  "A giraffe exploring a labyrinth made of giant books in an endless, magical library.",
  "A giraffe performing stand-up comedy on stage with an audience of mythical creatures.",
  "A giraffe flying a kite shaped like a dragon in a field of giant, colorful flowers.",
  "A giraffe piloting a submarine through an underwater canyon filled with glowing, bioluminescent creatures.",
  "A giraffe as a samurai practicing sword fighting in a cherry blossom garden with floating lanterns.",
  "A giraffe racing a spaceship through a wormhole with planets and stars zooming by.",
  "A giraffe playing a grand organ in a cathedral made of crystal with light streaming through stained glass windows.",
  "A giraffe as an archaeologist uncovering ancient artifacts in a desert with pyramids and sphinxes.",
  "A giraffe as a scientist conducting experiments in a laboratory filled with futuristic technology.",
  "A giraffe snowboarding down a rainbow-colored slope in a magical, winter wonderland.",
  "A giraffe hosting a fashion show with models dressed in avant-garde, otherworldly outfits.",
  "A giraffe piloting a hot air balloon shaped like a giant bird through a sky filled with flying fish.",
  "A giraffe as a detective solving a mystery in a Victorian-era city with cobblestone streets and gas lamps.",
  "A giraffe playing a saxophone in a jazz club with a diverse crowd of alien patrons.",
  "A giraffe as a lifeguard on a beach with waves made of light and sand that glows in the dark.",
  "A giraffe planting a garden on the rooftop of a skyscraper in a city filled with greenery and flowers.",
  "A giraffe playing hide-and-seek with fairies in a forest with giant mushrooms and sparkling streams.",
  "A giraffe performing in a circus with trapeze artists and acrobats in a tent filled with swirling colors.",
  "A giraffe driving a monster truck over a track made of giant candy and sweets in a whimsical landscape.",
  "A giraffe as a blacksmith forging a magical sword in a workshop filled with enchanted tools and glowing runes.",
  "A giraffe practicing martial arts on a mountaintop with a view of a vast, surreal landscape.",
  "A giraffe playing in a symphony orchestra with musicians from different eras and worlds.",
  "A giraffe exploring a city made of gears and cogs in a steampunk universe.",
  "A giraffe flying a magic carpet over a city of floating lanterns and shimmering buildings.",
  "A giraffe painting the night sky with a brush, creating stars and constellations.",
  "A giraffe as a captain of a spaceship navigating through a nebula with vibrant colors and swirling gases.",
  "A giraffe as a gardener tending to a garden of giant, carnivorous plants in a mystical forest.",
  "A giraffe skiing on a slope made of glittering crystals in an enchanted, icy realm.",
  "A giraffe playing the drums in a band with other animals on a stage in the middle of a jungle.",
  "A giraffe conducting an orchestra of animated musical instruments in a fantastical concert hall.",
  "A giraffe exploring an ancient, haunted castle with ghosts and glowing, spectral figures.",
  "A giraffe piloting a hoverboard through a city with buildings made of light and energy.",
  "A giraffe playing hopscotch on a sidewalk with giant, colorful blocks in a dreamlike city.",
  "A giraffe baking in a kitchen where the ingredients are floating and the utensils are animated.",
  "A giraffe ice fishing on a frozen lake under the shimmering lights of the aurora borealis.",
  "A giraffe walking through a futuristic city with robotic legs and cybernetic enhancements.",
  "A giraffe swinging from vine to vine in a dense jungle filled with bioluminescent plants.",
  "A giraffe dancing ballet on a stage made of clouds with a backdrop of a golden sunset.",
  "A giraffe piloting a spaceship shaped like a giant leaf through a forest of giant trees.",
  "A giraffe playing the violin on a cliffside overlooking a sea of stars and galaxies.",
  "A giraffe as a race car driver speeding through a track in a city made entirely of neon lights.",
  "A giraffe as a samurai meditating in a bamboo forest surrounded by floating lanterns.",
  "A giraffe riding a dragon through a sky filled with floating islands and ancient ruins.",
  "A giraffe sculpting a statue out of sand on a beach with crystal-clear waters and colorful fish.",
  "A giraffe competing in an Olympic swimming event in a pool suspended in the sky.",
  "A giraffe playing basketball on a court made of ice in a futuristic, icy city.",
  "A giraffe piloting a jetpack through a city with skyscrapers made of glass and steel.",
  "A giraffe performing a magic show with levitating objects and glowing, mystical symbols.",
  "A giraffe as a scientist exploring an alien planet with bizarre and colorful flora and fauna.",
  "A giraffe surfing on a wave made of glowing, bioluminescent water in a tropical paradise.",
  "A giraffe riding a motorcycle through a desert with sand dunes and ancient, buried statues.",
  "A giraffe playing the harp in a garden with flowers that change color with the music.",
  "A giraffe as a cowboy riding a mechanical bull in a futuristic, Wild West-themed city.",
  "A giraffe painting a landscape on a canvas that extends infinitely into the horizon.",
  "A giraffe hosting a TV show in a studio with holographic sets and animated characters.",
  "A giraffe exploring an underwater cave with glowing crystals and mysterious sea creatures.",
  "A giraffe as an astronaut floating in space with a view of Earth and distant galaxies.",
  "A giraffe conducting a train through a city made of clockwork and gears in a steampunk world.",
  "A giraffe playing a grand piano on a floating platform in the middle of a serene lake.",
  "A giraffe as a detective solving a mystery in a city with noir-style lighting and shadowy figures.",
  "A giraffe performing a trapeze act in a circus with acrobats and clowns in vibrant costumes.",
  "A giraffe racing a jet ski on a river made of liquid light in a futuristic landscape.",
  "A giraffe baking a cake in a kitchen where the ingredients float and mix themselves.",
  "A giraffe exploring an enchanted forest with trees that whisper secrets and glow with magic.",
  "A giraffe flying a glider over a landscape of floating islands and waterfalls.",
  "A giraffe as a superhero saving a city from a giant, robotic monster.",
  "A giraffe playing chess on a giant board with pieces that move and come to life.",
  "A giraffe driving a sports car on a racetrack suspended in the air with views of a sprawling city below.",
  "A giraffe conducting an orchestra of robots in a futuristic concert hall.",
  "A giraffe painting a mural on the side of a mountain with a view of a sprawling valley below.",
  "A giraffe surfing on a giant wave with dolphins and sea turtles in a vibrant, tropical ocean.",
  "A giraffe playing the drums in a band with other animals on a stage in the middle of a jungle.",
  "A giraffe as an explorer charting a map of an uncharted, mystical island.",
  "A giraffe hosting a cooking show in a kitchen with ingredients that float and utensils that move on their own.",
  "A giraffe performing a ballet on a stage made of ice in a winter wonderland.",
  "A giraffe as a pilot flying a plane through a sky filled with colorful, glowing clouds.",
  "A giraffe painting a mural on the side of a skyscraper with a view of a bustling city below.",
  "A giraffe playing a saxophone in a jazz club with a smoky, dimly lit atmosphere.",
  "A giraffe riding a roller coaster through a city made of neon lights and holograms.",
  "A giraffe as a scientist conducting experiments in a laboratory with floating, holographic screens.",
  "A giraffe playing tennis on a court suspended in the air with a view of a futuristic city below.",
  "A giraffe exploring an ancient, underwater temple with glowing hieroglyphics and mysterious artifacts.",
  "A giraffe as a firefighter rescuing a cat from the top of a burning building.",
  "A giraffe skiing down a mountain made of candy in a whimsical, fantastical landscape.",
  "A giraffe piloting a spaceship through a field of asteroids with distant stars and galaxies in the background.",
  "A giraffe conducting an orchestra in a grand concert hall with musicians from different eras and worlds.",
  "A zebra leading a herd of bicycles through a bustling city intersection.",
  "A zebra surfing on a lava wave in a volcanic eruption.",
  "A zebra dressed as a firefighter putting out a fire on a floating island.",
  "A zebra exploring a cave filled with glowing crystals and ancient artifacts.",
  "A zebra in a futuristic suit racing a hoverbike through a neon desert.",
  "A zebra performing a tightrope act between two hot air balloons.",
  "A zebra conducting a laser light show in a high-tech arena.",
  "A zebra dressed as a wizard brewing a potion in a magical forest.",
  "A zebra playing the violin on a floating stage surrounded by fireflies.",
  "A zebra piloting a futuristic boat through a city submerged in water.",
  "A zebra participating in a futuristic fashion show with holographic clothes.",
  "A zebra exploring a digital world with pixelated landscapes and characters.",
  "A zebra painting a portrait on the side of a colossal digital screen.",
  "A zebra dressed as an ancient warrior leading a holographic army.",
  "A zebra in a spacesuit repairing a satellite in orbit around Earth.",
  "A zebra performing a circus act with floating hoops of light.",
  "A zebra skiing down a mountain made of holographic grids.",
  "A zebra exploring a futuristic library with floating books and digital shelves.",
  "A zebra conducting an experiment in a zero-gravity science lab.",
  "A zebra in a high-tech racing suit competing in a jetpack race.",
  "A zebra dressed as a medieval knight guarding a futuristic castle.",
  "A zebra performing a dance routine on a stage made of liquid light.",
  "A zebra surfing on a wave of stars in outer space.",
  "A zebra exploring a haunted forest with ghostly apparitions and glowing plants.",
  "A zebra painting a mural on the walls of a floating digital fortress.",
  "A zebra dressed as a samurai meditating in a cyberpunk dojo.",
  "A zebra operating a giant robotic arm in a high-tech factory.",
  "A zebra playing the harp on a stage surrounded by floating orbs of light.",
  "A zebra conducting a choir of holographic singers in a futuristic cathedral.",
  "A zebra skiing down a slope made of neon lights in a digital world.",
  "A zebra exploring an ancient temple with glowing runes and digital artifacts.",
  "A zebra dressed as a superhero rescuing a city from a giant robot.",
  "A zebra performing magic tricks on a floating platform above a futuristic city.",
  "A zebra piloting a spaceship through a wormhole in a neon-lit galaxy.",
  "A zebra painting a cosmic mural on the walls of a space station.",
  "A zebra dressed as an ancient Egyptian priest commanding holographic spirits.",
  "A zebra in a futuristic racing suit driving a light cycle through a digital grid.",
  "A zebra conducting a symphony in a concert hall made of liquid crystal.",
  "A zebra surfing on a wave of liquid metal in a high-tech landscape.",
  "A zebra exploring a virtual reality world with glowing landscapes and creatures.",
  "A zebra performing acrobatics on a stage made of floating light panels.",
  "A zebra skiing down a mountain made of rainbow-colored ice.",
  "A zebra dressed as a futuristic soldier battling holographic enemies.",
  "A zebra piloting a submarine through a sea of digital code.",
  "A zebra painting a surreal landscape on a canvas in a dreamlike virtual world.",
  "A zebra conducting a light show in a concert hall made of floating holograms.",
  "A zebra dressed as a samurai battling a giant robot in a cyberpunk city.",
  "A zebra performing a magic show on a floating island surrounded by digital clouds.",
  "A zebra exploring a futuristic zoo with robotic animals and holographic exhibits.",
  "A zebra in a spacesuit navigating an asteroid field with glowing crystals.",
  "A zebra conducting a laser orchestra in a digital concert hall.",
  "A zebra skiing down a slope made of holographic grids in a neon-lit world.",
  "A zebra dressed as a knight fighting a dragon made of digital code.",
  "A zebra piloting a spaceship through a galaxy of pulsating stars.",
  "A zebra painting a mural on the walls of a floating neon fortress.",
  "A zebra performing acrobatics on a tightrope suspended above a cyberpunk city.",
  "A zebra dressed as a magician performing tricks on a stage made of liquid light.",
  "A zebra exploring a haunted mansion with glowing ghosts and digital apparitions.",
  "A zebra conducting an orchestra in a concert hall made of floating light panels.",
  "A zebra skiing down a mountain of bioluminescent ice in a fantasy landscape.",
  "A zebra operating a giant mech in a high-tech battle arena.",
  "A zebra dressed as an astronaut floating in a neon-lit space station.",
  "A zebra painting a surreal landscape on a canvas in a zero-gravity environment.",
  "A zebra performing ballet on a stage surrounded by holographic butterflies.",
  "A zebra exploring a dense jungle with glowing plants and robotic animals.",
  "A zebra conducting a magic show on a floating island in a digital sky.",
  "A zebra dressed as a superhero battling a giant robot in a futuristic city.",
  "A zebra piloting a futuristic boat through a neon-lit city submerged in water.",
  "A zebra playing the violin on a floating stage surrounded by digital fireflies.",
  "A zebra conducting a science experiment in a zero-gravity lab.",
  "A zebra dressed as a samurai meditating under a waterfall of liquid light.",
  "A zebra performing a circus act with floating hoops of neon light.",
  "A zebra exploring a high-tech library with floating books and holographic shelves.",
  "A zebra skiing down a mountain of holographic grids in a digital landscape.",
  "A zebra conducting a light show in a concert hall made of liquid crystal.",
  "A zebra piloting a submarine through a sea of digital code.",
  "A zebra dressed as an ancient warrior leading a holographic army into battle.",
  "A zebra painting a cosmic mural on the walls of a futuristic space station.",
  "A zebra performing a dance routine on a stage made of floating light panels.",
  "A zebra exploring a virtual reality world with glowing landscapes and digital creatures.",
  "A zebra surfing on a wave of liquid metal in a high-tech digital world.",
  "A zebra dressed as a futuristic soldier battling holographic enemies in a cyber arena.",
  "A zebra conducting a laser light show in a concert hall made of floating holograms.",
  "A zebra skiing down a slope made of neon lights in a digital fantasy landscape.",
  "A zebra exploring an ancient temple with glowing runes and digital artifacts.",
  "A zebra dressed as a superhero rescuing a city from a giant robot in a cyberpunk setting.",
  "A zebra performing magic tricks on a floating platform above a futuristic metropolis.",
  "A zebra piloting a spaceship through a wormhole in a neon-lit galaxy.",
  "A zebra painting a cosmic mural on the walls of a space station orbiting Earth.",
  "A zebra dressed as an ancient Egyptian priest commanding holographic spirits in a digital temple.",
  "A zebra in a futuristic racing suit driving a light cycle through a neon grid.",
  "A zebra conducting a symphony in a concert hall made of liquid crystal.",
  "A zebra surfing on a wave of liquid metal in a neon-lit high-tech world.",
  "A zebra exploring a virtual reality world with glowing landscapes and holographic creatures.",
  "A zebra performing acrobatics on a stage made of floating light panels.",
  "A zebra skiing down a mountain made of rainbow-colored ice in a digital fantasy landscape.",
  "A zebra dressed as a futuristic soldier battling holographic enemies in a cyber arena.",
  "A zebra piloting a submarine through a sea of digital code in a high-tech landscape.",
  "A zebra painting a surreal landscape on a canvas in a dreamlike virtual reality world.",
  "A zebra conducting a light show in a concert hall made of floating holographic panels.",
  "A zebra dressed as a knight fighting a dragon made of digital code in a cyberpunk setting.",
  "A zebra piloting a spaceship through a galaxy of pulsating neon stars.",
  "A zebra painting a mural on the walls of a floating neon fortress in a digital sky.",
  "A zebra performing acrobatics on a tightrope suspended above a futuristic cityscape.",
  "A zebra wearing a spacesuit planting a flag on the moon.",
  "A zebra dressed as a detective investigating a crime scene in a futuristic city.",
  "A zebra playing a grand piano on top of a floating iceberg.",
  "A zebra riding a unicycle on a tightrope between two skyscrapers.",
  "A zebra painting a mural on the Great Wall of China.",
  "A zebra conducting an orchestra of robots in a digital concert hall.",
  "A zebra performing ballet on a stage made of clouds.",
  "A zebra dressed as a samurai facing a dragon in an ancient Japanese village.",
  "A zebra cooking a gourmet meal in a high-tech kitchen.",
  "A zebra in a diving suit exploring an underwater city.",
  "A zebra piloting a spaceship through a galaxy of neon stars.",
  "A zebra playing chess with a holographic opponent in a cyberpunk setting.",
  "A zebra as a superhero flying above a futuristic metropolis.",
  "A zebra in a magician’s outfit performing a levitation trick on a floating island.",
  "A zebra exploring an ancient Egyptian pyramid with glowing hieroglyphs.",
  "A zebra in traditional Viking armor rowing a ship through a stormy sea.",
  "A zebra attending a masquerade ball in a lavish, futuristic ballroom.",
  "A zebra as an astronaut floating in space among colorful nebulas.",
  "A zebra skiing down a rainbow-colored mountain slope.",
  "A zebra dressed as a medieval knight fighting a giant robotic spider.",
  "A zebra participating in a steampunk parade with mechanical floats.",
  "A zebra playing a harp on a stage surrounded by bioluminescent plants.",
  "A zebra in a cybernetic suit hacking into a virtual reality mainframe.",
  "A zebra performing a magic show on a flying carpet above a desert.",
  "A zebra dressed as an ancient Roman gladiator in a high-tech arena.",
  "A zebra conducting a science experiment in a zero-gravity laboratory.",
  "A zebra running a marathon on a track made of floating asteroids.",
  "A zebra exploring a haunted house with ghostly apparitions.",
  "A zebra in an elaborate Victorian outfit walking through a futuristic cityscape.",
  "A zebra participating in a futuristic car race through a neon-lit city.",
  "A zebra dressed as a pharaoh commanding an army of holographic soldiers.",
  "A zebra ice skating on a frozen lake under an aurora borealis.",
  "A zebra exploring a dense jungle with glowing flora and ancient ruins.",
  "A zebra painting a cosmic landscape on a canvas in outer space.",
  "A zebra dressed as a pilot flying a vintage biplane through a thunderstorm.",
  "A zebra navigating a labyrinth of floating platforms in the sky.",
  "A zebra in a futuristic suit battling a giant alien creature in a metropolis.",
  "A zebra dressed as a wizard casting spells in a mystical forest.",
  "A zebra playing basketball in an anti-gravity court.",
  "A zebra as a samurai meditating under a waterfall of liquid light.",
  "A zebra operating a giant mecha robot in a futuristic battlefield.",
  "A zebra hosting a cooking show in a floating kitchen studio.",
  "A zebra painting a mural on the side of a colossal spaceship.",
  "A zebra dressed as an explorer discovering a lost city in the clouds.",
  "A zebra participating in a dance-off in a neon-lit cyber club.",
  "A zebra performing a high-wire act above a futuristic cityscape.",
  "A zebra in a spacesuit gardening on a floating asteroid.",
  "A zebra in a futuristic racing suit driving a hovercar through a canyon.",
  "A zebra attending a royal ball in a castle made of crystal.",
  "A zebra playing a futuristic instrument in a concert hall made of light.",
  "A zebra in a traditional martial arts outfit sparring with a holographic opponent.",
  "A zebra surfing on a wave of liquid gold in a desert landscape.",
  "A zebra dressed as a pirate captain steering a spaceship through an asteroid field.",
  "A zebra performing acrobatics on a stage made of floating holograms.",
  "A zebra in an elaborate fantasy armor fighting a dragon in a digital world.",
  "A zebra conducting a symphony in a concert hall made entirely of ice.",
  "A zebra exploring a high-tech jungle with robotic animals and glowing plants.",
  "A zebra painting a portrait on a canvas suspended in mid-air.",
  "A zebra in a futuristic racing suit competing in a hoverboard race.",
  "A zebra dressed as a Victorian-era scientist experimenting with electricity.",
  "A zebra ice skating on a lake of molten lava.",
  "A zebra operating a steam-powered mech in a steampunk city.",
  "A zebra performing ballet on the edge of a skyscraper rooftop.",
  "A zebra in a spacesuit building a sandcastle on a distant planet.",
  "A zebra dressed as a knight battling a cybernetic dragon.",
  "A zebra skiing down a mountain of crystal in a fantasy landscape.",
  "A zebra exploring a city submerged underwater with bioluminescent buildings.",
  "A zebra painting a surreal landscape on a canvas in a dreamlike world.",
  "A zebra dressed as a ninja scaling a futuristic skyscraper.",
  "A zebra conducting a futuristic orchestra in a concert hall made of holograms.",
  "A zebra riding a hoverboard through a neon-lit cyberpunk city.",
  "A zebra dressed as an ancient Egyptian god commanding a fleet of spaceships.",
  "A zebra performing a magic trick on a floating platform in the sky.",
  "A zebra piloting a futuristic submarine through an alien ocean.",
  "A zebra playing a grand piano on a stage surrounded by floating lanterns.",
  "A zebra dressed as a gladiator battling robotic opponents in a coliseum.",
  "A zebra ice skating on a rink made of stardust in outer space.",
  "A zebra painting a mural on the side of a giant futuristic skyscraper.",
  "A zebra dressed as a samurai fighting a giant robot in a neon-lit city.",
  "A zebra performing acrobatics on a tightrope suspended between two mountains.",
  "A zebra conducting a symphony in a hall made of floating crystal shards.",
  "A zebra surfing on a wave of rainbow-colored water in a fantasy world.",
  "A zebra exploring a haunted mansion with ghostly holograms.",
  "A zebra dressed as a magician performing tricks in a floating castle.",
  "A zebra operating a futuristic mech in a high-tech warzone.",
  "A zebra playing a futuristic harp in a garden of bioluminescent flowers.",
  "A zebra dressed as an astronaut exploring a neon-lit alien city.",
  "A zebra skiing down a mountain made of diamonds in a surreal landscape.",
  "A zebra conducting a high-tech orchestra in a digital concert hall.",
  "A zebra piloting a spaceship through a galaxy of colorful nebulas.",
  "A zebra exploring a high-tech jungle with robotic plants and animals.",
  "A zebra painting a cosmic landscape on a canvas in a zero-gravity environment.",
  "A zebra dressed as a superhero fighting a giant robot in a futuristic city.",
  "A zebra performing ballet on a stage made of glowing holograms.",
  "A zebra ice skating on a lake made of liquid light under an aurora borealis.",
  "A zebra exploring an underwater city with bioluminescent skyscrapers.",
  "A zebra conducting a magic show on a floating island in the sky.",
  "A zebra riding a unicycle on a tightrope above a futuristic metropolis.",
  "A zebra wearing scuba gear, exploring an underwater cave filled with glowing jellyfish.",
  "A zebra piloting a hot air balloon across a desert landscape, avoiding sandstorms.",
  "A zebra at a formal dinner party, debating quantum physics with renowned scientists.",
  "A zebra conducting an orchestra of penguins in a snowy Antarctic concert hall.",
  "A zebra as the CEO of a high-tech startup, presenting their latest invention to skeptical investors.",
  "A zebra navigating a maze of mirrors in a carnival funhouse.",
  "A zebra as a professional skydiver, performing acrobatic maneuvers mid-air.",
  "A zebra riding a skateboard through a futuristic cityscape with flying cars.",
  "A zebra as a detective, solving a mystery involving stolen jewels in a medieval castle.",
  "A zebra as a master chef, preparing a gourmet meal in a busy restaurant kitchen.",
  "A zebra as a superhero, using its stripes to camouflage while fighting crime in a neon-lit city.",
  "A zebra as an astronaut, exploring an alien planet with bizarre flora and fauna.",
  "A zebra as a fashion designer, creating avant-garde outfits for a high-profile fashion show.",
  "A zebra as a medieval knight, jousting in a tournament against mythical creatures.",
  "A zebra as a professional gamer, competing in a virtual reality championship.",
  "A zebra as a mountain climber, scaling a treacherous peak in the Himalayas.",
  "A zebra as a diplomat, negotiating peace between rival factions in a war-torn country.",
  "A zebra as a submarine captain, exploring the depths of the ocean for lost treasure.",
  "A zebra as a wildlife photographer, capturing rare animals on the African savannah.",
  "A zebra as a time traveler, visiting ancient civilizations and witnessing historical events.",
  "A zebra as a wizard, casting spells to defend a magical kingdom from dark forces.",
  "A zebra as a race car driver, competing in a high-speed rally through rugged terrain.",
  "A zebra as a botanist, studying exotic plants in a dense rainforest.",
  "A zebra as a tour guide, leading tourists on safari through the Serengeti.",
  "A zebra as a submarine chef, cooking gourmet meals for a deep-sea research crew.",
  "A zebra as a spy, infiltrating a top-secret enemy base under cover of darkness.",
  "A zebra as a concert pianist, performing a classical masterpiece in a grand concert hall.",
  "A zebra as a detective, solving a murder mystery in a futuristic space station.",
  "A zebra as a firefighter, rescuing endangered animals from a wildfire in the Amazon rainforest.",
  "A zebra as a professor, teaching advanced mathematics to a class of brilliant students.",
  "A zebra as a stunt double, performing daring feats for a Hollywood action movie.",
  "A zebra as a robot engineer, designing autonomous machines for planetary exploration.",
  "A zebra as a professional surfer, riding giant waves in an international surfing competition.",
  "A zebra as a quantum physicist, conducting experiments with particles in a cutting-edge laboratory.",
  "A zebra as a circus performer, balancing on a tightrope under the big top.",
  "A zebra as a paranormal investigator, exploring haunted locations around the world.",
  "A zebra as a submarine archaeologist, uncovering ancient artifacts on the ocean floor.",
  "A zebra as a space explorer, discovering new planets in a distant galaxy.",
  "A zebra as a fashion model, strutting down the runway in haute couture.",
  "A zebra as a wilderness survival expert, teaching survival skills in the Arctic tundra.",
  "A zebra as a professional wrestler, competing in a championship match against a gorilla.",
  "A zebra as an art restorer, repairing priceless paintings in a museum.",
  "A zebra as a master gardener, cultivating rare flowers in a botanical garden.",
  "A zebra as a ski instructor, guiding beginners down a snowy mountain slope.",
  "A zebra as a submarine pilot, navigating through an underwater labyrinth.",
  "A zebra as a celebrity chef, hosting a cooking show with exotic ingredients.",
  "A zebra as a space tourist, enjoying weightlessness aboard a luxury space station.",
  "A zebra as a treasure hunter, searching for ancient artifacts in a remote jungle.",
  "A zebra as a castle architect, designing a fortress for a medieval kingdom.",
  "A zebra skydiving over a cityscape filled with hot air balloons and floating islands.",
  "A zebra wearing a tuxedo and playing a grand piano on a moonlit beach with bioluminescent waves.",
  "A zebra riding a unicycle on a tightrope stretched between two skyscrapers during a snowstorm.",
  "A zebra in a spacesuit planting a flag on the surface of Mars with Earth visible in the background.",
  "A zebra serving sushi in a futuristic restaurant on a space station orbiting Jupiter.",
  "A zebra reading a book under a giant mushroom in a forest filled with glowing plants and fireflies.",
  "A zebra painting a surrealist mural on a wall in an underwater city populated by mermaids and sea creatures.",
  "A zebra flying a biplane through a canyon filled with giant crystal formations and floating lanterns.",
  "A zebra conducting an orchestra composed of robotic musicians in a cyberpunk concert hall.",
  "A zebra wearing a detective hat investigating a crime scene in a city of anthropomorphic animals.",
  "A zebra surfing on a lava wave in a volcanic landscape with dragons flying overhead.",
  "A zebra dressed as a samurai fighting a dragon in a cherry blossom garden during a thunderstorm.",
  "A zebra performing a magic trick on a stage in a grand Victorian theater filled with an audience of ghosts.",
  "A zebra playing chess with an alien in a futuristic city with flying cars and holographic billboards.",
  "A zebra exploring an ancient temple filled with booby traps and glowing runes in a dense jungle.",
  "A zebra in a steampunk outfit piloting an airship through a sky filled with mechanical birds.",
  "A zebra competing in a robot wrestling match in a neon-lit arena with cheering spectators.",
  "A zebra skiing down a rainbow-colored slope in a magical winter wonderland with talking snowmen.",
  "A zebra hosting a cooking show in a kitchen made entirely of candy and sweets.",
  "A zebra meditating on a floating rock in a serene landscape with waterfalls flowing upwards.",
  "A zebra leading a parade of mythical creatures through a vibrant and fantastical carnival.",
  "A zebra driving a vintage car on a road that twists and turns through a surreal dreamscape.",
  "A zebra playing a harp on a cloud with musical notes turning into butterflies.",
  "A zebra playing soccer on a field where the grass is made of tiny stars and the sky is filled with auroras.",
  "A zebra playing a VR game in a futuristic city with neon lights and holograms.",
  "A zebra exploring a haunted mansion with moving paintings and shifting corridors.",
  "A zebra painting a sunset on an easel in a desert filled with giant cacti and mysterious ruins.",
  "A zebra playing the drums in a rock band of fantasy creatures on a stage with pyrotechnics.",
  "A zebra riding a giant snail through a forest of oversized flowers and glowing mushrooms.",
  "A zebra in a superhero costume flying over a city in the middle of a meteor shower.",
  "A zebra fishing in a lake where the fish are made of light and the water reflects a starry night sky.",
  "A zebra performing a circus act on a trapeze in a big top filled with exotic animals.",
  "A zebra piloting a submarine through an underwater city with towering coral skyscrapers.",
  "A zebra exploring a labyrinth filled with moving walls and floating orbs of light.",
  "A zebra riding a skateboard on a half-pipe made of glass in a futuristic cityscape.",
  "A zebra playing an electric guitar on a mountaintop with a thunderstorm raging in the background.",
  "A zebra racing a futuristic hovercar through a neon-lit city filled with giant holographic advertisements.",
  "A zebra flying a dragon through a sky filled with floating islands and castles in the clouds.",
  "A zebra planting a tree in a desert where the sand dunes are made of shimmering gold.",
  "A zebra diving into a pool of liquid rainbow in a fantasy landscape with multi-colored mountains.",
  "A zebra playing a grand piano in the middle of a bustling train station.",
  "A zebra exploring a library where the books float and rearrange themselves on the shelves.",
  "A zebra sitting at a cafe table on the edge of a cliff overlooking a city in the clouds.",
  "A zebra painting graffiti on a wall in a futuristic city with robots and flying cars.",
  "A zebra in a spacesuit floating in zero gravity inside a space station with Earth visible through a window.",
  "A zebra riding a mechanical bull in a cyberpunk bar filled with neon lights and holograms.",
  "A zebra playing chess with a giant, sentient tree in a forest filled with magical creatures.",
  "A zebra exploring a cave filled with glowing crystals and ancient, mysterious symbols.",
  "A zebra sailing a pirate ship through a sky filled with floating islands and giant sea creatures.",
  "A zebra playing a grand piano on a floating platform in the middle of a serene lake.",
  "A zebra riding a roller coaster through a city made entirely of glass and light.",
  "A zebra dressed as a wizard casting a spell in a library filled with floating books and magical artifacts.",
  "A zebra piloting a spaceship through an asteroid field with a distant planet visible in the background.",
  "A zebra riding a bicycle on a path made of light through a futuristic city.",
  "A zebra playing the violin on a cliff overlooking a sea of clouds at sunset.",
  "A zebra exploring a jungle temple filled with ancient, glowing runes and booby traps.",
  "A zebra in a futuristic battle suit fighting robots in a neon-lit city.",
  "A zebra racing a hoverboard through a cyberpunk city with flying cars and holographic advertisements.",
  "A zebra performing a ballet on a stage surrounded by giant, glowing flowers.",
  "A zebra exploring a haunted forest filled with ghostly apparitions and moving trees.",
  "A zebra painting a mural on a wall in an underwater city with mermaids and sea creatures watching.",
  "A zebra riding a dragon through a stormy sky filled with lightning and floating islands.",
  "A zebra playing a harp in a serene garden with waterfalls flowing upwards.",
  "A zebra exploring a desert filled with giant, ancient statues and mysterious ruins.",
  "A zebra piloting a steampunk airship through a sky filled with mechanical birds.",
  "A zebra performing a magic show on a stage in a grand, Victorian theater filled with an audience of ghosts.",
  "A zebra riding a giant snail through a forest of oversized flowers and glowing mushrooms.",
  "A zebra playing an electric guitar on a mountaintop with a thunderstorm raging in the background.",
  "A zebra racing a futuristic hovercar through a neon-lit city filled with giant holographic advertisements.",
  "A zebra flying a dragon through a sky filled with floating islands and castles in the clouds.",
  "A zebra planting a tree in a desert where the sand dunes are made of shimmering gold.",
  "A zebra diving into a pool of liquid rainbow in a fantasy landscape with multi-colored mountains.",
  "A zebra playing a grand piano in the middle of a bustling train station.",
  "A zebra exploring a library where the books float and rearrange themselves on the shelves.",
  "A zebra sitting at a cafe table on the edge of a cliff overlooking a city in the clouds.",
  "A zebra painting graffiti on a wall in a futuristic city with robots and flying cars.",
  "A zebra in a spacesuit floating in zero gravity inside a space station with Earth visible through a window.",
  "A zebra riding a mechanical bull in a cyberpunk bar filled with neon lights and holograms.",
  "A zebra playing chess with a giant, sentient tree in a forest filled with magical creatures.",
  "A zebra exploring a cave filled with glowing crystals and ancient, mysterious symbols.",
  "A zebra sailing a pirate ship through a sky filled with floating islands and giant sea creatures.",
  "A zebra playing a grand piano on a floating platform in the middle of a serene lake.",
  "A zebra riding a roller coaster through a city made entirely of glass and light.",
  "A zebra dressed as a wizard casting a spell in a library filled with floating books and magical artifacts.",
  "A zebra piloting a spaceship through an asteroid field with a distant planet visible in the background.",
  "A zebra riding a bicycle on a path made of light through a futuristic city.",
  "A zebra playing the violin on a cliff overlooking a sea of clouds at sunset.",
  "A zebra exploring a jungle temple filled with ancient, glowing runes and booby traps.",
  "A zebra in a futuristic battle suit fighting robots in a neon-lit city.",
  "A zebra racing a hoverboard through a cyberpunk city with flying cars and holographic advertisements.",
  "A zebra performing a ballet on a stage surrounded by giant, glowing flowers.",
  "A zebra exploring a haunted forest filled with ghostly apparitions and moving trees.",
  "A zebra painting a mural on a wall in an underwater city with mermaids and sea creatures watching.",
  "A zebra riding a dragon through a stormy sky filled with lightning and floating islands.",
  "A zebra playing a harp in a serene garden with waterfalls flowing upwards.",
  "A zebra exploring a desert filled with giant, ancient statues and mysterious ruins.",
  "A zebra piloting a steampunk airship through a sky filled with mechanical birds.",
  "A zebra performing a magic show on a stage in a grand, Victorian theater filled with an audience of ghosts.",
  "A zebra skydiving over a tropical island with a giant rainbow and dolphins jumping out of the water.",
  "A zebra wearing a firefighter uniform rescuing a kitten from a burning building with robot firefighters assisting.",
  "A zebra conducting an orchestra of fairies and woodland creatures in an enchanted forest.",
  "A zebra wearing an astronaut suit exploring a city on the moon with Earthrise in the background.",
  "A zebra dressed as a king or queen presiding over a court of talking animals in a medieval castle.",
  "A zebra playing the cello on a gondola floating through a city of glowing canals and crystal buildings.",
  "A zebra surfing on a massive tidal wave made of sparkling stardust under a sky full of shooting stars.",
  "A zebra dressed as a detective solving a mystery in a Victorian-era city with steam-powered gadgets.",
  "A zebra as a high-tech cyborg hacking into a computer network in a futuristic, neon-lit city.",
  "A zebra exploring an ancient, abandoned spaceship filled with alien technology and mysterious symbols.",
  "A zebra driving a classic convertible car through a psychedelic landscape of swirling colors and shapes.",
  "A zebra as a scientist mixing potions in a magical laboratory filled with bubbling cauldrons and spell books.",
  "A zebra dancing the tango with a robot partner on a rooftop overlooking a futuristic city.",
  "A zebra playing a grand piano in a vast desert with sand dunes shaped like musical notes.",
  "A zebra as a pirate captain steering a ship through a stormy sea with ghost ships in the distance.",
  "A zebra in traditional samurai armor practicing swordsmanship in a serene Japanese garden.",
  "A zebra exploring a city made entirely of candy and sweets with gingerbread houses and lollipop trees.",
  "A zebra dressed as a knight fighting a dragon in a mystical forest with glowing plants.",
  "A zebra playing a trumpet in a New Orleans-style jazz band on a floating stage in a river.",
  "A zebra in a medieval alchemist's workshop filled with potions, spellbooks, and mystical artifacts.",
  "A zebra piloting a hot air balloon over a landscape of floating islands and waterfalls.",
  "A zebra as a superhero flying over a cityscape during a dramatic sunset with storm clouds gathering.",
  "A zebra exploring a city made of glass and light with buildings that change color and shape.",
  "A zebra dressed as a wizard casting spells in a magical tower filled with ancient scrolls and artifacts.",
  "A zebra playing a saxophone in a smoky jazz club filled with anthropomorphic animals.",
  "A zebra driving a futuristic motorcycle through a neon-lit cityscape with holographic billboards.",
  "A zebra exploring a haunted amusement park with ghostly rides and attractions.",
  "A zebra as a time traveler exploring different historical eras in a single, sprawling cityscape.",
  "A zebra wearing a futuristic spacesuit walking on a planet with strange, alien flora and fauna.",
  "A zebra as a chef cooking in a high-tech kitchen with robotic assistants and futuristic gadgets.",
  "A zebra exploring an ancient, mystical library where the books float and whisper secrets.",
  "A zebra dressed as a rock star performing on a stage with a band of anthropomorphic animals.",
  "A zebra piloting a flying car through a city filled with towering skyscrapers and sky bridges.",
  "A zebra dressed as an ancient Egyptian pharaoh exploring a pyramid filled with treasures and traps.",
  "A zebra playing the flute in a magical forest where the trees glow and sing along with the music.",
  "A zebra driving a monster truck through a post-apocalyptic landscape with ruined buildings.",
  "A zebra dressed as a Victorian-era inventor working on a steam-powered flying machine.",
  "A zebra exploring a futuristic theme park with rides and attractions from different planets.",
  "A zebra as a painter creating a masterpiece on a giant canvas in the middle of a bustling city square.",
  "A zebra wearing a superhero cape rescuing citizens from a collapsing building in a futuristic city.",
  "A zebra exploring a mystical cave filled with glowing crystals and ancient artifacts.",
  "A zebra as a magician performing illusions on a stage with a backdrop of a starry night sky.",
  "A zebra piloting a spaceship through a galaxy filled with colorful nebulae and alien planets.",
  "A zebra dressed as a cowboy riding a robotic horse through a futuristic Wild West town.",
  "A zebra exploring a city submerged underwater with fish and mermaids swimming among the buildings.",
  "A zebra as a knight in shining armor fighting off an army of robots in a medieval castle.",
  "A zebra playing a violin on a bridge made of light spanning a river of stars.",
  "A zebra in a high-tech laboratory conducting experiments with glowing chemicals and holographic displays.",
  "A zebra dressed as a circus ringmaster leading a parade of fantastical creatures and acrobats.",
  "A zebra piloting a futuristic jet through a canyon filled with floating rocks and alien vegetation.",
  "A zebra exploring an ancient ruins filled with mysterious symbols and hidden passages.",
  "A zebra as a superhero fighting a giant monster in the middle of a bustling city.",
  "A zebra playing a harp in a serene garden surrounded by waterfalls and glowing flowers.",
#   "A bear emerges from a quantum portal, startling a group of physicists conducting an experiment.",
#   "In a futuristic city, a cybernetically enhanced bear disrupts a high-tech bank heist.",
#   "At a royal banquet in medieval times, a bear starts speaking fluent Latin, much to the shock of the guests.",
#   "In a laboratory accident, a bear gains telekinetic powers and starts rearranging furniture.",
#   "During a space expedition, a bear stows away on a spaceship heading to Mars, causing chaos among the crew.",
#   "At a masquerade ball, a bear in an elaborate costume accidentally wins first prize for best-dressed guest.",
#   "In a courtroom drama, a bear is called as a witness in a murder trial and reveals surprising details about the case.",
#   "In an enchanted forest, a bear becomes the reluctant guardian of a lost magical artifact.",
#   "At a rock concert, a bear unexpectedly joins the band on stage and plays the drums with precision.",
#   "In a post-apocalyptic world, a bear becomes the leader of a band of survivors, negotiating peace with rival factions.",
#   "During a synchronized swimming competition, a bear performs a flawless routine alongside professional athletes.",
#   "At a prestigious art gallery, a bear becomes obsessed with a particular abstract painting and refuses to leave.",
#   "In a submarine expedition, a bear accidentally activates a dormant underwater volcano, endangering the crew.",
#   "During a diplomatic summit, a bear accidentally eats a rare, ceremonial flower crucial for negotiations.",
#   "In a small town, a bear discovers an ancient map leading to a long-lost pirate treasure.",
#   "At a high-tech research facility, a bear accidentally activates a time machine, sending scientists into different historical eras.",
#   "In a ballet performance, a bear joins the dancers on stage and performs a graceful pirouette.",
#   "During a film shoot, a bear wanders onto the set and stars in an impromptu scene, stealing the spotlight.",
#   "In a circus, a bear learns to juggle chainsaws, impressing even the most skeptical audience members.",
#   "At a cooking competition, a bear accidentally creates a gourmet dish that becomes an overnight sensation.",
#   "In a remote mountain village, a bear develops a reputation as a healer using unusual methods.",
#   "During a NASA mission, a bear accidentally activates an experimental anti-gravity device aboard the International Space Station.",
#   "In a boardroom meeting, a bear surprises everyone by presenting a detailed business strategy for a new start-up.",
#   "At a ski resort, a bear competes in a downhill race and narrowly avoids a collision with professional skiers.",
#   "In a haunted mansion, a bear befriends a ghost and helps solve the mystery of its untimely death.",
#   "During a fashion show, a bear models a designer gown and becomes an overnight fashion icon.",
#   "At a summer camp, a bear becomes the mascot and leads campers in a series of unexpected adventures.",
#   "In a robotics competition, a bear accidentally activates a prototype robot that demonstrates human-like intelligence.",
#   "During a music festival, a bear sneaks backstage and jams with a famous rock band, electrifying the crowd.",
#   "At a scientific conference, a bear surprises researchers by presenting a breakthrough theory on gravitational waves.",
#   "In a fantasy realm, a bear becomes the loyal companion of a young sorcerer-in-training.",
#   "During a marathon race, a bear joins the runners and completes the course with remarkable speed.",
#   "At a wedding reception, a bear steals the bride's bouquet and runs off, leading to a comedic chase through the venue.",
#   "In a space colony, a bear becomes an unexpected hero by rescuing stranded astronauts during a solar storm.",
#   "During a police chase, a bear commandeers a motorcycle and leads officers on a wild pursuit through the city.",
#   "At a theme park, a bear accidentally triggers a malfunction in a roller coaster, sending thrill-seekers on an unplanned adventure.",
#   "In a courtroom sketch artist competition, a bear's drawing of the judge goes viral for its uncanny accuracy.",
#   "During a theatrical performance, a bear joins the actors on stage and delivers an impromptu Shakespearean monologue.",
#   "At an archaeological dig site, a bear uncovers a mysterious artifact that rewrites the history of ancient civilizations.",
#   "In a robotics lab, a bear inadvertently activates a swarm of miniature drones, causing chaos in the research facility.",
#   "During a high-stakes poker game, a bear bluffs its way to victory against seasoned players.",
#   "At a circus performance, a bear becomes the star attraction by mastering a complex acrobatic routine.",
#   "In a courtroom trial, a bear is called to testify as an expert witness on animal behavior.",
#   "During a lunar mission, a bear accidentally activates a prototype rover and explores the moon's surface.",
#   "At a fashion runway show, a bear models avant-garde designer clothing and captivates the audience.",
#   "In a medieval tournament, a bear competes in jousting matches and wins the admiration of noble spectators.",
#   "During a film premiere, a bear accidentally interrupts the red carpet event and steals the spotlight from celebrities.",
#   "At an archaeological excavation, a bear discovers a hidden chamber filled with ancient artifacts of unknown origin.",
#   "In a futuristic laboratory, a bear interacts with a humanoid robot prototype, showcasing unexpected intelligence.",
#   "During a music concert, a bear climbs on stage and performs a duet with the lead singer, to the audience's delight.",
#   "At a prestigious golf tournament, a bear wanders onto the course and showcases remarkable skill in avoiding hazards.",
#   "In a superhero convention, a bear cosplays as a famous comic book character and wins a costume contest.",
#   "During a firework display, a bear accidentally triggers a spectacular show of pyrotechnics in a city park.",
#   "At a science fair, a bear demonstrates a makeshift invention that astounds judges with its practicality.",
#   "In a jungle expedition, a bear rescues explorers from a sinking raft using improvised flotation devices.",
#   "During a chess tournament, a bear challenges the reigning champion to a match and plays with surprising strategy.",
#   "At a futuristic theme park, a bear interacts with lifelike animatronic characters, causing confusion among visitors.",
  "A sheep herding competition on Mars, where the sheep are genetically modified to survive in low oxygen environments.",
  "A sheep trying to navigate a maze designed by a mischievous AI, filled with holographic illusions of wolves.",
  "A sheep knitting a sweater for a robotic shepherd, using fiber spun from electronic circuit components.",
  "A sheep participating in a synchronized swimming competition with dolphins, using a virtual reality simulator in Antarctica.",
  "A sheep attending a diplomatic summit between humans and an advanced civilization of alien sheep, translating complex negotiations using a brain-computer interface.",
  "A sheep training to become a cybernetic detective, solving mysteries in a futuristic city plagued by digital crimes.",
  "A sheep leading a team of drones to rescue stranded climbers on an icy mountain peak during a blizzard.",
  "A sheep developing a new form of renewable energy using its wool, while navigating corporate espionage from rival energy companies.",
  "A sheep serving as a test subject for a time-travel experiment, accidentally transported to ancient Rome during a gladiatorial battle.",
  "A sheep competing in a cooking show for extraterrestrial judges, using ingredients harvested from asteroid mining expeditions.",
  "A sheep discovering a hidden underwater city populated by intelligent octopuses, with whom it must collaborate to find a lost treasure.",
  "A sheep serving as a therapist for robots experiencing existential crises, programmed with emotions they don't understand.",
  "A sheep navigating a zero-gravity obstacle course aboard a space station, competing against AI-controlled drones.",
  "A sheep attempting to break the world record for the longest flight by a solar-powered hot air balloon, constructed from biodegradable materials.",
  "A sheep participating in a cross-species talent show, showcasing its ability to play classical music on a piano using hoof-friendly keys.",
  "A sheep piloting a submarine through an underwater cave system, discovering ancient artifacts left by an ancient civilization of underwater sheep.",
  "A sheep joining an elite squad of firefighters equipped with advanced exoskeletons, battling wildfires in a rainforest threatened by climate change.",
  "A sheep becoming a viral sensation on social media for its ability to solve complex mathematical puzzles using a specially designed app.",
  "A sheep training as a spy in a Cold War-era setting, tasked with infiltrating a high-security enemy base disguised as a diplomatic envoy.",
  "A sheep leading a team of scientists on an expedition to study a newly discovered species of intelligent plants in the Amazon rainforest.",
  "A sheep participating in a reality TV show where contestants compete to survive on a deserted island inhabited by friendly yet mischievous robots.",
  "A sheep volunteering as a test subject for a breakthrough medical procedure that grants animals the ability to communicate verbally with humans.",
  "A sheep navigating a series of ancient puzzles and traps in an Egyptian pyramid, searching for a legendary artifact said to grant eternal wisdom.",
  "A sheep serving as a navigator on a space exploration mission to a distant galaxy, communicating with alien species using a universal translator device.",
  "A sheep competing in a global race for renewable energy vehicles, driving a solar-powered car across diverse terrains and climates.",
  "A sheep training as a rescue dog in a post-apocalyptic world, searching for survivors in collapsed buildings and navigating radioactive wastelands.",
  "A sheep participating in a futuristic sports league where animals and robots compete together in teams, playing a hybrid of soccer and drone racing.",
  "A sheep leading an expedition to a remote Arctic research station, studying the effects of climate change on polar bear behavior.",
  "A sheep accidentally activating a time machine prototype, jumping through different historical eras while trying to find a way back home.",
  "A sheep forming a rock band with other animals, performing concerts on stages made entirely of recycled materials in a world where music is outlawed.",
#   "A sheep enrolling in a virtual reality academy for superheroes, training to harness its unique abilities to fight virtual villains and save simulated cities.",
#   "A sheep serving as a cultural ambassador between Earth and a civilization of sentient plants on a distant planet, mediating interplanetary disputes.",
#   "A sheep participating in an underwater dance competition with creatures from the deepest parts of the ocean, choreographed to bioluminescent music.",
#   "A sheep joining a team of archaeologists exploring a lost city buried beneath the sands of the Sahara Desert, uncovering secrets of an ancient sheep civilization.",
#   "A sheep developing a revolutionary AI-driven farming system, using drones and autonomous vehicles to maximize crop yields in a sustainable manner.",
#   "A sheep training as a chef in a restaurant run entirely by robots, mastering culinary skills to create dishes that fuse traditional and futuristic flavors.",
#   "A sheep competing in a chess tournament against AI opponents programmed with advanced strategic algorithms, aiming to become the first animal grandmaster.",
#   "A sheep embarking on a quest to find a legendary philosopher's stone that grants animals the ability to speak human languages.",
#   "A sheep working as a detective in a cyberpunk city, solving crimes committed by hackers who manipulate reality through augmented reality interfaces.",
#   "A sheep participating in a global space race, piloting a spacecraft powered by renewable energy sources to explore uncharted planets.",
#   "A sheep volunteering for a medical experiment that enhances its intelligence, leading to philosophical debates with other enhanced animals about the nature of consciousness.",
#   "A sheep joining an expedition to study the behavior of giant creatures living deep beneath the ocean's surface, communicating with them using underwater drones.",
#   "A sheep competing in a futuristic equestrian event where animals and robots race together through a high-tech obstacle course.",
#   "A sheep training as a rescue pilot in a world where flying sheep are essential for transporting medical supplies to remote villages inaccessible by land.",
#   "A sheep serving as a test subject for a virtual reality simulation that recreates historical battles, analyzing tactics used by ancient civilizations.",
#   "A sheep participating in a reality TV show where contestants compete to design and build sustainable habitats for endangered species in remote locations.",
#   "A sheep developing a new form of communication with alien species, using bioluminescent patterns to convey complex ideas across different planets.",
#   "A sheep joining a team of scientists studying the effects of zero-gravity on plant growth aboard a space station orbiting Mars.",
#   "A sheep competing in an international fencing tournament for animals, using specially designed electronic swords to duel opponents from different species.",
#   "A sheep embarking on a journey to find a legendary lost city in the depths of the Amazon rainforest, guided by ancient maps and cryptic clues.",
#   "A sheep participating in a virtual reality game where players explore a simulated world filled with mythical creatures and ancient civilizations.",
#   "A sheep serving as a detective in a futuristic noir city, solving crimes committed by rogue AI with advanced cognitive abilities.",
#   "A sheep training as a pilot in a squadron of flying vehicles used for emergency response missions in disaster-prone areas.",
#   "A sheep competing in a global cooking competition where contestants prepare dishes using ingredients sourced from Mars colonization missions.",
#   "A sheep volunteering for a mission to explore an abandoned space station orbiting a distant planet, uncovering mysteries left behind by its previous inhabitants.",
#   "A sheep joining an expedition to study the migration patterns of robotic whales in the oceans of Europa, communicating with them through encrypted signals.",
#   "A sheep participating in a virtual reality simulation where players compete in a high-speed hoverboard race through a futuristic cityscape.",
#   "A sheep serving as a mediator between humans and a civilization of intelligent fungi discovered deep within Earth's crust, negotiating peaceful coexistence.",
#   "A sheep training as a space explorer, piloting a spacecraft equipped with advanced technology to search for signs of extraterrestrial life in distant star systems.",
#   "A sheep competing in a global robotics competition where animals team up with AI-controlled robots to solve complex challenges.",
#   "A sheep embarking on a quest to find a legendary artifact hidden in a labyrinthine maze beneath an ancient Mayan temple.",
#   "A sheep participating in a virtual reality adventure game where players collaborate to solve puzzles and uncover the secrets of a digital world.",
#   "A sheep serving as a guide for tourists exploring a futuristic city built entirely underwater, showcasing advanced marine technology and sustainable living practices.",
#   "A sheep joining an archaeological expedition to uncover the secrets of an ancient civilization buried beneath the ice of Antarctica, deciphering lost languages and technologies.",
#   "A sheep competing in a futuristic racing league where animals pilot anti-gravity vehicles through neon-lit cityscapes, navigating through twists, turns, and obstacles.",
#   "A sheep training as a rescue diver in a team of marine animals, performing underwater missions to save stranded sailors and recover lost treasures.",
#   "A sheep participating in a global competition where animals and robots collaborate to create artworks that explore themes of artificial intelligence and creativity.",
#   "A sheep volunteering for a mission to terraform a distant planet, using its unique biological traits to assist in the creation of habitable environments.",
#   "A sheep joining a team of scientists studying the migration patterns of bioluminescent creatures in the depths of the Mariana Trench, using specialized deep-sea exploration vehicles.",
#   "A sheep competing in a futuristic sports league where animals and cyborgs engage in extreme sports challenges, pushing the limits of physical endurance and technological innovation.",
#   "A sheep embarking on a journey to find a legendary lost civilization in the jungles of Borneo, deciphering ancient myths and navigating treacherous terrain.",
#   "A sheep participating in a virtual reality simulation where players collaborate to build sustainable cities on Mars, balancing environmental factors and technological advancements.",
#   "A sheep serving as a test subject for a groundbreaking medical procedure that enhances its senses to superhuman levels, navigating a world of heightened perceptions and challenges.",
#   "A sheep joining an expedition to study the behavior of intelligent cephalopods in the depths of the Pacific Ocean, communicating with them through a specially designed underwater language.",
#   "A sheep competing in a global robotics competition where animals team up with AI-controlled robots to solve complex challenges.",
#   "A sheep embarking on a quest to find a legendary artifact hidden in a labyrinthine maze beneath an ancient Mayan temple.",
#   "A sheep participating in a virtual reality adventure game where players collaborate to solve puzzles and uncover the secrets of a digital world.",
#   "A sheep serving as a guide for tourists exploring a futuristic city built entirely underwater, showcasing advanced marine technology and sustainable living practices",
  "A cat balancing on a unicycle while juggling three flaming torches in a crowded subway station during rush hour.",
  "A cat wearing a detective's hat, inspecting a crime scene with miniature forensic tools on a tiny desk.",
  "A cat operating a control panel in a spaceship hurtling through an asteroid field, navigating with intense concentration.",
  "A cat conducting a symphony orchestra in a concert hall filled with animals playing various instruments.",
  "A cat dressed in medieval armor, defending a castle from an invading army of mice armed with miniature siege engines.",
  "A cat serving as a diplomat in a United Nations-style assembly of animals discussing global environmental policy.",
  "A cat participating in a high-stakes poker game with dogs, raccoons, and birds in a glamorous Las Vegas casino.",
  "A cat leading a team of mountaineering mice up a treacherous, snowy peak in the Himalayas.",
  "A cat teaching a class of penguins how to swim gracefully in a synchronized swimming competition.",
  "A cat managing a circus where all the performers are insects showcasing acrobatics and tightrope walking.",
  "A cat organizing a film festival for forest creatures, screening movies under the stars with popcorn and cozy blankets.",
  "A cat organizing a grand banquet for woodland creatures, complete with a miniature banquet hall and tiny gourmet dishes.",
  "A cat directing a fashion show for hedgehogs, styling them in avant-garde outfits made of leaves and flowers.",
  "A cat leading an archaeological dig in a desert, uncovering ancient artifacts from a lost civilization of rodents.",
  "A cat hosting a talk show on a miniature set with celebrity guests who are all small woodland animals.",
  "A cat coaching a team of squirrels in a soccer match against a team of rabbits in a bustling urban park.",
  "A cat managing a beach resort for crabs, providing umbrellas, beach chairs, and tiny surfboards.",
  "A cat organizing a space mission to Mars for a crew of adventurous hamsters, complete with a mini rocket and space suits.",
  "A cat running a detective agency for solving mysteries in a city populated by tiny creatures.",
  "A cat leading a rock band composed of frogs, beetles, and snails performing a concert in a lush garden.",
#   "A cat painting a mural on the side of a barn with the help of artistic chickens and sheep.",
#   "A cat organizing a circus performance where miniature elephants, lions, and zebras perform under a tiny big top.",
#   "A cat training a team of ants to compete in an obstacle course race against other insect teams.",
#   "A cat hosting a cooking show for forest animals, preparing gourmet meals with ingredients foraged from the woods.",
#   "A cat organizing a wildlife photography contest for birds, squirrels, and rabbits in a picturesque national park.",
#   "A cat leading a protest march of woodland creatures advocating for conservation in a bustling city square.",
#   "A cat organizing a miniature carnival for mice, complete with rides, games, and cotton candy stands.",
#   "A cat managing a construction project where tiny beavers and rabbits build a miniature dam across a creek.",
#   "A cat hosting a talent show for backyard critters, showcasing singing birds, dancing squirrels, and performing frogs.",
#   "A cat curating an art exhibition for insects, displaying intricate sculptures and paintings in a gallery made from leaves.",
#   "A cat managing a miniature hospital for injured birds and squirrels, with tiny nurses and doctors providing care.",
#   "A cat organizing a synchronized swimming performance for fish and turtles in a tranquil pond.",
#   "A cat running a bakery for forest animals, baking tiny cakes and pastries decorated with edible flowers.",
#   "A cat leading a rescue mission to save stranded turtles and frogs during a flood in a miniature village.",
#   "A cat organizing a miniature golf tournament for rabbits and hedgehogs in a whimsical garden.",
#   "A cat teaching a group of young mice how to perform magic tricks in a school for aspiring illusionists.",
#   "A cat organizing a puppet show for frogs and toads, telling stories with intricately crafted puppets.",
#   "A cat leading a treasure hunt for rabbits and squirrels in a dense forest, searching for hidden gems and coins.",
#   "A cat managing a ski resort for snowboarding rabbits and skiing foxes on a snowy mountain.",
#   "A cat organizing a ballet performance for birds and butterflies in a magical forest clearing.",
#   "A cat hosting a science fair for young chipmunks and raccoons, showcasing inventions and experiments.",
#   "A cat leading a rescue mission to save baby birds stranded in a tree during a storm.",
#   "A cat managing a toy factory for making miniature dolls and toys for woodland creatures.",
#   "A cat organizing a storytelling festival for rabbits, hedgehogs, and owls around a campfire.",
#   "A cat leading a team of tiny firefighters to extinguish a forest fire threatening a community of squirrels and birds.",
#   "A cat running a bookstore for insects, with shelves filled with tiny books and reading nooks made from leaves.",
#   "A cat organizing a race for snails and beetles in a garden maze, complete with obstacles and finish lines.",
#   "A cat hosting a comedy show for forest creatures, with stand-up performances by witty squirrels and chipmunks.",
#   "A cat leading an expedition to explore an underwater cave with a team of adventurous fish and crabs.",
#   "A cat managing a toy train station for transporting mice and rabbits on miniature tracks through a whimsical countryside.",
#   "A cat organizing a puppet theater production starring woodland creatures performing classic fairy tales.",
#   "A cat leading a rescue mission to save a lost baby deer in a dense forest inhabited by mischievous raccoons.",
  "A dog in a space suit exploring the surface of Mars, trying to communicate with alien vegetation.",
  "A dog wearing a detective's hat, solving a mystery involving missing socks in a laundromat.",
  "A dog leading a team of pirates in search of a legendary bone buried on a deserted island.",
  "A dog attending a fancy ball, trying to impress the other animals with its salsa dancing skills.",
  "A dog piloting a submarine through an underwater maze filled with ancient artifacts.",
  "A dog managing a bustling coffee shop where all the customers are birds.",
  "A dog participating in a high-stakes poker game with a group of cunning squirrels.",
  "A dog navigating a parallel universe where gravity changes direction every hour.",
  "A dog conducting a symphony orchestra made up entirely of cats.",
  "A dog racing against time to defuse a bomb hidden in a busy city square.",
  "A dog as a celebrity chef in a cooking show, preparing a gourmet meal for a panel of skeptical rabbits.",
  "A dog running for president of a fictional country where animals can vote.",
  "A dog trapped inside a video game, battling pixelated monsters to rescue its owner.",
  "A dog as the captain of a spaceship, facing a mutiny from its crew of talking hamsters.",
  "A dog as a famous artist, creating avant-garde sculptures out of recycled trash.",
  "A dog as a master spy, infiltrating a secret society of ninja cats.",
  "A dog as a professor at an underwater university, teaching marine biology to fish students.",
  "A dog as a time traveler, accidentally altering history by saving a butterfly in ancient Egypt.",
  "A dog as a superhero, using its bark to create sonic waves to fight crime.",
  "A dog as a mountain climber, scaling the highest peaks in a world where mountains float in the sky.",
#   "A dog as a paranormal investigator, hunting ghosts in a haunted mansion filled with mischievous spirits.",
#   "A dog as a fashion designer, creating trend-setting outfits for a glamorous runway show.",
#   "A dog as a professional wrestler, taking on opponents twice its size in an epic championship match.",
#   "A dog as a circus performer, mastering tightrope walking while juggling flaming bones.",
#   "A dog as a submarine captain, exploring the depths of the ocean in search of lost underwater civilizations.",
#   "A dog as a competitive eater, facing off against a team of competitive squirrels in a hot dog eating contest.",
#   "A dog as a wilderness survival expert, teaching survival skills to a group of lost penguins in Antarctica.",
#   "A dog as a race car driver, competing in a high-speed race against a team of cheetahs.",
#   "A dog as a museum curator, managing a collection of ancient artifacts from a lost civilization.",
#   "A dog as a private investigator, solving a mystery involving disappearing toys in a neighborhood of cats.",
#   "A dog as a submarine chef, preparing gourmet meals for a crew of deep-sea explorers.",
#   "A dog as a stand-up comedian, performing a hilarious routine for an audience of laughing hyenas.",
#   "A dog as an archaeologist, uncovering the remains of a dinosaur buried beneath a suburban backyard.",
#   "A dog as a master chess player, competing in a tournament against a team of cunning foxes.",
#   "A dog as a space explorer, discovering a new galaxy inhabited by friendly alien creatures.",
#   "A dog as a firefighter, rescuing kittens from a burning skyscraper in a city populated by squirrels.",
#   "A dog as a secret agent, infiltrating a spy organization run by sophisticated penguins.",
#   "A dog as a time-traveling inventor, accidentally creating a rift in time that threatens to erase its own existence.",
#   "A dog as a professional gamer, competing in a virtual reality tournament against a team of robotic cats.",
#   "A dog as a stunt double, performing daring feats in action movies for a famous actor cat.",
#   "A dog as a ghost hunter, investigating paranormal activity in a haunted castle inhabited by ghostly mice.",
#   "A dog as a master chef, preparing a feast for a royal banquet attended by noble rabbits.",
#   "A dog as a wildlife photographer, capturing rare moments of animal behavior in the Amazon rainforest.",
#   "A dog as a submarine explorer, discovering a lost underwater city inhabited by mer-dogs.",
#   "A dog as a treasure hunter, searching for buried pirate treasure on a remote desert island.",
#   "A dog as a superhero sidekick, assisting a crime-fighting cat in protecting the city from supervillains.",
#   "A dog as a master illusionist, performing magic tricks that defy the laws of physics for an audience of skeptical squirrels.",
#   "A dog as a professional golfer, competing in a championship tournament against a team of golfing bears.",
#   "A dog as a space engineer, building a rocket ship capable of traveling to distant galaxies.",
#   "A dog as a detective, solving a mystery involving a missing diamond necklace in a mansion filled with elegant frogs.",
#   "A dog as a master sushi chef, preparing delicate sushi rolls for a group of discerning panda customers.",
#   "A dog as a deep-sea explorer, discovering a hidden underwater cavern filled with glowing jellyfish.",
#   "A dog as a courtroom lawyer, defending an innocent parrot accused of stealing birdseed.",
#   "A dog as a racehorse jockey, competing in a high-stakes horse race against a team of racing turtles.",
#   "A dog as a wildlife conservationist, protecting endangered species in a jungle inhabited by talking monkeys.",
#   "A dog as a submarine navigator, guiding a crew of adventurous cats through treacherous underwater caves.",
#   "A dog as a master architect, designing a futuristic cityscape for a community of sophisticated rabbits.",
#   "A dog as a celebrity chef, hosting a cooking show where it prepares gourmet meals for a panel of skeptical squirrels.",
#   "A dog as a professional athlete, competing in a championship tennis match against a team of tennis-playing tortoises.",
#   "A dog as a deep-sea explorer, discovering a hidden underwater cavern filled with glowing jellyfish.",
#   "A dog as a courtroom lawyer, defending an innocent parrot accused of stealing birdseed.",
#   "A dog as a racehorse jockey, competing in a high-stakes horse race against a team of racing turtles.",
#   "A dog as a wildlife conservationist, protecting endangered species in a jungle inhabited by talking monkeys",
  "A horse galloping through a library, knocking over bookshelves and scattering books everywhere.",
  "A horse wearing a spacesuit, trotting across the surface of Mars, with a colony of curious Martian creatures following it.",
  "A horse in a detective's office, examining clues with a magnifying glass, trying to solve a mysterious disappearance.",
  "A horse in a submarine, navigating deep ocean trenches while scientists conduct marine biology experiments around it.",
  "A horse in a wizard's laboratory, accidentally triggering a magical explosion that turns all the furniture into enchanted creatures.",
  "A horse in a medieval jousting tournament, wearing armor and challenging a dragon to a duel.",
  "A horse in a futuristic city, using cybernetic enhancements to compete in a high-speed hoverboard race.",
  "A horse on a film set, starring in a blockbuster action movie, dodging explosions and performing stunts.",
  "A horse at a royal banquet, wearing a crown and presiding over a diplomatic negotiation between kingdoms.",
  "A horse in a time-travel experiment, accidentally transported to ancient Rome during a gladiator battle.",
  "A horse in a haunted mansion, uncovering ghostly secrets while searching for a hidden treasure.",
  "A horse on a pirate ship, leading a crew of swashbucklers in a quest for a legendary buried treasure.",
  "A horse in a circus, performing acrobatic tricks on a tightrope while elephants and clowns cheer it on.",
  "A horse in a jungle expedition, discovering a lost civilization and befriending its indigenous inhabitants.",
  "A horse in a superhero team, using its superpowers to fight off an alien invasion threatening Earth.",
  "A horse in a space station, assisting astronauts in repairing critical systems during a solar storm.",
  "A horse in a fairy tale kingdom, attending a magical ball where enchanted creatures dance under a starlit sky.",
  "A horse in a culinary competition, showcasing its gourmet cooking skills in a high-stakes cooking show.",
  "A horse in a botanical garden, where it accidentally eats a rare plant and gains the ability to communicate with plants.",
  "A horse in a parallel universe, where animals have evolved to develop human-like societies and technology.",
#   "A horse in a concert hall, performing a symphony composed specifically for equine musicians.",
#   "A horse in a futuristic hospital, assisting doctors in performing groundbreaking medical procedures using advanced technology.",
#   "A horse in a spy mission, infiltrating a secret enemy base to retrieve stolen top-secret documents.",
#   "A horse in a space opera, piloting a starship through asteroid fields while engaging in epic space battles.",
#   "A horse in a post-apocalyptic wasteland, leading a group of survivors in a quest to find a safe haven.",
#   "A horse in a fantasy realm, battling mythical creatures and uncovering ancient prophecies.",
#   "A horse in a sports stadium, participating in a futuristic sport where teams compete using jetpacks and laser cannons.",
#   "A horse in a laboratory experiment, undergoing genetic modification that gives it enhanced intelligence and abilities.",
#   "A horse in a submarine adventure, exploring the depths of the ocean and encountering undiscovered sea creatures.",
#   "A horse in a steampunk city, navigating airships and solving mysteries in a world powered by steam and gears.",
#   "A horse in a wildlife documentary, being studied by scientists for its unique ability to communicate with other species.",
#   "A horse in a cyberpunk dystopia, hacking into corporate databases to expose corruption and injustice.",
#   "A horse in a magical forest, where it encounters mythical creatures and learns ancient spells from wise old wizards.",
#   "A horse in a time-traveling adventure, visiting different historical eras and influencing key events in history.",
#   "A horse in a virtual reality game, competing against players from around the world in a quest for digital supremacy.",
#   "A horse in a medieval fantasy realm, on a quest to rescue a kidnapped princess from an evil sorcerer's tower.",
#   "A horse in a futuristic cityscape, where it serves as a police mount in a high-tech urban environment.",
#   "A horse in a zombie apocalypse, leading a group of survivors in a desperate bid for survival against hordes of undead.",
#   "A horse in a celestial observatory, discovering a new star system and unraveling the mysteries of the universe.",
#   "A horse in a time-traveling adventure, accidentally causing a rift in the space-time continuum and altering history.",
#   "A horse in a fantasy realm, embarking on a quest to retrieve a legendary artifact from the lair of a fearsome dragon.",
#   "A horse in a submarine exploration, discovering an underwater city inhabited by ancient sea creatures.",
#   "A horse in a space colony, where it becomes a beloved mascot and symbol of unity among colonists from different planets.",
#   "A horse in a magical school, where it learns spells and potions alongside young wizards and witches.",
#   "A horse in a futuristic battlefield, equipped with advanced weaponry and armor, leading a charge against enemy forces.",
#   "A horse in a parallel universe, where it encounters alternate versions of itself and teams up to solve interdimensional mysteries.",
#   "A horse in a cyberpunk dystopia, leading a rebellion against oppressive AI overlords who control society.",
#   "A horse in a time-traveling adventure, meeting famous historical figures and influencing their decisions.",
#   "A horse in a superhero team, using its unique powers to protect the city from supervillains and alien threats.",
#   "A horse in a space exploration mission, landing on a distant planet and discovering alien life forms.",
#   "A horse in a post-apocalyptic wasteland, where it forms an unlikely alliance with mutant creatures to survive.",
#   "A horse in a virtual reality game, where it competes in gladiatorial battles and quests for digital fame and fortune.",
#   "A horse in a medieval fantasy realm, where it serves as a loyal steed to a brave knight on a quest for honor and glory.",
#   "A horse in a futuristic cityscape, where it becomes a symbol of resistance in a society ruled by corrupt corporations.",
#   "A horse in a zombie apocalypse, leading a group of survivors in a daring escape from a city overrun by the undead.",
#   "A horse in a celestial observatory, where it discovers a comet on a collision course with Earth and must find a way to prevent disaster.",
#   "A horse in a time-traveling adventure, where it accidentally alters the outcome of a crucial historical event and must set things right.",
#   "A horse in a fantasy realm, where it embarks on a quest to find the legendary Sword of Destiny and fulfill an ancient prophecy.",
#   "A horse in a submarine exploration, where it discovers a lost underwater civilization and unravels the mysteries of its downfall.",
#   "A horse in a space colony, where it becomes a beloved companion to colonists and assists in terraforming efforts on a distant planet.",
#   "A horse in a magical school, where it learns to harness its innate magical abilities and becomes a star pupil.",
#   "A horse in a futuristic battlefield, where it pilots a state-of-the-art mech suit and leads a resistance against invading alien forces.",
#   "A horse in a parallel universe, where it encounters alternate versions of itself and must work together to solve a cosmic mystery.",
#   "A horse in a cyberpunk dystopia, where it joins a group of hackers and activists to overthrow a tyrannical AI regime.",
#   "A horse in a time-traveling adventure, where it meets famous historical figures and plays a crucial role in shaping the course of history.",
#   "A horse in a superhero team, where it uses its extraordinary powers to protect the city from supervillains and global threats.",
#   "A horse in a space exploration mission, where it lands on a distant planet and discovers alien life forms that challenge its understanding of the universe.",
#   "A horse in a post-apocalyptic wasteland, where it forms an unlikely alliance with mutant creatures to survive in a world ravaged by nuclear war.",
#   "A horse in a virtual reality game, where it competes in gladiatorial battles and quests for digital fame and fortune.",
#   "A horse in a medieval fantasy realm, where it serves as a loyal steed to a brave knight on a quest for honor and glory.",
#   "A horse in a futuristic cityscape, where it becomes a symbol of resistance in a society ruled by corrupt corporations.",
#   "A horse in a zombie apocalypse, leading a group of survivors in a daring escape from a city overrun by the undead.",
#   "A horse in a celestial observatory, where it discovers a comet on a collision course with Earth and must find a way to prevent disaster.",
#   "A horse in a time-traveling adventure, where it accidentally alters the outcome of a crucial historical event and must set things right.",
#   "A horse in a fantasy realm, where it embarks on a quest to find the legendary Sword of Destiny and fulfill an ancient prophecy.",
#   "A horse in a submarine exploration, where it discovers a lost underwater civilization and unravels the mysteries of its downfall.",
#   "A horse in a space colony, where it becomes a beloved companion to colonists and assists in terraforming efforts on a distant planet.",
#   "A horse in a magical school, where it learns to harness its innate magical abilities and becomes a star pupil.",
#   "A horse in a futuristic battlefield, where it pilots a state-of-the-art mech suit and leads a resistance against invading alien forces.",
#   "A horse in a parallel universe, where it encounters alternate versions of itself and must work together to solve a cosmic mystery.",
#   "A horse in a cyberpunk dystopia, where it joins a group of hackers and activists to overthrow a tyrannical AI regime.",
#   "A horse in a time-traveling adventure, where it meets famous historical figures and plays a crucial role in shaping the course of history.",
#   "A horse in a superhero team, where it uses its extraordinary powers to protect the city from supervillains and global threats.",
#   "A horse in a space exploration mission, where it lands on a distant planet and discovers alien life forms that challenge its understanding of the universe.",
  "A cow wearing a top hat and monocle while riding a unicycle through a busy city intersection.",
  "A cow with wings delivering mail to a remote mountain hut during a blizzard.",
  "A cow conducting a symphony orchestra in a grand concert hall.",
  "A cow driving a submarine through an underwater cave system.",
  "A cow dressed as a detective solving a mystery in a haunted mansion.",
  "A cow working as a lifeguard at a crowded beach, blowing a whistle and wearing floaties.",
  "A cow as a chef in a five-star restaurant, preparing a gourmet meal for demanding customers.",
  "A cow as a librarian organizing books on a floating library in a magical forest.",
  "A cow as a tour guide leading a group of penguins on a sightseeing tour of Antarctica.",
  "A cow as a scientist conducting experiments with a time machine in a laboratory.",
  "A cow as an astronaut repairing a malfunctioning satellite in outer space.",
  "A cow as a magician performing tricks in front of an amazed audience.",
  "A cow as a pirate captain leading a crew on a treasure hunt in the Caribbean.",
  "A cow as a teacher instructing a class of aliens in a futuristic classroom.",
  "A cow as a ninja stealthily infiltrating a high-security corporate office.",
  "A cow as a race car driver competing in the Formula 1 championship.",
  "A cow as a fashion designer showcasing a new collection at Paris Fashion Week.",
  "A cow as a superhero rescuing citizens from a burning skyscraper.",
  "A cow as a diplomat negotiating a peace treaty between warring factions of garden gnomes.",
  "A cow as a rock star performing a concert in front of a massive crowd.",
#   "A cow as a sushi chef preparing sushi rolls with precision and expertise.",
#   "A cow as a deep-sea diver exploring a coral reef and discovering hidden treasures.",
#   "A cow as a robot engineer designing and building a new generation of robots.",
#   "A cow as a firefighter rescuing kittens from a tree in a busy urban neighborhood.",
#   "A cow as a painter creating a masterpiece on a canvas in an art gallery.",
#   "A cow as a circus performer walking a tightrope with circus animals watching below.",
#   "A cow as a meteorologist predicting the weather on a futuristic holographic display.",
#   "A cow as a film director shooting a blockbuster movie on location in the Amazon rainforest.",
#   "A cow as a gardener tending to a garden filled with exotic plants and flowers.",
#   "A cow as a detective solving a crime in a noir-style cityscape.",
#   "A cow as a marathon runner competing in an ultramarathon across desert terrain.",
#   "A cow as a space explorer navigating through asteroid fields in a spaceship.",
#   "A cow as a chef competing in a cooking competition judged by famous food critics.",
#   "A cow as a surgeon performing a delicate operation in a high-tech hospital.",
#   "A cow as a pilot flying a hot air balloon over picturesque countryside.",
#   "A cow as an archaeologist uncovering ancient artifacts in a remote jungle temple.",
#   "A cow as a music conductor leading a choir of singing birds in a magical forest.",
#   "A cow as a mountain climber scaling the tallest peak in the Himalayas.",
#   "A cow as a spy infiltrating a secret underground bunker.",
#   "A cow as a park ranger protecting endangered species in a wildlife sanctuary.",
#   "A cow as a photographer capturing stunning images of wildlife in Africa.",
#   "A cow as a baker preparing delicious pastries in a quaint village bakery.",
#   "A cow as a diver exploring a sunken shipwreck filled with lost treasure.",
#   "A cow as a racehorse jockey competing in the Kentucky Derby.",
#   "A cow as a game developer designing a virtual reality adventure game.",
#   "A cow as a ballet dancer performing a graceful routine on stage.",
#   "A cow as a scientist conducting experiments on a remote island laboratory.",
#   "A cow as a ski instructor teaching penguins how to ski on an icy slope.",
#   "A cow as an explorer discovering a hidden civilization in the depths of the Amazon.",
#   "A cow as a chef cooking up a storm in a top-secret government bunker.",
#   "A cow as a detective solving mysteries in a futuristic cityscape.",
#   "A cow as a spy infiltrating an international criminal organization.",
#   "A cow as a musician performing a concert for a crowd of woodland creatures.",
#   "A cow as an astronaut exploring the surface of Mars in a futuristic rover.",
  "An elephant wearing a top hat and monocle, attempting to navigate a crowded subway during rush hour.",
  "An elephant trying to blend in at a dog obedience school, participating in a sit-stay exercise.",
  "An elephant using a smartphone to order a coffee from a tiny espresso stand in a busy urban park.",
  "An elephant playing hide-and-seek in a cluttered antique shop filled with delicate porcelain figurines.",
  "An elephant attempting to parallel park a miniature car in a dollhouse-sized garage.",
  "An elephant receiving a high-tech spa treatment with lasers and massage chairs designed for humans.",
  "An elephant participating in a hot air balloon race alongside conventional hot air balloons.",
  "An elephant learning to salsa dance with a group of experienced ballroom dancers.",
  "An elephant hosting a cooking show where it prepares gourmet meals using tiny kitchen utensils.",
  "An elephant trying to board a space shuttle bound for Mars, surrounded by astronauts in full gear.",
  "An elephant attending a formal gala, dressed in a tuxedo and mingling with guests.",
  "An elephant navigating a complex obstacle course designed for Olympic gymnasts.",
  "An elephant performing a solo in a choir of human singers at a prestigious concert hall.",
  "An elephant attempting to assemble an intricate Lego set designed for children.",
  "An elephant competing in a professional soccer match against human athletes.",
  "An elephant riding a miniature horse in a tiny Wild West-themed town.",
  "An elephant participating in a synchronized swimming competition with human swimmers.",
  "An elephant participating in a fashion show, showcasing designer clothes made for smaller animals.",
  "An elephant attempting to solve a Rubik's Cube in a speedcubing competition.",
  "An elephant trying to fly a small remote-controlled airplane through a series of hoops.",
#   "An elephant attending a lecture at a university, sitting among students in an oversized lecture hall.",
#   "An elephant practicing yoga in a crowded studio filled with human practitioners.",
#   "An elephant attempting to play a grand piano designed for children.",
#   "An elephant working as a lifeguard at a busy beach, wearing oversized sunglasses and carrying a whistle.",
#   "An elephant trying to board a crowded subway during a blizzard, carrying a large snow shovel.",
#   "An elephant participating in a breakdancing competition against human dancers.",
#   "An elephant trying to navigate a hedge maze designed for children at a theme park.",
#   "An elephant attending a rock concert, standing among a crowd of enthusiastic fans.",
#   "An elephant attempting to perform magic tricks at a children's birthday party.",
#   "An elephant trying to skateboard down a steep ramp in a skatepark designed for tiny animals.",
#   "An elephant participating in a marathon race through city streets filled with human runners.",
#   "An elephant attempting to ski down a steep slope in a snowy mountain resort.",
#   "An elephant trying to navigate a busy shopping mall during a massive sale event.",
#   "An elephant attempting to climb a rock climbing wall designed for children.",
#   "An elephant participating in a game of chess against a grandmaster, using oversized chess pieces.",
#   "An elephant attending a high-stakes poker game in a glamorous casino.",
#   "An elephant attempting to perform in a circus trapeze act, swinging alongside human performers.",
#   "An elephant trying to drive a miniature electric car in a children's amusement park.",
#   "An elephant attending a formal ballet performance, seated in an oversized theater seat.",
#   "An elephant attempting to ride a mechanical bull at a country fair.",
#   "An elephant trying to navigate a high ropes course designed for adventure seekers.",
#   "An elephant participating in a professional wrestling match against human wrestlers.",
#   "An elephant attempting to perform a stand-up comedy routine on a tiny stage in a comedy club.",
#   "An elephant trying to navigate a crowded city sidewalk during a parade.",
#   "An elephant attempting to play miniature golf on a course designed for children.",
#   "An elephant participating in a yoga retreat, surrounded by human participants in meditation.",
#   "An elephant trying to operate a food truck selling tiny portions of gourmet cuisine.",
#   "An elephant attempting to surf on a tiny surfboard in a large ocean wave.",
#   "An elephant attending a ballet class, learning intricate dance moves alongside human dancers.",
#   "An elephant trying to navigate a hedge maze designed for children at a botanical garden.",
#   "An elephant participating in a professional basketball game against human players.",
  "A bird with neon feathers playing a grand piano underwater",
  "A bird made of glass flying through a city of mirrors",
  "A bird wearing a knight's armor fighting a dragon",
  "A bird made of clouds soaring through a thunderstorm",
  "A bird with mechanical wings performing surgery on another bird",
  "A bird with a paintbrush tail painting a landscape on a giant canvas",
  "A bird sitting on a throne made of diamonds in an alien palace",
  "A bird playing chess with a robot in a futuristic skyscraper",
  "A bird with butterfly wings flying through a garden of giant flowers",
  "A bird conducting an orchestra of other birds in a floating concert hall",
  "A bird wearing a wizard's hat casting spells in a magical forest",
  "A bird made of fire flying through a volcanic eruption",
  "A bird dressed as a pirate sailing a ship in a stormy sea",
  "A bird sculpting a statue out of ice in the Arctic",
  "A bird with a guitar performing at a rock concert in space",
  "A bird reading a giant book in an ancient library with endless shelves",
  "A bird wearing a spacesuit exploring an alien planet",
  "A bird with jewel-encrusted feathers dancing at a royal ball",
  "A bird with rainbow-colored wings flying through a sky full of hot air balloons",
  "A bird made of gold flying through a city made of gears and clockwork",
#   "A bird with neon lights on its feathers flying through a cyberpunk cityscape",
#   "A bird sitting on a giant mushroom in a forest of glowing plants",
#   "A bird riding a unicorn through a mystical realm",
#   "A bird blowing glass ornaments in a fantastical workshop",
#   "A bird painting graffiti on the walls of an abandoned castle",
#   "A bird with holographic wings flying through a virtual reality landscape",
#   "A bird performing acrobatics on a tightrope between two skyscrapers",
#   "A bird made of gemstones flying through a crystal cave",
#   "A bird surfing on a wave made of lava in a fiery ocean",
#   "A bird wearing a steampunk outfit flying through a Victorian city",
#   "A bird with a quill writing in a magical book that glows",
#   "A bird with starry feathers flying through a galaxy",
#   "A bird meditating on top of a floating lotus flower in a serene pond",
#   "A bird with a lightbulb for a head illuminating a dark forest",
#   "A bird playing a harp made of spider silk in a moonlit meadow",
#   "A bird dancing with fireflies in an enchanted forest",
#   "A bird with wings made of leaves flying through an autumn landscape",
#   "A bird wearing a crown made of stars sitting on a crescent moon",
#   "A bird carving intricate designs into a giant tree",
#   "A bird conducting a light show in a sky filled with auroras",
#   "A bird swimming through a sea of stars",
#   "A bird creating a tornado by flapping its wings",
#   "A bird with crystal wings flying through a winter wonderland",
#   "A bird wearing a samurai armor in a bamboo forest",
#   "A bird flying through a labyrinth of mirrors",
#   "A bird with wings that change colors like a chameleon",
#   "A bird guiding a ship through a stormy night with its glowing feathers",
#   "A bird building a nest out of gemstones in a cave",
#   "A bird made of music notes flying through a symphony",
#   "A bird juggling planets in space",
#   "A bird weaving a tapestry with its beak in a mystical workshop",
#   "A bird playing a flute that controls the weather",
#   "A bird performing a ballet on a stage made of clouds",
#   "A bird with a rainbow tail flying through a waterfall",
#   "A bird creating a mural on a skyscraper with its beak",
#   "A bird wearing a superhero cape saving another bird",
#   "A bird with a crystal beak breaking through a diamond wall",
#   "A bird planting seeds that grow into glowing flowers",
#   "A bird with metallic feathers reflecting the city lights at night",
#   "A bird crafting jewelry out of moonstones in an ancient temple",
#   "A bird with neon plumage flying through a futuristic cityscape",
#   "A bird performing magic tricks on a stage in a dark forest",
#   "A bird with feathered wings flying through a landscape of floating islands",
#   "A bird wearing a golden mask flying through a masquerade ball",
#   "A bird creating waves in a sea of sand dunes",
#   "A bird exploring a labyrinth made of glass",
#   "A bird with a sword fighting a shadowy figure in a dark alley",
#   "A bird conducting a fireworks display in a city at night",
#   "A bird with luminous feathers lighting up a cave full of stalactites",
#   "A bird planting a tree that grows instantly in a barren desert",
#   "A bird flying through a city where buildings are made of books",
#   "A bird playing a violin made of ice in a snow-covered forest",
#   "A bird with wings that shimmer like oil on water",
#   "A bird performing a tightrope walk between two hot air balloons",
#   "A bird crafting a stained glass window in a cathedral",
#   "A bird flying through a world made of candy",
#   "A bird exploring an underwater city made of coral",
#   "A bird with feathers that glow like embers flying through a dark forest",
#   "A bird wearing an astronaut helmet flying through space",
#   "A bird with a crown of flowers leading a parade of animals",
#   "A bird painting the sky with its wings at sunset",
#   "A bird with feathers made of liquid metal flying through an industrial landscape",
#   "A bird flying through a city where the buildings are made of giant seashells",
#   "A bird wearing a superhero mask and cape soaring above a city",
#   "A bird with wings like stained glass flying through a church",
#   "A bird creating a whirlpool in a sea of stars",
#   "A bird with lightning wings flying through a thunderstorm",
#   "A bird playing a piano made of ice on a frozen lake",
#   "A bird with rainbow-colored plumage dancing in a field of flowers",
#   "A bird exploring a forest where the trees are made of crystals",
#   "A bird wearing a jester's hat performing tricks in a medieval court",
#   "A bird with glowing feathers illuminating a dark cave",
#   "A bird flying through a city where the streets are rivers of lava",
#   "A bird creating a sandstorm in a desert with its wings",
#   "A bird with feathers that change colors with the seasons",
#   "A bird performing a concert on a stage made of stars",
#   "A bird flying through a forest where the leaves are made of gold",
#   "A bird wearing a crown and scepter ruling over a kingdom of animals",
#   "A bird with metallic wings flying through a futuristic metropolis",
#   "A bird painting a mural on the side of a mountain with its beak",
]

# Realistic prompts for image generation

In [ ]:
"""
realistic prompts by GPT-4o
- giraffe, 430 prompts
    - https://chatgpt.com/share/c715fcfc-adb9-41e5-835b-ae1aeb848d06
    - https://chatgpt.com/share/62369aca-7101-4936-a283-de4047e00775
- zebra, 410 prompts
    - https://chatgpt.com/share/bc2570b1-387f-4c2f-aac8-b5e604996886
    - https://chatgpt.com/share/6aa0c1cb-662b-4d3b-9d1b-6ef4bcad4cc5
    - https://chatgpt.com/share/b68109de-e122-4216-b78e-bdffadc29d33
- bear
    - https://chatgpt.com/share/23d1bfa8-89cd-4ef1-8055-19b540573451
- sheep
    - https://chatgpt.com/share/f08e06ef-8110-4a57-9c63-3143043e16d8
- cat
    - https://chatgpt.com/share/2399cdec-866b-49cd-b5fc-3e3af32d9a79
- dog
    - https://chatgpt.com/share/28e73a56-137a-4313-8af6-432015a06a94
- horse
    - https://chatgpt.com/share/38bf99dd-d5a1-443f-8691-107fbc3a321f
- cow
    - https://chatgpt.com/share/57388743-bd64-4d06-b556-5620da1b9e6e
- elephant
    - https://chatgpt.com/share/a4f8cbc4-9c23-4fc0-afe2-8a91fbf2efde
- bird
    - https://chatgpt.com/share/47e9bdfa-1dd7-4bf1-ba97-cce98a0ea9d3
"""

realistic_prompts = [
  "A giraffe standing next to a tall tree in a savanna at dawn.",
  "A giraffe drinking from a small waterfall in a rocky landscape.",
  "A giraffe standing near a wooden picnic table in a park.",
  "A giraffe walking along a dirt road with mountains in the background.",
  "A giraffe reaching for leaves on a tree beside a field of sunflowers.",
  "A giraffe standing in a meadow with a wooden windmill in the distance.",
  "A giraffe drinking from a small, clear pond in a forest clearing.",
  "A giraffe standing near an old, stone bridge over a river.",
  "A giraffe walking through a field of tall, green grass.",
  "A giraffe reaching for leaves on a tree beside a rustic wooden barn.",
  "A giraffe standing in a meadow with colorful autumn leaves.",
  "A giraffe drinking from a shallow pool in a desert landscape.",
  "A giraffe standing next to a tall, leafy tree in a garden.",
  "A giraffe walking along a narrow trail in a mountainous region.",
  "A giraffe reaching for leaves on a tree beside a wooden fence.",
  "A giraffe standing in a field of tall, yellow daisies.",
  "A giraffe drinking from a small stream surrounded by ferns.",
  "A giraffe standing near an old, stone tower in a rural area.",
  "A giraffe walking through a field of blooming poppies.",
  "A giraffe reaching for leaves on a tree beside a stone wall.",
  "A giraffe standing in a meadow with rolling hills in the background.",
  "A giraffe drinking from a small pond surrounded by cattails.",
  "A giraffe standing next to a tall, slender tree in a park.",
  "A giraffe walking along a gravel path in a nature reserve.",
  "A giraffe reaching for leaves on a tree beside a wooden bridge.",
  "A giraffe standing in a field of tall, swaying grass.",
  "A giraffe drinking from a shallow pool in a forest clearing.",
  "A giraffe standing near an old, wooden fence in a meadow.",
  "A giraffe walking through a field of colorful wildflowers.",
  "A giraffe reaching for leaves on a tree beside a stone path.",
  "A giraffe standing in a meadow with a river flowing through it.",
  "A giraffe drinking from a small, clear stream in the mountains.",
  "A giraffe standing next to a tall, ancient tree in a park.",
  "A giraffe walking along a dirt path in a forest.",
  "A giraffe reaching for leaves on a tree beside a rustic barn.",
  "A giraffe standing in a field of bright, orange marigolds.",
  "A giraffe drinking from a shallow stream in a rocky area.",
  "A giraffe standing near a wooden bench in a quiet garden.",
  "A giraffe walking through a field of tall, golden grass.",
  "A giraffe reaching for leaves on a tree beside a narrow path.",
  "A giraffe standing in a meadow with tall, purple flowers.",
  "A giraffe drinking from a small pond in a lush valley.",
  "A giraffe standing next to a large tree in a forest.",
  "A giraffe walking along a narrow path in a nature reserve.",
  "A giraffe reaching for leaves on a tree beside a wooden fence.",
  "A giraffe standing in a field of bright, red poppies.",
  "A giraffe drinking from a clear, bubbling stream.",
  "A giraffe standing near an old, wooden barn in a rural area.",
  "A giraffe walking through a field of tall, green grass.",
  "A giraffe reaching for leaves on a tree beside a stone bridge.",
  "A giraffe standing in a meadow with a rainbow in the sky.",
  "A giraffe drinking from a shallow pool in a forest.",
  "A giraffe standing next to a tall, leafy tree in a park.",
  "A giraffe walking along a dirt path in a mountainous area.",
  "A giraffe reaching for leaves on a tree beside a wooden fence.",
  "A giraffe standing in a field of tall, yellow wildflowers.",
  "A giraffe drinking from a small stream surrounded by reeds.",
  "A giraffe standing near an old, stone wall in a meadow.",
  "A giraffe walking through a field of blooming daisies.",
  "A giraffe reaching for leaves on a tree beside a stone path.",
  "A giraffe standing in a meadow with rolling hills in the background.",
  "A giraffe drinking from a small, clear pond in a forest.",
  "A giraffe standing next to a tall, slender tree in a park.",
  "A giraffe walking along a gravel path in a botanical garden.",
  "A giraffe reaching for leaves on a tree beside a wooden bridge.",
  "A giraffe standing in a field of tall, swaying grass.",
  "A giraffe drinking from a shallow pool in a forest clearing.",
  "A giraffe standing near an old, wooden fence in a rural area.",
  "A giraffe walking through a field of colorful wildflowers.",
  "A giraffe reaching for leaves on a tree beside a stone wall.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a small pond in a forest clearing.",
  "A giraffe standing next to a tall, ancient tree in a park.",
  "A giraffe walking along a narrow path in a dense forest.",
  "A giraffe reaching for leaves on a tree beside a wooden barn.",
  "A giraffe standing in a field of bright, yellow marigolds.",
  "A giraffe drinking from a shallow stream in a rocky landscape.",
  "A giraffe standing near a wooden bench in a quiet garden.",
  "A giraffe walking through a field of tall, golden grass.",
  "A giraffe reaching for leaves on a tree beside a narrow dirt path.",
  "A giraffe standing in a meadow with tall, purple flowers.",
  "A giraffe drinking from a small pond in a lush, green valley.",
  "A giraffe standing next to a large tree in a forest.",
  "A giraffe walking along a narrow path in a botanical garden.",
  "A giraffe reaching for leaves on a tree beside a wooden fence.",
  "A giraffe standing in a field of bright, red poppies.",
  "A giraffe drinking from a clear, bubbling stream.",
  "A giraffe standing near an old, wooden barn in a rural area.",
  "A giraffe walking through a field of tall, green grass.",
  "A giraffe reaching for leaves on a tree beside a stone bridge.",
  "A giraffe standing in a meadow with a rainbow in the background.",
  "A giraffe drinking from a shallow pool in a forest clearing.",
  "A giraffe standing next to a tall, leafy tree in a park.",
  "A giraffe walking along a dirt path in a mountainous region.",
  "A giraffe reaching for leaves on a tree beside a wooden fence.",
  "A giraffe standing in a field of tall, yellow daisies.",
  "A giraffe drinking from a small stream surrounded by ferns.",
  "A giraffe standing near an old, stone wall in a meadow.",
  "A giraffe walking through a field of blooming poppies.",
  "A giraffe standing near an old, wooden fence in a grassy field.",
  "A giraffe walking along a winding trail in a dense jungle.",
  "A giraffe reaching for leaves on a tree beside a small pond.",
  "A giraffe standing in a meadow with a rainbow in the background.",
  "A giraffe drinking from a shallow, rocky stream.",
  "A giraffe standing next to a tall, weathered tree in a forest clearing.",
  "A giraffe walking through a field of tall, purple heather.",
  "A giraffe reaching for leaves on a tree beside a stone bridge.",
  "A giraffe standing in a lush, green field with wildflowers.",
  "A giraffe drinking from a small, clear stream in a forest.",
  "A giraffe standing near an old, stone well in a rural area.",
  "A giraffe walking along a narrow trail in a dense, tropical forest.",
  "A giraffe reaching for leaves on a tree beside a wooden bench.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a shallow pool in a rocky landscape.",
  "A giraffe standing next to a large, ancient oak tree.",
  "A giraffe walking through a field of tall, yellow wildflowers.",
  "A giraffe reaching for leaves on a tree beside a rustic stone fence.",
  "A giraffe standing in a clearing in a dense, misty forest.",
  "A giraffe drinking from a small, calm lake at sunset.",
  "A giraffe standing near a wooden bridge over a narrow creek.",
  "A giraffe walking along a winding path in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a small, wooden barn.",
  "A giraffe standing in a meadow with tall grass and wildflowers.",
  "A giraffe drinking from a clear, bubbling stream in a forest.",
  "A giraffe standing next to a tall, slender tree in a grassy field.",
  "A giraffe walking through a field of bright, orange poppies.",
  "A giraffe reaching for leaves on a tree beside a stone wall.",
  "A giraffe standing in a clearing with sunlight filtering through the trees.",
  "A giraffe drinking from a shallow, clear pool in a forest.",
  "A giraffe standing near an old, wooden cabin in a rural area.",
  "A giraffe walking along a narrow path in a dense, tropical jungle.",
  "A giraffe reaching for leaves on a tree beside a wooden fence.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a shallow, rocky stream.",
  "A giraffe standing next to a large, ancient tree in a forest clearing.",
  "A giraffe walking through a field of tall, purple flowers.",
  "A giraffe reaching for leaves on a tree beside a stone bridge.",
  "A giraffe standing in a lush, green field with wildflowers.",
  "A giraffe drinking from a small, clear stream in a forest.",
  "A giraffe standing near an old, stone wall in a rural area.",
  "A giraffe walking along a narrow trail in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a wooden bench.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a shallow pool in a rocky landscape.",
  "A giraffe standing next to a large, ancient oak tree.",
  "A giraffe walking through a field of tall, yellow wildflowers.",
  "A giraffe reaching for leaves on a tree beside a rustic stone fence.",
  "A giraffe standing in a clearing in a dense, misty forest.",
  "A giraffe drinking from a small, calm lake at sunset.",
  "A giraffe standing near a wooden bridge over a narrow creek.",
  "A giraffe walking along a winding path in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a small, wooden barn.",
  "A giraffe standing in a meadow with tall grass and wildflowers.",
  "A giraffe drinking from a clear, bubbling stream in a forest.",
  "A giraffe standing next to a tall, slender tree in a grassy field.",
  "A giraffe walking through a field of bright, orange poppies.",
  "A giraffe reaching for leaves on a tree beside a stone wall.",
  "A giraffe standing in a clearing with sunlight filtering through the trees.",
  "A giraffe drinking from a shallow, clear pool in a forest.",
  "A giraffe standing near an old, wooden cabin in a rural area.",
  "A giraffe walking along a narrow path in a dense, tropical jungle.",
  "A giraffe reaching for leaves on a tree beside a wooden fence.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a shallow, rocky stream.",
  "A giraffe standing next to a large, ancient tree in a forest clearing.",
  "A giraffe walking through a field of tall, purple flowers.",
  "A giraffe reaching for leaves on a tree beside a stone bridge.",
  "A giraffe standing in a lush, green field with wildflowers.",
  "A giraffe drinking from a small, clear stream in a forest.",
  "A giraffe standing near an old, stone wall in a rural area.",
  "A giraffe walking along a narrow trail in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a wooden bench.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a shallow pool in a rocky landscape.",
  "A giraffe standing next to a large, ancient oak tree.",
  "A giraffe walking through a field of tall, yellow wildflowers.",
  "A giraffe reaching for leaves on a tree beside a rustic stone fence.",
  "A giraffe standing in a clearing in a dense, misty forest.",
  "A giraffe drinking from a small, calm lake at sunset.",
  "A giraffe standing near a wooden bridge over a narrow creek.",
  "A giraffe walking along a winding path in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a small, wooden barn.",
  "A giraffe standing in a meadow with tall grass and wildflowers.",
  "A giraffe drinking from a clear, bubbling stream in a forest.",
  "A giraffe standing next to a tall, slender tree in a grassy field.",
  "A giraffe walking through a field of bright, orange poppies.",
  "A giraffe reaching for leaves on a tree beside a stone wall.",
  "A giraffe standing in a clearing with sunlight filtering through the trees.",
  "A giraffe drinking from a shallow, clear pool in a forest.",
  "A giraffe standing near an old, wooden cabin in a rural area.",
  "A giraffe walking along a narrow path in a dense, tropical jungle.",
  "A giraffe reaching for leaves on a tree beside a wooden fence.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a shallow, rocky stream.",
  "A giraffe standing next to a large, ancient tree in a forest clearing.",
  "A giraffe walking through a field of tall, purple flowers.",
  "A giraffe reaching for leaves on a tree beside a stone bridge.",
  "A giraffe standing in a lush, green field with wildflowers.",
  "A giraffe drinking from a small, clear stream in a forest.",
  "A giraffe standing near an old, stone wall in a rural area.",
  "A giraffe walking along a narrow trail in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a wooden bench.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a shallow pool in a rocky landscape.",
  "A giraffe standing next to a large, ancient oak tree.",
  "A giraffe walking through a field of tall, yellow wildflowers.",
  "A giraffe reaching for leaves on a tree beside a rustic stone fence.",
  "A giraffe standing in a clearing in a dense, misty forest.",
  "A giraffe drinking from a small, calm lake at sunset.",
  "A giraffe standing near a wooden bridge over a narrow creek.",
  "A giraffe walking along a winding path in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a small, wooden barn.",
  "A giraffe standing in a meadow with tall grass and wildflowers.",
  "A giraffe drinking from a clear, bubbling stream in a forest.",
  "A giraffe standing next to a tall, slender tree in a grassy field.",
  "A giraffe walking through a field of bright, orange poppies.",
  "A giraffe reaching for leaves on a tree beside a stone wall.",
  "A giraffe standing in a clearing with sunlight filtering through the trees.",
  "A giraffe drinking from a shallow, clear pool in a forest.",
  "A giraffe standing near an old, wooden cabin in a rural area.",
  "A giraffe walking along a narrow path in a dense, tropical jungle.",
  "A giraffe reaching for leaves on a tree beside a wooden fence.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a shallow, rocky stream.",
  "A giraffe standing next to a large, ancient tree in a forest clearing.",
  "A giraffe walking through a field of tall, purple flowers.",
  "A giraffe reaching for leaves on a tree beside a stone bridge.",
  "A giraffe standing in a lush, green field with wildflowers.",
  "A giraffe drinking from a small, clear stream in a forest.",
  "A giraffe standing near an old, stone wall in a rural area.",
  "A giraffe walking along a narrow trail in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a wooden bench.",
  "A giraffe standing in a meadow with a gentle stream flowing through it.",
  "A giraffe drinking from a shallow pool in a rocky landscape.",
  "A giraffe standing next to a large, ancient oak tree.",
  "A giraffe walking through a field of tall, yellow wildflowers.",
  "A giraffe reaching for leaves on a tree beside a rustic stone fence.",
  "A giraffe standing in a clearing in a dense, misty forest.",
  "A giraffe drinking from a small, calm lake at sunset.",
  "A giraffe standing near a wooden bridge over a narrow creek.",
  "A giraffe walking along a winding path in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a small, wooden barn.",
  "A giraffe standing in a meadow with tall grass and wildflowers.",
  "A giraffe standing at the edge of a dense forest, reaching for leaves on a tall tree.",
  "A giraffe drinking water from a small, clear pond at sunrise.",
  "A giraffe walking along a dirt path in a savanna during a golden sunset.",
  "A giraffe peeking over the top of a wooden fence in a rural farm setting.",
  "A giraffe standing in the middle of a large, open field with wildflowers blooming.",
  "A giraffe nibbling on the leaves of a tree next to an old stone barn.",
  "A giraffe walking gracefully through a misty, early morning landscape.",
  "A giraffe standing near a calm river with its reflection in the water.",
  "A giraffe reaching for leaves on a tree beside a weathered wooden bridge.",
  "A giraffe walking along a sandy path with tall grass on either side.",
  "A giraffe standing in a clearing in a dense jungle with sunlight filtering through the trees.",
  "A giraffe eating leaves from a tree near a small, rustic cabin in the woods.",
  "A giraffe walking across a meadow filled with colorful wildflowers.",
  "A giraffe standing next to an old windmill in a rural countryside.",
  "A giraffe drinking from a shallow stream in a lush, green valley.",
  "A giraffe standing on a hill overlooking a vast, rolling landscape at dawn.",
  "A giraffe reaching for leaves on a tall tree in the middle of a quiet park.",
  "A giraffe walking along a gravel path in a botanical garden.",
  "A giraffe standing in the shadow of a large, ancient tree.",
  "A giraffe eating leaves from a tree near a small, tranquil lake.",
  "A giraffe walking through a field of tall, golden grass.",
  "A giraffe standing beside a quaint wooden bridge over a small creek.",
  "A giraffe reaching for leaves on a tree next to an old, stone wall.",
  "A giraffe walking through a misty forest at dawn.",
  "A giraffe standing in a meadow with snow-capped mountains in the distance.",
  "A giraffe drinking from a small pond surrounded by reeds and water lilies.",
  "A giraffe standing in the middle of a field of blooming sunflowers.",
  "A giraffe reaching for leaves on a tree beside a narrow dirt road.",
  "A giraffe walking through a dense forest with sunlight streaming through the canopy.",
  "A giraffe standing in a clearing with tall, green grass and wildflowers.",
  "A giraffe drinking from a clear, mountain stream.",
  "A giraffe standing near a small, wooden barn in a rural area.",
  "A giraffe walking along a narrow path in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tall tree beside a stone path.",
  "A giraffe standing in a field of tall, swaying grass at sunset.",
  "A giraffe drinking from a small, clear lake in the mountains.",
  "A giraffe standing near an old, abandoned farmhouse.",
  "A giraffe walking along a dirt path lined with wildflowers.",
  "A giraffe reaching for leaves on a tree beside a rustic wooden fence.",
  "A giraffe standing in a lush, green meadow with rolling hills in the background.",
  "A giraffe drinking from a shallow stream in a dense forest.",
  "A giraffe standing next to a tall tree in a quiet, peaceful park.",
  "A giraffe walking through a field of blooming lavender.",
  "A giraffe reaching for leaves on a tree beside a small, wooden cabin.",
  "A giraffe standing in a meadow with wildflowers and tall grass.",
  "A giraffe drinking from a small pond surrounded by trees.",
  "A giraffe standing in a clearing with sunlight filtering through the trees.",
  "A giraffe reaching for leaves on a tall tree beside a dirt road.",
  "A giraffe walking through a field of tall, golden wheat.",
  "A giraffe standing near a small, rustic barn in the countryside.",
  "A giraffe drinking from a clear, mountain lake.",
  "A giraffe standing in a meadow with rolling hills and wildflowers.",
  "A giraffe walking along a narrow path in a dense forest.",
  "A giraffe reaching for leaves on a tree beside a stone wall.",
  "A giraffe standing in a field of blooming daisies.",
  "A giraffe drinking from a small stream in a lush, green valley.",
  "A giraffe standing near an old windmill in a rural area.",
  "A giraffe walking through a field of tall, swaying grass.",
  "A giraffe reaching for leaves on a tree beside a rustic wooden bridge.",
  "A giraffe standing in a meadow with snow-capped mountains in the distance.",
  "A giraffe drinking from a small pond surrounded by reeds.",
  "A giraffe standing in a field of blooming sunflowers.",
  "A giraffe walking along a gravel path in a botanical garden.",
  "A giraffe reaching for leaves on a tall tree beside a narrow dirt road.",
  "A giraffe standing in a clearing in a dense forest.",
  "A giraffe drinking from a clear, mountain stream.",
  "A giraffe standing near a small, wooden barn in the countryside.",
  "A giraffe walking along a narrow path lined with wildflowers.",
  "A giraffe reaching for leaves on a tree beside a stone path.",
  "A giraffe standing in a field of tall, golden grass at sunset.",
  "A giraffe drinking from a small, clear lake in the forest.",
  "A giraffe standing near an old, abandoned farmhouse in a rural area.",
  "A giraffe walking through a dense forest with sunlight filtering through the trees.",
  "A giraffe reaching for leaves on a tree beside a rustic wooden fence.",
  "A giraffe standing in a lush, green meadow with rolling hills.",
  "A giraffe drinking from a shallow stream in a dense, leafy forest.",
  "A giraffe standing next to a tall tree in a quiet park.",
  "A giraffe walking through a field of blooming lavender.",
  "A giraffe reaching for leaves on a tree beside a small wooden cabin.",
  "A giraffe standing in a meadow with wildflowers and tall grass.",
  "A giraffe drinking from a small pond surrounded by trees and reeds.",
  "A giraffe standing in a clearing with sunlight streaming through the trees.",
  "A giraffe reaching for leaves on a tall tree beside a dirt path.",
  "A giraffe walking through a field of tall, golden wheat.",
  "A giraffe standing near a small, rustic barn in a countryside area.",
  "A giraffe drinking from a clear, mountain lake in the forest.",
  "A giraffe standing in a meadow with rolling hills and wildflowers.",
  "A giraffe walking along a narrow path in a dense, leafy forest.",
  "A giraffe reaching for leaves on a tree beside a stone wall.",
  "A giraffe standing in a field of blooming daisies at sunrise.",
  "A giraffe drinking from a small stream in a lush valley.",
  "a giraffe drinking water from a serene river at sunset",
  "a giraffe walking through a bustling city street during the morning rush hour",
  "a giraffe standing in a meadow filled with wildflowers under a clear blue sky",
  "a giraffe browsing leaves from a tall tree in a dense forest",
  "a giraffe interacting with children at a zoo enclosure",
  "a giraffe peeking through the window of a countryside farmhouse",
  "a giraffe strolling along a sandy beach with waves crashing nearby",
  "a giraffe riding in the back of a pickup truck on a rural road",
  "a giraffe looking over a fence at a suburban backyard barbecue",
  "a giraffe standing under a streetlight in a quiet town square at night",
  "a giraffe inside a spacious art gallery, gazing at a large painting",
  "a giraffe standing in a foggy meadow at dawn",
  "a giraffe in a pumpkin patch during autumn, surrounded by colorful leaves",
  "a giraffe helping a farmer in a vast cornfield",
  "a giraffe standing near a group of campers around a bonfire",
  "a giraffe wandering through a busy outdoor market filled with vendors and shoppers",
  "a giraffe looking curiously at a playground full of children",
  "a giraffe standing at the edge of a cliff overlooking a canyon",
  "a giraffe walking through a quiet residential neighborhood on a snowy day",
  "a giraffe standing near a group of cyclists on a mountain trail",
  "a giraffe in the middle of a sunflower field during a clear day",
  "a giraffe bending down to drink from a small pond in a park",
  "a giraffe standing in the middle of a sports field during a local game",
  "a giraffe peering into the window of a quaint bookstore on a rainy day",
  "a giraffe lying down in a grassy field under a starry night sky",
  "a giraffe standing in the middle of a bustling train station",
  "a giraffe walking along a scenic hiking trail through the hills",
  "a giraffe standing on a bridge overlooking a peaceful river in a forest",
  "a giraffe standing beside a large windmill on a rural farm",
  "a giraffe grazing in an open field with a rainbow in the background",
  "a giraffe standing at the edge of a serene lake during the golden hour",
  "a giraffe walking along a path lined with cherry blossom trees in full bloom",
  "a giraffe standing in a snow-covered forest under a clear winter sky",
  "a giraffe observing a group of painters at an outdoor art class",
  "a giraffe near a rustic cabin in the mountains during a foggy morning",
  "a giraffe walking through a field of lavender on a sunny day",
  "a giraffe standing in a bustling town square during a summer festival",
  "a giraffe peering into a cozy café filled with people enjoying their meals",
  "a giraffe standing near a group of hikers resting at a scenic overlook",
  "a giraffe walking through a dense jungle with a waterfall in the background",
  "a giraffe standing in a vast desert landscape with sand dunes",
  "a giraffe walking along a country road with blooming flowers on both sides",
  "a giraffe peeking into a greenhouse filled with exotic plants",
  "a giraffe standing on a rocky shoreline with crashing waves",
  "a giraffe observing a family having a picnic in a park",
  "a giraffe standing at the edge of a vast wheat field during harvest time",
  "a giraffe looking at an artist painting on an easel in a meadow",
  "a giraffe walking through a vineyard during grape harvest season",
  "a giraffe standing near a carousel at an amusement park",
  "a giraffe walking along a dock with boats anchored nearby",
  "a giraffe standing at the entrance of a forest path with sunlight filtering through trees",
  "a giraffe observing a yoga class in a tranquil park",
  "a giraffe walking along a tree-lined boulevard during autumn",
  "a giraffe standing near a historic lighthouse by the sea",
  "a giraffe peeking into a tent at a campsite",
  "a giraffe standing in an open-air café on a sunny day",
  "a giraffe wandering through an abandoned industrial complex",
  "a giraffe observing a street musician performing in a lively square",
  "a giraffe walking along a rural dirt road with farmhouses in the distance",
  "a giraffe standing near a small pond with ducks swimming",
  "a giraffe standing in a vast prairie with bison grazing nearby",
  "a giraffe looking at a large fountain in the center of a city park",
  "a giraffe wandering through a forest clearing with deer",
  "a giraffe walking through a crowded festival with colorful decorations",
  "a giraffe standing near a construction site in a developing area",
  "a giraffe observing a group of surfers on a busy beach",
  "a giraffe standing in a field of tall grass with a lone tree in the distance",
  "a giraffe walking through a snowy village with smoke rising from chimneys",
  "a giraffe standing near a railway track with a passing train",
  "a giraffe observing a group of people practicing tai chi in a park",
  "a giraffe standing in a blooming orchard during spring",
  "a giraffe walking through a foggy wetland with birds flying overhead",
  "a giraffe standing at a scenic overlook with rolling hills in the distance",
  "a giraffe peering into a small workshop with tools and woodwork",
  "a giraffe standing near a stone bridge over a peaceful stream",
  "a giraffe walking along a lakeside path with canoes docked nearby",
  "a giraffe standing near a herd of cattle in a pasture",
  "a giraffe observing a group of dancers performing in an outdoor stage",
  "a giraffe standing near a large clock tower in a town square",
  "a giraffe walking through a quiet alleyway in a historic town",
  "a giraffe standing near a community garden in a suburban area",
  "a giraffe observing a potter working at a wheel in an open studio",
  "a giraffe standing near a rustic wooden barn on a clear day",
  "a giraffe walking through a maze of tall hedges in a formal garden",
  "a giraffe standing near a field of bluebonnets under a wide sky",
  "a giraffe observing a beekeeper tending to hives in an orchard",
  "a giraffe walking through a field of goldenrod in late summer",
  "a giraffe standing near a small chapel on a hillside",
  "a giraffe observing a group of volunteers cleaning a beach",
  "a giraffe walking along a forest path with fallen leaves",
  "a giraffe standing near a group of children flying kites in a park",
  "a giraffe observing a farmer's market with stalls and fresh produce",
  "a giraffe walking through a field of red poppies on a breezy day",
  "a giraffe standing near a bus stop in a busy urban area",
  "a giraffe observing a group of people painting a mural on a building",
  "a giraffe walking through a quiet countryside lane with stone walls",
  "A zebra standing on a grassy plain under a clear blue sky.",
  "A zebra drinking water from a small, serene pond surrounded by tall reeds.",
  "A zebra walking along a dirt path in a lush, green forest.",
  "A zebra grazing near a large, solitary tree in a vast savannah.",
  "A zebra standing on a rocky hillside with a panoramic view of the valley below.",
  "A zebra resting under the shade of an acacia tree during a hot afternoon.",
  "A zebra crossing a shallow river with pebbles visible beneath the water.",
  "A zebra standing in a field of wildflowers with a gentle breeze blowing.",
  "A zebra walking along a sandy beach with waves gently lapping at the shore.",
  "A zebra standing in a misty meadow at sunrise, the grass covered in dew.",
  "A zebra trotting through a small village with rustic huts and dirt roads.",
  "A zebra standing near a waterfall, the mist creating a rainbow in the background.",
  "A zebra grazing in a meadow with rolling hills in the distance.",
  "A zebra standing on a cliff edge overlooking a vast ocean.",
  "A zebra walking through a dry, arid desert with cacti scattered around.",
  "A zebra standing near a campfire, with tents and camping gear in the background.",
  "A zebra walking through a dense jungle, sunlight filtering through the canopy.",
  "A zebra standing in a field of tall, golden grass swaying in the wind.",
  "A zebra resting near a small, tranquil lake with mountains in the background.",
  "A zebra standing on a hilltop during a thunderstorm, lightning illuminating the sky.",
  "A zebra walking along a narrow trail in a dense forest with thick undergrowth.",
  "A zebra standing in a snowy landscape with a few bare trees around.",
  "A zebra grazing near an old, abandoned farmhouse in a rural area.",
  "A zebra walking along a cobblestone path in a quaint, historic town.",
  "A zebra standing on the edge of a canyon with a river flowing below.",
  "A zebra resting near a large, ancient tree with twisted branches.",
  "A zebra walking through a field of sunflowers under a bright, sunny sky.",
  "A zebra standing in a quiet, foggy valley early in the morning.",
  "A zebra grazing near a small, rustic bridge over a stream.",
  "A zebra walking along a dirt road with fields of wheat on either side.",
  "A zebra standing near a windmill on a breezy day.",
  "A zebra resting in the shade of a large boulder in a desert landscape.",
  "A zebra walking through an orchard with blossoming trees.",
  "A zebra standing near a wooden fence with a picturesque barn in the background.",
  "A zebra grazing on a hillside covered in lavender fields.",
  "A zebra walking along a narrow path with vineyards on either side.",
  "A zebra standing near a stone well in a quiet countryside.",
  "A zebra resting near a fallen log in a dense forest.",
  "A zebra walking through a meadow with a rainbow in the distance.",
  "A zebra standing near a small, rustic chapel in a serene setting.",
  "A zebra grazing near a crystal-clear mountain stream.",
  "A zebra walking along a dirt path with wildflowers on either side.",
  "A zebra standing on a sandy dune with a clear blue sky overhead.",
  "A zebra resting under the shade of a palm tree on a tropical island.",
  "A zebra walking along a narrow trail in the mountains.",
  "A zebra standing near a quaint cottage with a thatched roof.",
  "A zebra grazing in a meadow with butterflies fluttering around.",
  "A zebra walking through a field of poppies under a bright sun.",
  "A zebra standing near a stone wall covered in ivy.",
  "A zebra resting near a small, bubbling brook in a forest.",
  "A zebra walking along a path with tall grasses on either side.",
  "A zebra standing near an old, wooden bridge over a river.",
  "A zebra grazing near a rustic windmill in a rural area.",
  "A zebra walking through a field of daisies on a sunny day.",
  "A zebra standing near a large, ancient stone monument.",
  "A zebra resting near a pond with lily pads and reeds.",
  "A zebra walking along a sandy path with seashells scattered around.",
  "A zebra standing near a wooden pier extending into a lake.",
  "A zebra grazing near a small, secluded cabin in the woods.",
  "A zebra walking along a path lined with cherry blossom trees.",
  "A zebra standing near a rustic barn with a weathered roof.",
  "A zebra resting near a large rock formation in a desert.",
  "A zebra walking through a meadow with wildflowers in bloom.",
  "A zebra standing near a stone archway covered in vines.",
  "A zebra grazing near a tranquil pond with ducks swimming.",
  "A zebra walking along a narrow path with tall ferns on either side.",
  "A zebra standing near a rustic stone cottage in a hilly area.",
  "A zebra resting under the shade of a large oak tree.",
  "A zebra walking through a field of lavender under a blue sky.",
  "A zebra standing near a small waterfall in a forest.",
  "A zebra grazing near a wooden fence with a distant farmhouse.",
  "A zebra walking along a sandy beach with seagulls flying overhead.",
  "A zebra standing near a large, twisted tree in a meadow.",
  "A zebra resting near a bubbling stream in a quiet valley.",
  "A zebra walking through a field of tall, golden grass.",
  "A zebra standing near a stone fountain in a garden.",
  "A zebra grazing near a quiet, country road.",
  "A zebra walking along a dirt path in a sunflower field.",
  "A zebra standing near a small, rustic bridge over a creek.",
  "A zebra resting near a large, moss-covered boulder.",
  "A zebra walking through a meadow with a distant mountain range.",
  "A zebra standing near a quaint, wooden cabin.",
  "A zebra grazing near a tranquil lake with trees reflected in the water.",
  "A zebra walking along a narrow path in a dense forest.",
  "A zebra standing near a stone wall in a quiet countryside.",
  "A zebra resting near a small pond with water lilies.",
  "A zebra walking through a field of daisies in full bloom.",
  "A zebra standing near a rustic, weathered barn.",
  "A zebra grazing near a clear mountain stream.",
  "A zebra walking along a sandy trail with tall grasses.",
  "A zebra standing near an old, wooden bridge in a rural setting.",
  "A zebra resting near a large rock in a forest.",
  "A zebra walking through a meadow with a rainbow overhead.",
  "A zebra standing near a quaint stone cottage.",
  "A zebra grazing near a pond with ducks and reeds.",
  "A zebra walking along a path lined with wildflowers.",
  "A zebra grazing in a lush green meadow under a clear blue sky.",
  "A zebra standing on the edge of a calm river with its reflection visible in the water.",
  "A zebra walking along a dirt path in a dense forest during autumn.",
  "A zebra drinking from a small pond surrounded by tall grass.",
  "A zebra standing on a sandy beach with gentle waves in the background.",
  "A zebra lying down on a sunlit savannah with acacia trees in the distance.",
  "A zebra crossing a shallow stream with stones visible under the water.",
  "A zebra standing near a wooden fence on a farm during sunset.",
  "A zebra grazing next to a vibrant flower field with various colors.",
  "A zebra walking through a rocky canyon with towering cliffs on either side.",
  "A zebra standing in front of a rustic barn with hay bales around.",
  "A zebra walking along a narrow trail in a mountain valley.",
  "A zebra standing in a snow-covered field with snowflakes falling gently.",
  "A zebra standing beside a rustic wooden cabin in a forest.",
  "A zebra walking through a foggy meadow early in the morning.",
  "A zebra standing in a field of tall, golden wheat under a cloudy sky.",
  "A zebra standing on a hilltop overlooking a picturesque village.",
  "A zebra walking along a pebble-strewn beach with driftwood scattered around.",
  "A zebra standing under a large oak tree with fallen leaves around.",
  "A zebra crossing a wooden bridge over a small stream in a forest.",
  "A zebra standing near a windmill with tulip fields in the background.",
  "A zebra standing on a grassy knoll with wildflowers in bloom.",
  "A zebra walking along a cobblestone path in a quaint village.",
  "A zebra standing near a tranquil lake with mountains in the background.",
  "A zebra standing in a vineyard with rows of grapevines stretching out.",
  "A zebra standing on a pier with boats docked nearby.",
  "A zebra grazing in a meadow with a rainbow arcing across the sky.",
  "A zebra standing in a field of lavender with butterflies fluttering around.",
  "A zebra walking along a dirt road with old, weathered buildings nearby.",
  "A zebra standing in a sunflower field with bright yellow blooms.",
  "A zebra standing on a cliffside with the ocean waves crashing below.",
  "A zebra standing near an ancient stone bridge over a river.",
  "A zebra walking through a grassy meadow dotted with wildflowers.",
  "A zebra standing in a field with a rustic windmill turning slowly.",
  "A zebra grazing near a crystal-clear lake with fish swimming below.",
  "A zebra standing in front of an old, ivy-covered stone wall.",
  "A zebra walking along a railway track with fields on either side.",
  "A zebra standing near a fountain in a quaint village square.",
  "A zebra standing in a field of daisies with a gentle breeze blowing.",
  "A zebra standing on a sandy dune with sparse vegetation around.",
  "A zebra walking through a meadow with colorful kites flying overhead.",
  "A zebra standing near an abandoned farmhouse with overgrown weeds.",
  "A zebra standing in a lush green valley with a waterfall in the distance.",
  "A zebra grazing near a wooden dock with lily pads floating on the water.",
  "A zebra walking along a trail in a dense forest with sunlight filtering through.",
  "A zebra standing near a stone well with ivy creeping up its sides.",
  "A zebra standing in a meadow with a hot air balloon floating in the sky.",
  "A zebra walking through a grassy field with old, wooden wagon wheels.",
  "A zebra standing on a hill with rolling green fields below.",
  "A zebra standing near a rustic wooden bridge over a babbling brook.",
  "A zebra grazing in a field of poppies with vibrant red petals.",
  "A zebra walking along a sandy path with wildflowers on either side.",
  "A zebra standing in front of a stone cottage with a thatched roof.",
  "A zebra standing in a field of corn with tall stalks swaying in the breeze.",
  "A zebra standing near a small wooden cabin in a snowy landscape.",
  "A zebra walking along a trail with autumn leaves crunching underfoot.",
  "A zebra standing in a meadow with a windmill and blooming flowers.",
  "A zebra grazing near a small pond with lily pads and reeds.",
  "A zebra standing in front of a rustic barn with a wooden fence.",
  "A zebra walking through a grassy meadow with a stone wall in the distance.",
  "A zebra standing near a quaint stone cottage with ivy-covered walls.",
  "A zebra standing in a field of golden sunflowers with a clear sky.",
  "A zebra standing near an old windmill with wildflowers blooming around.",
  "A zebra standing in a lush green meadow with a rainbow overhead.",
  "A zebra standing on a hilltop with a panoramic view of the countryside.",
  "A zebra grazing in a meadow with colorful hot air balloons in the sky.",
  "A zebra standing near a wooden dock with a lake and mountains behind.",
  "A zebra walking along a dirt path with tall grass on either side.",
  "A zebra standing in a field of lavender with bees buzzing around.",
  "A zebra standing near an old stone bridge with water flowing underneath.",
  "A zebra walking through a dense forest with rays of sunlight streaming through.",
  "A zebra standing near a rustic cabin with a thatched roof and a wooden fence.",
  "A zebra standing in a field of wildflowers with butterflies fluttering around.",
  "A zebra grazing near a crystal-clear stream with fish swimming below.",
  "A zebra standing on a sandy beach with gentle waves lapping at the shore.",
  "A zebra walking through a grassy meadow with a rainbow arcing overhead.",
  "A zebra standing near a wooden windmill with tulip fields in the background.",
  "A zebra standing in a meadow with a vibrant sunset casting a warm glow.",
  "A zebra standing near a stone well with ivy creeping up the sides.",
  "A zebra walking along a cobblestone path with quaint cottages on either side.",
  "A zebra standing in a sunflower field with bright yellow blooms all around.",
  "A zebra standing on a hilltop with rolling green fields stretching out below.",
  "A zebra standing near a wooden bridge over a babbling brook in a forest.",
  "A zebra grazing in a field of poppies with vibrant red petals.",
  "A zebra walking along a sandy path with wildflowers blooming on either side.",
  "A zebra standing in front of a stone cottage with a thatched roof and a garden.",
  "A zebra standing in a field of corn with tall stalks swaying in the breeze.",
  "A zebra standing near a small wooden cabin in a snowy landscape with falling snow.",
  "A zebra walking along a trail with autumn leaves crunching underfoot.",
  "A zebra standing in a meadow with a windmill turning slowly in the distance.",
  "A zebra grazing near a small pond with lily pads and reeds along the edges.",
  "A zebra standing in front of a rustic barn with a wooden fence and hay bales.",
  "A zebra walking through a grassy meadow with a stone wall and wildflowers.",
  "A zebra standing near a quaint stone cottage with ivy-covered walls and a garden.",
  "A zebra standing in a field of golden sunflowers with a bright blue sky.",
  "A zebra standing near an old windmill with wildflowers blooming in the foreground.",
  "A zebra standing in a lush green meadow with a rainbow in the sky.",
  "A zebra standing on a hilltop with a panoramic view of the countryside and forests.",
  "A zebra grazing in a meadow with colorful hot air balloons drifting overhead.",
  "A zebra standing near a wooden dock with a lake and mountains in the background.",
  "A zebra walking along a dirt path with tall grass swaying in the breeze.",
  "A zebra standing in a field of lavender with bees buzzing around the flowers.",
  "A zebra standing near an old stone bridge with water flowing gently underneath.",
  "A zebra walking through a dense forest with rays of sunlight filtering through the trees.",
  "A zebra standing near a rustic cabin with a thatched roof and a wooden fence around it.",
  "A zebra standing in a field of wildflowers with butterflies fluttering among the blooms.",
  "A zebra grazing near a crystal-clear stream with fish swimming in the water.",
  "A zebra standing on a sandy beach with gentle waves lapping at the shoreline.",
  "A zebra walking through a grassy meadow with a rainbow arching overhead.",
  "A zebra standing near a wooden windmill with tulip fields in full bloom.",
  "A zebra standing in a meadow with a vibrant sunset casting a warm glow over the landscape.",
  "A zebra standing near a stone well with ivy creeping up the sides and flowers blooming around.",
  "A zebra walking along a cobblestone path with quaint cottages and gardens on either side.",
  "A zebra standing in a sunflower field with bright yellow blooms stretching out as far as the eye can see.",
  "A zebra standing on a hilltop with rolling green fields and a clear blue sky.",
  "A zebra standing near a wooden bridge over a babbling brook in a peaceful forest.",
  "A zebra grazing in a field of poppies with vibrant red petals swaying in the breeze.",
  "A zebra walking along a sandy path with wildflowers blooming on either side and a clear sky above.",
  "A zebra standing in front of a stone cottage with a thatched roof and a garden full of colorful flowers.",
  "A zebra standing in a field of corn with tall stalks swaying in the breeze and a clear blue sky.",
  "A zebra standing near a small wooden cabin in a snowy landscape with falling snow and a cozy fire inside.",
  "A zebra walking along a trail with autumn leaves crunching underfoot and trees ablaze with fall colors.",
  "A zebra standing in a meadow with a windmill turning slowly in the distance and wildflowers in bloom.",
  "A zebra grazing near a small pond with lily pads and reeds along the edges and a clear blue sky.",
  "A zebra standing in front of a rustic barn with a wooden fence and hay bales scattered around.",
  "A zebra walking through a grassy meadow with a stone wall and wildflowers in bloom and a clear blue sky.",
  "A zebra standing near a quaint stone cottage with ivy-covered walls and a garden full of colorful flowers.",
  "A zebra standing in a field of golden sunflowers with a bright blue sky and a gentle breeze blowing.",
  "A zebra standing near an old windmill with wildflowers blooming in the foreground and a clear blue sky.",
  "A zebra standing in a lush green meadow with a rainbow arching across the sky.",
  "A zebra standing on a hilltop with a panoramic view of the countryside and forests stretching out below.",
  "A zebra grazing in a meadow with colorful hot air balloons drifting overhead and a clear blue sky.",
  "A zebra standing near a wooden dock with a lake and mountains in the background and a gentle breeze blowing.",
  "A zebra walking along a dirt path with tall grass swaying in the breeze and a clear blue sky above.",
  "A zebra standing in a field of lavender with bees buzzing around the flowers and a clear blue sky.",
  "A zebra standing near an old stone bridge with water flowing gently underneath and a clear blue sky above.",
  "A zebra walking through a dense forest with rays of sunlight filtering through the trees and a gentle breeze blowing.",
  "A zebra standing near a rustic cabin with a thatched roof and a wooden fence around it and a clear blue sky.",
  "A zebra standing in a field of wildflowers with butterflies fluttering among the blooms and a clear blue sky above.",
  "A zebra grazing near a crystal-clear stream with fish swimming in the water and a clear blue sky overhead.",
  "A zebra standing on a sandy beach with gentle waves lapping at the shoreline and a clear blue sky above.",
  "A zebra walking through a grassy meadow with a rainbow arching overhead and a clear blue sky.",
  "A zebra standing near a wooden windmill with tulip fields in full bloom and a clear blue sky above.",
  "A zebra standing in a meadow with a vibrant sunset casting a warm glow over the landscape and a clear sky.",
  "A zebra standing near a stone well with ivy creeping up the sides and flowers blooming around and a clear blue sky.",
  "A zebra walking along a cobblestone path with quaint cottages and gardens on either side and a clear blue sky above.",
  "A zebra standing in a sunflower field with bright yellow blooms stretching out as far as the eye can see and a clear sky.",
  "A zebra standing on a hilltop with rolling green fields and a clear blue sky stretching out below.",
  "A zebra standing near a wooden bridge over a babbling brook in a peaceful forest with a clear blue sky above.",
  "A zebra grazing in a field of poppies with vibrant red petals swaying in the breeze and a clear blue sky above.",
  "A zebra walking along a sandy path with wildflowers blooming on either side and a clear blue sky overhead.",
  "A zebra standing in front of a stone cottage with a thatched roof and a garden full of colorful flowers and a clear sky.",
  "A zebra standing in a field of corn with tall stalks swaying in the breeze and a clear blue sky above.",
  "A zebra standing near a small wooden cabin in a snowy landscape with falling snow and a cozy fire inside.",
  "A zebra walking along a trail with autumn leaves crunching underfoot and trees ablaze with fall colors and a clear sky.",
  "A zebra standing in a meadow with a windmill turning slowly in the distance and wildflowers in bloom and a clear sky.",
  "A zebra grazing near a small pond with lily pads and reeds along the edges and a clear blue sky above.",
  "A zebra standing in front of a rustic barn with a wooden fence and hay bales scattered around and a clear blue sky.",
  "A zebra walking through a grassy meadow with a stone wall and wildflowers in bloom and a clear blue sky above.",
  "A zebra standing near a quaint stone cottage with ivy-covered walls and a garden full of colorful flowers and a clear sky.",
  "A zebra standing in a field of golden sunflowers with a bright blue sky and a gentle breeze blowing.",
  "A zebra standing near an old windmill with wildflowers blooming in the foreground and a clear blue sky above.",
  "A zebra standing in a lush green meadow with a rainbow arching across the sky and a clear blue sky.",
  "A zebra standing on a hilltop with a panoramic view of the countryside and forests stretching out below and a clear sky.",
  "A zebra grazing in a meadow with colorful hot air balloons drifting overhead and a clear blue sky above.",
  "A zebra standing near a wooden dock with a lake and mountains in the background and a gentle breeze blowing.",
  "A zebra walking along a dirt path with tall grass swaying in the breeze and a clear blue sky overhead.",
  "A zebra standing in a field of lavender with bees buzzing around the flowers and a clear blue sky above.",
  "A zebra standing near an old stone bridge with water flowing gently underneath and a clear blue sky above.",
  "A zebra walking through a dense forest with rays of sunlight filtering through the trees and a clear blue sky above.",
  "A zebra standing near a rustic cabin with a thatched roof and a wooden fence around it and a clear blue sky overhead.",
  "A zebra standing in a field of wildflowers with butterflies fluttering among the blooms and a clear blue sky above.",
  "A zebra grazing near a crystal-clear stream with fish swimming in the water and a clear blue sky above.",
  "A zebra standing on a sandy beach with gentle waves lapping at the shoreline and a clear blue sky above.",
  "A zebra walking through a grassy meadow with a rainbow arching overhead and a clear blue sky above.",
  "A zebra standing near a wooden windmill with tulip fields in full bloom and a clear blue sky above.",
  "A zebra standing in a meadow with a vibrant sunset casting a warm glow over the landscape and a clear blue sky.",
  "A zebra standing near a stone well with ivy creeping up the sides and flowers blooming around and a clear blue sky above.",
  "A zebra walking along a cobblestone path with quaint cottages and gardens on either side and a clear blue sky overhead.",
  "A zebra standing in a sunflower field with bright yellow blooms stretching out as far as the eye can see and a clear blue sky above.",
  "A zebra standing on a hilltop with rolling green fields and a clear blue sky stretching out below.",
  "A zebra standing near a wooden bridge over a babbling brook in a peaceful forest with a clear blue sky above.",
  "A zebra grazing in a field of poppies with vibrant red petals swaying in the breeze and a clear blue sky above.",
  "A zebra walking along a sandy path with wildflowers blooming on either side and a clear blue sky above.",
  "A zebra standing in front of a stone cottage with a thatched roof and a garden full of colorful flowers and a clear blue sky above.",
  "A zebra standing in a field of corn with tall stalks swaying in the breeze and a clear blue sky above.",
  "A zebra standing near a small wooden cabin in a snowy landscape with falling snow and a clear blue sky above.",
  "A zebra walking along a trail with autumn leaves crunching underfoot and trees ablaze with fall colors and a clear blue sky above.",
  "A zebra standing in a meadow with a windmill turning slowly in the distance and wildflowers in bloom and a clear blue sky above.",
  "A zebra grazing near a small pond with lily pads and reeds along the edges and a clear blue sky above.",
  "A zebra standing in front of a rustic barn with a wooden fence and hay bales scattered around and a clear blue sky above.",
  "A zebra walking through a grassy meadow with a stone wall and wildflowers in bloom and a clear blue sky above.",
  "A zebra standing near a quaint stone cottage with ivy-covered walls and a garden full of colorful flowers and a clear blue sky above.",
  "A zebra standing in a field of golden sunflowers with a bright blue sky and a gentle breeze blowing.",
  "A zebra standing near an old windmill with wildflowers blooming in the foreground and a clear blue sky above.",
  "A zebra standing in a lush green meadow with a rainbow arching across the sky and a clear blue sky above.",
  "A zebra standing on a hilltop with a panoramic view of the countryside and forests stretching out below and a clear blue sky above.",
  "A zebra grazing in a meadow with colorful hot air balloons drifting overhead and a clear blue sky above.",
  "A zebra standing near a wooden dock with a lake and mountains in the background and a clear blue sky above.",
  "A zebra walking along a dirt path with tall grass swaying in the breeze and a clear blue sky above.",
  "A zebra standing in a field of lavender with bees buzzing around the flowers and a clear blue sky above.",
  "A zebra standing near an old stone bridge with water flowing gently underneath and a clear blue sky above.",
  "A zebra walking through a dense forest with rays of sunlight filtering through the trees and a clear blue sky above.",
  "A zebra standing near a rustic cabin with a thatched roof and a wooden fence around it and a clear blue sky above.",
  "A zebra standing in a field of wildflowers with butterflies fluttering among the blooms and a clear blue sky above.",
  "A zebra grazing near a crystal-clear stream with fish swimming in the water and a clear blue sky above.",
  "A zebra standing on a sandy beach with gentle waves lapping at the shoreline and a clear blue sky above.",
  "A zebra walking through a grassy meadow with a rainbow arching overhead and a clear blue sky above.",
  "A zebra standing near a wooden windmill with tulip fields in full bloom and a clear blue sky above.",
  "A zebra standing in a meadow with a vibrant sunset casting a warm glow over the landscape and a clear blue sky above.",
  "A zebra standing near a stone well with ivy creeping up the sides and flowers blooming around and a clear blue sky above.",
  "A zebra walking along a cobblestone path with quaint cottages and gardens on either side and a clear blue sky above.",
  "A zebra standing in a sunflower field with bright yellow blooms stretching out as far as the eye can see and a clear blue sky above.",
  "A zebra standing on a hilltop with rolling green fields and a clear blue sky stretching out below.",
  "A zebra standing near a wooden bridge over a babbling brook in a peaceful forest with a clear blue sky above.",
  "A zebra grazing in a field of poppies with vibrant red petals swaying in the breeze and a clear blue sky above.",
  "A zebra walking along a sandy path with wildflowers blooming on either side and a clear blue sky above.",
  "A zebra standing in front of a stone cottage with a thatched roof and a garden full of colorful flowers and a clear blue sky above.",
  "A zebra standing in a field of corn with tall stalks swaying in the breeze and a clear blue sky above.",
  "A zebra standing near a small wooden cabin in a snowy landscape with falling snow and a clear blue sky above.",
  "A zebra walking along a trail with autumn leaves crunching underfoot and trees ablaze with fall colors and a clear blue sky above.",
  "A zebra standing in a meadow with a windmill turning slowly in the distance and wildflowers in bloom and a clear blue sky above.",
  "A zebra standing at the edge of a serene lake during sunrise.",
  "A zebra grazing in a lush green meadow with wildflowers in full bloom.",
  "A zebra crossing a shallow river surrounded by dense forest.",
  "A zebra walking along a dirt path in a small village.",
  "A zebra drinking water from a wooden trough at a countryside farm.",
  "A zebra standing under a large acacia tree on a savanna.",
  "A zebra grazing near a group of wildebeests in a grassland.",
  "A zebra resting in the shade of a rocky outcrop on a hot day.",
  "A zebra standing on a hill with a panoramic view of the plains below.",
  "A zebra walking alongside a herd of elephants at a watering hole.",
  "A zebra standing on a sandy beach near the ocean waves.",
  "A zebra grazing near a small pond with lily pads and frogs.",
  "A zebra walking through a misty forest at dawn.",
  "A zebra standing in a field of tall, golden grass.",
  "A zebra crossing a narrow wooden bridge over a stream.",
  "A zebra grazing near a rustic wooden fence in a rural area.",
  "A zebra standing at the edge of a canyon, looking out over the vast landscape.",
  "A zebra walking along a rocky trail in the mountains.",
  "A zebra standing near a group of giraffes in an open plain.",
  "A zebra grazing beside a small, babbling brook.",
  "A zebra walking through a field of wildflowers in the spring.",
  "A zebra standing on a grassy knoll with a rainbow in the background.",
  "A zebra crossing a dry, cracked riverbed in a desert.",
  "A zebra grazing near a windmill on a farm.",
  "A zebra walking along a quiet country road.",
  "A zebra standing in the shade of a large baobab tree.",
  "A zebra grazing near a herd of antelope in a savanna.",
  "A zebra standing at the edge of a forest clearing.",
  "A zebra walking through a field of tall, swaying grass.",
  "A zebra standing near a small, secluded waterfall.",
  "A zebra grazing beside a wooden barn in a rural setting.",
  "A zebra crossing a muddy river during the rainy season.",
  "A zebra standing on a rocky beach with the ocean waves crashing nearby.",
  "A zebra grazing near a field of sunflowers in full bloom.",
  "A zebra walking through a foggy meadow at dawn.",
  "A zebra standing on a hilltop with storm clouds gathering overhead.",
  "A zebra grazing near a group of gazelles in a grassland.",
  "A zebra standing at the edge of a dense jungle.",
  "A zebra walking along a narrow mountain pass.",
  "A zebra standing near a group of flamingos at a lakeshore.",
  "A zebra grazing near a field of lavender flowers.",
  "A zebra walking through a snowy landscape in winter.",
  "A zebra standing on a rocky outcrop with the sun setting behind it.",
  "A zebra grazing near a herd of buffalo in a savanna.",
  "A zebra standing at the edge of a steep cliff.",
  "A zebra walking through a field of daisies in the summer.",
  "A zebra standing near a group of ostriches in an open plain.",
  "A zebra grazing near a small, tranquil pond.",
  "A zebra walking along a sandy riverbank.",
  "A zebra standing in the shade of a large palm tree.",
  "A zebra grazing near a field of red poppies.",
  "A zebra walking through a misty valley at dawn.",
  "A zebra standing on a grassy plain with a thunderstorm approaching.",
  "A zebra grazing near a herd of zebras in a grassland.",
  "A zebra standing at the edge of a forested area.",
  "A zebra walking along a rocky shoreline.",
  "A zebra standing near a group of zebras at a watering hole.",
  "A zebra grazing near a field of golden wheat.",
  "A zebra walking through a snowy forest in winter.",
  "A zebra standing on a hill with a view of a distant mountain range.",
  "A zebra grazing near a group of wildebeests in a savanna.",
  "A zebra standing at the edge of a dense thicket.",
  "A zebra walking along a narrow path through the tall grass.",
  "A zebra standing near a group of cranes in a marshy area.",
  "A zebra grazing near a field of daisies in the spring.",
  "A zebra walking through a foggy forest at dusk.",
  "A zebra standing on a rocky plateau with a clear blue sky overhead.",
  "A zebra grazing near a herd of zebras in an open plain.",
  "A zebra standing at the edge of a tranquil lake.",
  "A zebra walking along a sandy path through the dunes.",
  "A zebra standing in the shade of a large sycamore tree.",
  "A zebra grazing near a field of purple heather.",
  "A zebra walking through a misty meadow at dusk.",
  "A zebra standing on a grassy hill with a rainbow overhead.",
  "A zebra grazing near a group of ibex in a mountain meadow.",
  "A zebra standing at the edge of a rocky canyon.",
  "A zebra walking along a quiet country lane.",
  "A zebra standing near a group of wildebeests at a waterhole.",
  "A zebra grazing near a field of bluebells.",
  "A zebra walking through a snowy meadow in winter.",
  "A zebra standing on a ridge with a panoramic view of the landscape.",
  "A zebra grazing near a herd of zebras in a savanna.",
  "A zebra standing at the edge of a forest glade.",
  "A zebra walking along a riverbank at sunset.",
  "A zebra standing near a group of antelope in an open field.",
  "A zebra grazing near a field of clover.",
  "A zebra walking through a misty valley at dawn.",
  "A zebra standing on a rocky outcrop with the sun rising behind it.",
  "A zebra grazing near a herd of buffalo in a grassland.",
  "A zebra standing at the edge of a dense forest.",
  "A zebra walking along a narrow trail through the woods.",
  "A zebra standing near a group of flamingos at a lake.",
  "A bear fishing in a river during a sunny day, with water splashing around.",
  "A bear sitting under a tree in a forest, enjoying the shade and looking content.",
#   "A bear walking through a snowy landscape, leaving paw prints in the snow.",
#   "A bear climbing a tree to reach a beehive, with bees buzzing around.",
#   "A bear playing with its cubs in a meadow full of wildflowers.",
#   "A bear rummaging through a campsite, looking curious but not aggressive.",
#   "A bear swimming across a calm lake, with mountains in the background.",
#   "A bear standing on its hind legs to reach berries on a bush.",
#   "A bear resting in a cave, with a cozy nest of leaves and twigs.",
#   "A bear catching a salmon in a fast-flowing stream, water splashing everywhere.",
#   "A bear walking along a rocky shoreline, with the ocean waves crashing.",
#   "A bear looking at its reflection in a clear, tranquil pond.",
#   "A bear lying on its back in a grassy field, staring up at the clouds.",
#   "A bear searching for food in a dense forest, pushing aside fallen leaves.",
#   "A bear standing on a hilltop, overlooking a vast valley below.",
#   "A bear sitting on a log by a campfire, with stars twinkling above.",
#   "A bear sniffing a trail of ants leading to an anthill.",
#   "A bear investigating a picnic basket left unattended in a park.",
#   "A bear sitting in a meadow at sunrise, with dew-covered grass.",
#   "A bear scratching its back against a rough tree trunk.",
#   "A bear chasing butterflies in a field of tall grass.",
#   "A bear walking through an autumn forest, with leaves of red and gold.",
#   "A bear drinking from a mountain stream, surrounded by wildflowers.",
#   "A bear climbing over fallen logs in a dense woodland.",
#   "A bear standing on a large rock, surveying its surroundings.",
#   "A bear playing with a large ball in a grassy field.",
#   "A bear digging for roots in a clearing in the forest.",
#   "A bear sitting on a boulder, watching birds flying overhead.",
#   "A bear lying in a patch of sunlit grass, looking sleepy.",
#   "A bear investigating a hollow log in the woods.",
#   "A bear catching snowflakes on its tongue during a light snowfall.",
#   "A bear sitting by a waterfall, enjoying the mist on its fur.",
#   "A bear standing in a river, with water flowing around its legs.",
#   "A bear eating honey from a broken hive on the ground.",
#   "A bear watching fish swim in a clear, shallow stream.",
#   "A bear exploring a rocky mountain trail, with peaks in the distance.",
#   "A bear walking through a field of blooming flowers, bees buzzing nearby.",
#   "A bear sitting on a rock by a calm lake, with reflections of trees in the water.",
#   "A bear playing with a fallen branch in a forest clearing.",
#   "A bear resting on a bed of moss in a shady grove.",
#   "A bear walking along a dirt path in a sunlit forest.",
#   "A bear standing at the edge of a cliff, looking out over a forested valley.",
#   "A bear fishing in a small pond, with lily pads floating on the surface.",
#   "A bear investigating a tree stump covered in mushrooms.",
#   "A bear standing in the rain, with droplets glistening on its fur.",
#   "A bear walking through a meadow at dusk, with fireflies lighting up.",
#   "A bear climbing a rocky slope, with a view of a valley below.",
#   "A bear sitting on a fallen log, watching squirrels in the trees.",
#   "A bear walking along a stream, with dragonflies darting around.",
#   "A bear lying in a sunny patch of grass, with a gentle breeze blowing.",
#   "A bear exploring a forest after a recent rain, with everything glistening.",
#   "A bear sniffing wildflowers in a meadow at dawn.",
#   "A bear scratching its back on a rough boulder in a grassy field.",
#   "A bear playing with its cubs near a riverbank, with trees in the background.",
#   "A bear standing in a field of tall grass, watching birds fly overhead.",
#   "A bear sitting on a hilltop at sunset, with a colorful sky.",
#   "A bear walking through a forest in spring, with new leaves budding.",
#   "A bear investigating a pile of fallen leaves in a woodland clearing.",
#   "A bear drinking from a forest stream, with trees arching overhead.",
#   "A bear standing in a snowy clearing, with pine trees surrounding.",
#   "A bear playing with a large rock, pushing it around with its paws.",
#   "A bear resting under a tree on a hot summer day, panting slightly.",
#   "A bear walking through a field of sunflowers, with bees buzzing around.",
#   "A bear sitting on a log by a babbling brook, looking content.",
#   "A bear standing in a meadow at sunset, with long shadows.",
#   "A bear climbing a steep hillside, with rocks and grass.",
#   "A bear walking along a forest path, with ferns and moss.",
#   "A bear catching a fish in a shallow river, water splashing.",
#   "A bear sitting in a field of daisies, with butterflies fluttering.",
#   "A bear resting in the shade of a large tree, looking relaxed.",
#   "A bear investigating a fallen tree in a forest clearing.",
#   "A bear drinking from a lake at dawn, with mist rising.",
#   "A bear standing on a rock in the middle of a stream, fishing.",
#   "A bear playing with its cubs in a grassy meadow, with wildflowers.",
#   "A bear walking through a snowy forest, leaving tracks behind.",
#   "A bear sitting on a hillside, watching the sunset.",
#   "A bear scratching its back on a tree, with bark peeling off.",
#   "A bear standing in a river, surrounded by autumn foliage.",
#   "A bear exploring a campsite, with tents and gear scattered.",
#   "A bear lying in a patch of clover, looking sleepy.",
#   "A bear investigating a hollow tree in a dense forest.",
#   "A bear standing on a rock, looking out over a lake.",
#   "A bear walking through a meadow at sunrise, with dew sparkling.",
#   "A bear playing with a fallen branch in a forest.",
#   "A bear resting on a bed of pine needles in a grove.",
#   "A bear walking along a forest trail, with birds singing.",
#   "A bear standing at the edge of a lake, with reflections of mountains.",
#   "A bear fishing in a pond, with dragonflies hovering.",
#   "A bear investigating a tree stump in a woodland.",
#   "A bear standing in the rain, with droplets on its fur.",
#   "A bear walking through a meadow at dusk, with fireflies.",
#   "A bear climbing a rocky slope, with a view of a valley.",
#   "A bear sitting on a log, watching squirrels.",
#   "A bear walking along a stream, with dragonflies.",
#   "A bear lying in a sunny patch of grass, with a breeze.",
#   "A bear exploring a forest after rain, with everything glistening.",
#   "A bear sniffing wildflowers in a meadow at dawn.",
  "A sheep grazing on a lush green hillside as the sun sets behind the mountains.",
  "A shepherd leading a flock of sheep across a narrow stone bridge over a river.",
  "A sheep resting under the shade of a large oak tree in the middle of a meadow.",
  "A sheep standing on a rocky outcrop, looking over a vast, rolling landscape.",
  "A sheep drinking from a clear, shallow stream in a dense forest.",
  "A sheep being sheared in a rustic barn, with wool scattered on the floor.",
  "A sheep walking along a sandy beach, with waves gently lapping at the shore.",
  "A sheep standing in a field of wildflowers, surrounded by vibrant colors.",
  "A sheep exploring an abandoned farmhouse, with broken windows and overgrown weeds.",
  "A sheep being fed by a farmer in a small, enclosed pen on a farm.",
  "A sheep walking along a dirt path through a vineyard, with grapes hanging from the vines.",
  "A sheep playing with a group of lambs in a fenced pasture on a sunny day.",
  "A sheep standing on a cliff's edge, overlooking the ocean waves crashing below.",
  "A sheep being herded by a border collie across a wide, open field.",
  "A sheep lying in a bed of hay in a cozy barn, with sunlight streaming through the windows.",
  "A sheep grazing near a tranquil pond, with ducks swimming and reeds swaying.",
  "A sheep walking through a dense fog in a quiet, misty forest.",
  "A sheep being transported in the back of a pickup truck down a country road.",
  "A sheep standing in front of a rustic wooden cabin in the middle of a forest.",
  "A sheep lying on a grassy knoll, with a rainbow arcing across the sky.",
#   "A sheep standing on a farm's dirt road, with a tractor parked nearby.",
#   "A sheep grazing in a field of golden wheat under a bright blue sky.",
#   "A sheep walking through a pumpkin patch during the autumn harvest season.",
#   "A sheep resting near a campfire with a group of hikers in the background.",
#   "A sheep standing on a cobblestone street in a quaint village.",
#   "A sheep lying in a pile of freshly fallen autumn leaves.",
#   "A sheep grazing in a field next to a windmill turning slowly in the breeze.",
#   "A sheep exploring the ruins of an old stone castle, with ivy climbing the walls.",
#   "A sheep standing in the middle of a snowy field, with snowflakes falling around it.",
#   "A sheep grazing near a field of sunflowers under a clear, sunny sky.",
#   "A sheep standing next to a wooden fence with rolling hills in the background.",
#   "A sheep walking through a lavender field, with purple flowers stretching to the horizon.",
#   "A sheep being petted by a young child in a petting zoo.",
#   "A sheep standing in the middle of a cobblestone courtyard surrounded by ivy-covered buildings.",
#   "A sheep grazing near an ancient stone circle in a grassy field.",
#   "A sheep resting on a hilltop, with a panoramic view of the countryside below.",
#   "A sheep standing on the edge of a dock, looking out over a peaceful lake.",
#   "A sheep grazing in a field of tall, golden grass as a storm approaches in the distance.",
#   "A sheep walking through a market square with vendors selling fresh produce.",
#   "A sheep resting in a sunlit meadow with a butterfly perched on its back.",
#   "A sheep standing in the doorway of a rustic red barn, looking out at the fields.",
#   "A sheep grazing near a wooden bridge crossing a small creek in the forest.",
#   "A sheep walking along a narrow trail through a dense bamboo forest.",
#   "A sheep standing next to a scarecrow in a cornfield.",
#   "A sheep grazing on the slope of a terraced hillside with rice paddies.",
#   "A sheep resting in a flower-filled meadow with bees buzzing around.",
#   "A sheep standing in a field of blooming poppies under a clear sky.",
#   "A sheep walking along a rocky shoreline with seagulls flying overhead.",
#   "A sheep standing in the middle of a narrow cobblestone alley in an old town.",
#   "A sheep grazing near a row of grapevines in a sunny vineyard.",
#   "A sheep lying on a bed of moss in a dense, ancient forest.",
#   "A sheep standing in the middle of a winding dirt road through the countryside.",
#   "A sheep grazing near a crystal-clear lake surrounded by mountains.",
#   "A sheep walking through a field of goldenrod flowers in late summer.",
#   "A sheep standing on a pier with fishing boats moored in the harbor.",
#   "A sheep resting in a shaded garden with colorful flowers and butterflies.",
#   "A sheep grazing near an old stone bridge over a gently flowing river.",
#   "A sheep standing in a field of bluebells under a canopy of trees.",
#   "A sheep walking along a gravel path through an apple orchard in bloom.",
#   "A sheep standing in a clearing in the forest, with sunlight filtering through the trees.",
#   "A sheep grazing near a small waterfall in a forest glade.",
#   "A sheep resting on a grassy hillside overlooking a small village.",
#   "A sheep walking through a sunflower maze on a bright summer day.",
#   "A sheep standing in the doorway of an old stone cottage.",
#   "A sheep grazing in a field of clover with bees buzzing around.",
#   "A sheep standing on a wooden deck overlooking a serene lake.",
#   "A sheep walking along a path through a field of daisies.",
#   "A sheep resting in the shadow of a large, ancient tree.",
#   "A sheep standing in a lush, green pasture with rolling hills in the background.",
#   "A sheep grazing near a rustic wooden fence with wildflowers in bloom.",
#   "A sheep standing in the middle of a narrow dirt road lined with trees.",
#   "A sheep walking through a grassy meadow with birds singing in the trees.",
#   "A sheep resting in a field of tall, waving grass under a clear blue sky.",
#   "A sheep standing next to a stone wall with ivy climbing over it.",
#   "A sheep grazing near a field of blooming cherry blossoms.",
#   "A sheep lying on a hillside, watching the sun set over the horizon.",
#   "A sheep standing in the middle of a wheat field with the golden crop swaying in the breeze.",
#   "A sheep walking along a forest path covered in fallen autumn leaves.",
#   "A sheep standing on a riverbank with the water gently flowing past.",
#   "A sheep grazing near a meadow filled with colorful wildflowers.",
#   "A sheep standing on a ridge overlooking a deep valley.",
#   "A sheep walking through a field of lavender in full bloom.",
#   "A sheep resting in a sunny spot in a wooded glade.",
#   "A sheep standing next to a haystack in a rural farm setting.",
#   "A sheep grazing near a bubbling brook in a tranquil forest.",
#   "A sheep walking through a field of tall sunflowers.",
#   "A sheep standing in the shade of a large, spreading tree.",
#   "A sheep grazing near an old wooden barn with a weathered roof.",
#   "A sheep lying on a bed of fallen leaves in a quiet forest.",
#   "A sheep standing in a meadow with butterflies fluttering around.",
#   "A sheep walking along a narrow dirt path through a grassy field.",
#   "A sheep standing in the doorway of a weathered farmhouse.",
#   "A sheep grazing in a field of tall, waving grass under a bright sky.",
#   "A sheep standing next to a wooden signpost in the countryside.",
#   "A sheep resting in a shaded area of a sunny meadow.",
#   "A sheep walking through a field of blooming wildflowers.",
  "A cat sitting on a windowsill, watching the rain outside a cozy, dimly-lit room.",
  "A cat lounging on a sunlit patio, surrounded by potted plants and blooming flowers.",
  "A cat curled up on a plush armchair next to a roaring fireplace in a rustic cabin.",
  "A cat perched on a bookshelf, staring curiously at a bird outside the window.",
  "A cat playing with a ball of yarn on a hardwood floor in a bright living room.",
  "A cat napping in a hammock strung between two trees in a peaceful backyard.",
  "A cat exploring a cluttered artist’s studio, with paintbrushes and canvases scattered around.",
  "A cat drinking water from a small fountain in a tranquil garden.",
  "A cat sitting on a picnic blanket, surrounded by a spread of food and drinks in a park.",
  "A cat chasing a butterfly in a sun-drenched meadow with tall grass and wildflowers.",
  "A cat lounging on a beach towel, with the ocean waves gently crashing nearby.",
  "A cat sitting on a wooden dock, looking out over a calm lake at sunset.",
  "A cat perched on the edge of a rooftop, watching the bustling city below.",
  "A cat hiding under a table at a bustling outdoor café.",
  "A cat perched on a fence, watching children play in a neighborhood yard.",
  "A cat curled up in a basket filled with cozy blankets near a reading nook.",
  "A cat playing with fallen leaves in a vibrant autumn forest.",
  "A cat basking in the sun on a windowsill filled with potted herbs.",
  "A cat exploring a cozy attic filled with old books and antiques.",
  "A cat sitting on the edge of a bathtub, curiously watching the water.",
#   "A cat napping in a sunny spot on a tiled kitchen floor.",
#   "A cat perched on a bar stool at a lively home party.",
#   "A cat exploring a cluttered garage with tools and boxes scattered around.",
#   "A cat resting on a tree branch, high above a quiet suburban street.",
#   "A cat playing with a ribbon in a cozy bedroom filled with soft pillows and blankets.",
#   "A cat perched on a stone wall, watching a group of people having a barbecue.",
#   "A cat sitting on a windowsill in a bustling library, watching people come and go.",
#   "A cat lounging on a bench in a peaceful botanical garden.",
#   "A cat exploring a children's playroom filled with toys and colorful decorations.",
#   "A cat napping in a ray of sunlight on a carpeted floor in a quiet study.",
#   "A cat perched on the armrest of a couch, watching a movie on TV.",
#   "A cat sitting on a porch swing, gently swaying in the breeze.",
#   "A cat exploring a quaint bookstore, weaving between the shelves.",
#   "A cat napping on a quilt in a sunlit attic room.",
#   "A cat playing with a feather toy in a bright, modern living room.",
#   "A cat perched on a garden wall, watching a family having a picnic.",
#   "A cat curled up on a knitted blanket next to a steaming cup of tea.",
#   "A cat sitting on a windowsill in a busy kitchen, watching someone cook.",
#   "A cat lounging in a sunbeam on a wooden deck overlooking a lush garden.",
#   "A cat perched on a balcony railing, overlooking a bustling market street.",
#   "A cat napping on a soft cushion in a quiet reading corner.",
#   "A cat playing with a fallen pine cone in a dense forest.",
#   "A cat sitting on a windowsill in a cozy cabin, looking out at the snow-covered landscape.",
#   "A cat lounging on a bench in a quiet cemetery, surrounded by blooming flowers.",
#   "A cat exploring a quaint, cluttered workshop filled with tools and projects.",
#   "A cat napping on a plush rug in front of a crackling fireplace.",
#   "A cat perched on a windowsill in a high-rise apartment, watching the city lights.",
#   "A cat playing with a paper bag in a bustling kitchen.",
#   "A cat lounging on a sunlit rock by a babbling brook.",
#   "A cat sitting on a bench in a quiet park, watching ducks in a pond.",
#   "A cat perched on a tree stump in a dense forest, watching squirrels play.",
#   "A cat napping on a soft bed in a bright, airy bedroom.",
#   "A cat playing with a ball in a sunny backyard.",
#   "A cat lounging on a windowsill in a busy office, watching people work.",
#   "A cat perched on a fence in a bustling neighborhood, watching people walk by.",
#   "A cat napping on a pile of fresh laundry in a sunlit laundry room.",
#   "A cat playing with a leaf in a quiet courtyard.",
#   "A cat lounging on a soft pillow in a cozy, dimly-lit room.",
#   "A cat perched on a windowsill in a lively café, watching people outside.",
#   "A cat napping in a sunbeam on a wooden floor in a bright living room.",
#   "A cat exploring a cluttered attic filled with old toys and trinkets.",
#   "A cat lounging on a windowsill in a quiet library, watching the rain.",
#   "A cat perched on a bench in a bustling park, watching people jog by.",
#   "A cat napping on a soft blanket in a cozy nursery.",
#   "A cat playing with a feather in a sunlit garden.",
#   "A cat lounging on a windowsill in a busy kitchen, watching someone bake.",
#   "A cat perched on a fence in a quiet neighborhood, watching children play.",
#   "A cat napping in a sunny spot on a tiled patio.",
#   "A cat exploring a quaint, cluttered studio filled with art supplies.",
#   "A cat lounging on a soft cushion in a quiet reading nook.",
#   "A cat perched on a windowsill in a bustling café, watching people come and go.",
#   "A cat napping on a soft rug in front of a roaring fireplace.",
#   "A cat playing with a ribbon in a bright, airy bedroom.",
#   "A cat lounging on a sunlit deck overlooking a lush garden.",
#   "A cat perched on a balcony railing, watching the bustling street below.",
#   "A cat napping on a plush rug in a sunlit living room.",
#   "A cat playing with a ball in a bright, modern kitchen.",
#   "A cat lounging on a windowsill in a quiet office, watching people work.",
#   "A cat perched on a fence in a bustling neighborhood, watching cars drive by.",
#   "A cat napping on a soft bed in a sunlit bedroom.",
#   "A cat playing with a leaf in a vibrant autumn park.",
#   "A cat lounging on a bench in a peaceful garden.",
#   "A cat perched on a tree stump in a sunlit forest.",
#   "A cat napping in a cozy hammock strung between two trees.",
#   "A cat exploring a quaint, cluttered basement filled with old furniture.",
#   "A cat lounging on a windowsill in a quiet study, watching the rain.",
#   "A cat napping on a soft pillow in a sunlit nursery.",
#   "A cat playing with a feather toy in a bright, airy living room.",
#   "A cat lounging on a sunlit patio, surrounded by blooming flowers.",
#   "A cat perched on a bench in a bustling park, watching people walk by.",
#   "A cat napping on a soft blanket in a cozy bedroom.",
#   "A cat playing with a ball of yarn in a sunny craft room.",
#   "A cat lounging on a windowsill in a busy café, watching the rain.",
#   "A cat perched on a fence in a quiet neighborhood, watching the sunset.",
#   "A cat napping in a sunny spot on a tiled kitchen floor.",
#   "A cat exploring a cluttered garage filled with old tools and boxes.",
#   "A cat lounging on a bench in a peaceful cemetery, surrounded by blooming flowers.",
#   "A cat perched on a windowsill in a high-rise apartment, watching the city lights.",
  "a dog playing fetch in a sunny park with children",
  "a dog napping on a cozy blanket by a fireplace",
  "a dog running on the beach with waves crashing",
  "a dog riding in the backseat of a car with the window down",
  "a dog walking through a snowy forest",
  "a dog sitting at a table with a family during dinner",
  "a dog exploring a flower-filled meadow",
  "a dog sitting obediently next to a street musician",
  "a dog chasing a butterfly in a garden",
  "a dog resting under a tree in a bustling city park",
  "a dog digging a hole in a sandy backyard",
  "a dog splashing in a shallow stream",
  "a dog sitting on a porch watching the rain",
  "a dog accompanying a hiker on a mountain trail",
  "a dog waiting patiently outside a bakery",
  "a dog sniffing around a farmer's market",
  "a dog lying on a picnic blanket in a crowded park",
  "a dog playing tug-of-war with its owner in a backyard",
  "a dog sitting in a canoe on a calm lake",
  "a dog running alongside a cyclist on a country road",
#   "a dog jumping through a sprinkler on a hot day",
#   "a dog lounging in a sunbeam in a living room",
#   "a dog waiting for its owner outside a coffee shop",
#   "a dog playing in a pile of autumn leaves",
#   "a dog lying next to a tent in a campsite",
#   "a dog standing on a dock looking at the water",
#   "a dog riding on a skateboard in a skatepark",
#   "a dog lying in the grass watching people fly kites",
#   "a dog sitting next to a fisherman by a lake",
#   "a dog waiting at the bottom of a playground slide",
#   "a dog running through a field of tall grass",
#   "a dog sitting on a bench next to a couple",
#   "a dog watching kids play soccer in a park",
#   "a dog sitting in front of a fireplace with stockings hanging",
#   "a dog lying on a boat deck in the sun",
#   "a dog playing with a frisbee in a suburban yard",
#   "a dog standing in a river looking at fish",
#   "a dog resting on a porch swing with an elderly person",
#   "a dog running in circles on a sandy beach",
#   "a dog sitting in a kayak on a river",
#   "a dog jumping into a pile of snow",
#   "a dog lounging in a hammock with its owner",
#   "a dog watching fireworks from a hilltop",
#   "a dog walking along a pier at sunset",
#   "a dog chasing its tail in a grassy park",
#   "a dog standing in front of a snowman",
#   "a dog resting on a window seat looking outside",
#   "a dog playing with other dogs in a dog park",
#   "a dog sitting next to a bonfire on the beach",
#   "a dog looking up at a tree where a squirrel sits",
#   "a dog exploring a pumpkin patch",
#   "a dog sitting in a bike basket as its owner rides",
#   "a dog lying on a deck surrounded by potted plants",
#   "a dog splashing in a kiddie pool in a backyard",
#   "a dog walking through a field of sunflowers",
#   "a dog watching birds from a patio",
#   "a dog running on a muddy trail in the rain",
#   "a dog lying on a rug next to a Christmas tree",
#   "a dog sitting next to a food truck with people around",
#   "a dog playing in the grass under a rainbow",
#   "a dog waiting by a tent at a music festival",
#   "a dog sitting in front of a barbecue grill",
#   "a dog running alongside a jogger in a park",
#   "a dog sitting on a blanket at a drive-in movie",
#   "a dog playing with children in a backyard pool",
#   "a dog lying on a dock watching boats go by",
#   "a dog standing on a bridge overlooking a river",
#   "a dog resting on a picnic table in the woods",
#   "a dog watching a horse grazing in a field",
#   "a dog sitting in a living room full of moving boxes",
#   "a dog playing in the surf at a beach",
#   "a dog sitting next to a gardener in a vegetable patch",
#   "a dog lying on a porch with a lemonade stand nearby",
#   "a dog standing on a hilltop with a panoramic view",
#   "a dog running through a corn maze",
#   "a dog lying on a carpet next to a wood stove",
#   "a dog walking through a quiet neighborhood at dusk",
#   "a dog sitting in a field of wildflowers",
#   "a dog resting under a patio table at a cafe",
#   "a dog playing in a backyard with a sandbox",
#   "a dog standing in a snow-covered street",
#   "a dog walking along a forest path with fallen leaves",
#   "a dog sitting in a shopping cart at an outdoor market",
#   "a dog lying in front of a fan on a hot day",
#   "a dog jumping into a pond to fetch a stick",
#   "a dog resting on a bench at a bus stop",
#   "a dog running through an open field at dawn",
#   "a dog lying on a sun-drenched dock by a lake",
#   "a dog sitting by a fireplace with a roaring fire",
#   "a dog watching children playing on a swing set",
#   "a dog walking on a leash through a bustling farmers market",
#   "a dog chasing seagulls on a sandy shore",
#   "a dog lying on a blanket at a summer picnic",
#   "a dog sitting in a canoe with a fishing pole nearby",
#   "a dog playing with a ball in a front yard",
#   "a dog standing in front of a colorful mural",
  "A horse galloping across a vast, golden wheat field at sunset.",
  "A horse standing calmly by a crystal-clear river, drinking water.",
  "A horse being ridden by a young girl through a dense, misty forest.",
  "A horse grazing peacefully in a lush green meadow dotted with wildflowers.",
  "A horse trotting along a sandy beach as the waves crash against the shore.",
  "A horse pulling a wooden cart filled with hay down a country road.",
  "A horse jumping over a wooden fence in a paddock during a riding lesson.",
  "A horse being groomed by a stable hand in a rustic barn.",
  "A horse running alongside a flock of sheep in a rolling pasture.",
  "A horse standing beside a weathered old barn under a bright blue sky.",
  "A horse being led by a farmer through a field of tall, swaying grass.",
  "A horse lying down in a sunny pasture, taking a nap.",
  "A horse being saddled up by a cowboy for a trail ride.",
  "A horse playing in a muddy paddock after a rainstorm.",
  "A horse standing in a snowy field with steam rising from its nostrils.",
  "A horse being fed apples by a young boy near an old stone wall.",
  "A horse trotting through a village market with a rider.",
  "A horse crossing a shallow stream in a wooded area.",
  "A horse resting in the shade of a large oak tree.",
  "A horse walking along a dirt path lined with colorful autumn leaves.",
#   "A horse being shod by a blacksmith in a small workshop.",
#   "A horse carrying supplies up a rocky mountain trail.",
#   "A horse standing in front of a picturesque farmhouse with a white picket fence.",
#   "A horse galloping through a field of sunflowers.",
#   "A horse grazing near a quiet pond with lily pads.",
#   "A horse being ridden by a knight in armor in a medieval reenactment.",
#   "A horse standing in a stable with a bale of hay.",
#   "A horse trotting through a lavender field at dawn.",
#   "A horse being washed by its owner with a hose.",
#   "A horse pulling a plow through a vegetable garden.",
#   "A horse standing in a field of tall, golden grass under a stormy sky.",
#   "A horse being led onto a trailer for transport.",
#   "A horse running freely in a large, open field.",
#   "A horse standing at the edge of a cliff, looking out over a valley.",
#   "A horse being examined by a veterinarian in a clean, modern clinic.",
#   "A horse trotting through a quiet forest, leaves crunching underfoot.",
#   "A horse standing by a wooden fence, its mane blowing in the wind.",
#   "A horse pulling a carriage through a city park.",
#   "A horse being trained in a round pen by a handler.",
#   "A horse standing in a barn stall, its coat shiny and clean.",
#   "A horse galloping through a rain-soaked field.",
#   "A horse standing in the middle of a dirt road with a distant farmhouse.",
#   "A horse grazing near a stone well in a rural village.",
#   "A horse standing at the edge of a lake as the sun sets.",
#   "A horse being ridden by a child in a riding competition.",
#   "A horse running alongside a train in a grassy field.",
#   "A horse standing in a field with wind turbines in the background.",
#   "A horse being led through a market square crowded with people.",
#   "A horse standing on a hillside with a flock of birds flying overhead.",
#   "A horse being ridden through a field of tall, swaying corn.",
#   "A horse standing in a foggy meadow at dawn.",
#   "A horse pulling a sleigh through a snowy forest.",
#   "A horse being bathed in a sunlit stable yard.",
#   "A horse standing by a wooden fence covered in ivy.",
#   "A horse grazing in a field of purple heather.",
#   "A horse being petted by a group of children at a petting zoo.",
#   "A horse running along a dirt road with a mountain range in the distance.",
#   "A horse standing by a rustic wooden bridge over a stream.",
#   "A horse being led through a field of tall, golden sunflowers.",
#   "A horse standing in a field with rolling hills in the background.",
#   "A horse being ridden along a forest path dappled with sunlight.",
#   "A horse standing by a stone wall covered in moss.",
#   "A horse galloping through a field of blooming poppies.",
#   "A horse being led into a stable by a rider in full riding gear.",
#   "A horse grazing near a wooden cabin in the woods.",
#   "A horse standing by a windmill on a farm.",
#   "A horse running through a field of tall, green grass under a clear blue sky.",
#   "A horse being ridden through a picturesque village with cobblestone streets.",
#   "A horse standing by a serene lake surrounded by mountains.",
#   "A horse grazing in a meadow filled with daisies.",
#   "A horse being led by a rider through a bustling fairground.",
#   "A horse standing in a field with a rainbow in the background.",
#   "A horse galloping through a field of goldenrod.",
#   "A horse being groomed by a rider with a soft brush.",
#   "A horse standing in a field with a herd of cows.",
#   "A horse being led through a field of tall, golden wheat.",
#   "A horse grazing near a wooden bench in a park.",
#   "A horse standing by a stone fountain in a garden.",
#   "A horse running through a field of blooming lavender.",
#   "A horse being led through a forest with tall pine trees.",
#   "A horse standing by a wooden fence in a rural farmyard.",
#   "A horse being ridden along a coastal cliff path.",
#   "A horse standing by a field of red poppies in full bloom.",
#   "A horse grazing in a field with a distant castle on a hill.",
#   "A horse being led through a quiet village with thatched cottages.",
#   "A horse standing by a wooden barn with a thatched roof.",
#   "A horse galloping through a field of tall, swaying grass at dusk.",
#   "A horse being led by a rider through a bustling market square.",
#   "A horse standing by a tranquil pond with ducks swimming.",
#   "A horse grazing near a stone wall in a rural pasture.",
#   "A horse being ridden along a dirt path through a meadow.",
#   "A horse standing in a field with a distant windmill.",
#   "A horse being led through a field of blooming wildflowers.",
#   "A horse standing by a rustic wooden barn under a bright blue sky.",
#   "A horse running through a field of blooming sunflowers.",
#   "A horse being groomed by a rider with a curry comb.",
#   "A horse standing in a field with a herd of sheep.",
  "A cow grazing peacefully in a lush green meadow at sunrise.",
  "A cow standing in the middle of a dirt road, with a farm in the background.",
  "A cow lying down in the shade of a large oak tree on a hot summer day.",
  "A cow being milked by a farmer in a rustic barn.",
  "A cow wading in a shallow stream on a sunny afternoon.",
  "A cow standing next to a wooden fence, with rolling hills in the distance.",
  "A cow munching on hay in a spacious, open barn.",
  "A cow walking alongside a flock of sheep in a grassy field.",
  "A cow drinking water from a pond surrounded by wildflowers.",
  "A cow standing in a muddy pasture after a heavy rain.",
  "A cow resting in a field of blooming wildflowers.",
  "A cow wandering through a cornfield at sunset.",
  "A cow standing in the middle of a rural dirt road, blocking a tractor.",
  "A cow grazing near an old, weathered barn with peeling paint.",
  "A cow standing beside a scarecrow in a vegetable garden.",
  "A cow staring curiously at a passing butterfly in a meadow.",
  "A cow standing next to a red barn with a white fence.",
  "A cow grazing near a small, bubbling creek.",
  "A cow standing in a field with a rainbow in the background.",
  "A cow lying in a field of tall grass with birds perched on its back.",
#   "A cow standing in a snowy pasture, its breath visible in the cold air.",
#   "A cow munching on fresh grass in a field dotted with dandelions.",
#   "A cow standing under a shady tree next to a wooden swing.",
#   "A cow standing near a windmill in a wide-open field.",
#   "A cow grazing in a field as a train passes by in the distance.",
#   "A cow standing in a pasture under a starry night sky.",
#   "A cow being brushed by a young child in front of a barn.",
#   "A cow standing next to a chicken coop with hens pecking at the ground.",
#   "A cow lying down in a field while a dog naps beside it.",
#   "A cow grazing near a field of sunflowers on a clear day.",
#   "A cow standing in a field with a stormy sky overhead.",
#   "A cow walking through a pumpkin patch in the fall.",
#   "A cow standing in a pasture with a wooden signpost in the background.",
#   "A cow grazing next to a white picket fence covered in ivy.",
#   "A cow standing in a field with a mountain range in the distance.",
#   "A cow drinking from a trough in a farmyard.",
#   "A cow standing in a field as a group of children play nearby.",
#   "A cow grazing near an old stone wall in the countryside.",
#   "A cow lying in a field of clover on a breezy day.",
#   "A cow standing in a field of tall, golden wheat.",
#   "A cow grazing near a field of blooming lavender.",
#   "A cow standing in a pasture with a bright red tractor in the background.",
#   "A cow resting under the shade of a large tree with a farmhouse visible in the distance.",
#   "A cow grazing near a pond with ducks swimming on the water.",
#   "A cow standing next to a barn decorated with holiday lights.",
#   "A cow lying in a field with a wind turbine spinning in the background.",
#   "A cow grazing near a field of bright orange poppies.",
#   "A cow standing next to a rustic wooden bridge over a small stream.",
#   "A cow wandering through a field of tall, swaying grass on a windy day.",
#   "A cow grazing near a field of ripening tomatoes.",
#   "A cow standing in a field as hot air balloons rise in the sky.",
#   "A cow drinking from a stream that flows through a forested area.",
#   "A cow standing in a field with a line of trees in the background.",
#   "A cow grazing near a field of vibrant yellow daffodils.",
#   "A cow lying in the shade of an apple tree in an orchard.",
#   "A cow standing next to a haystack in a farmyard.",
#   "A cow grazing near a field of purple heather.",
#   "A cow standing in a field with a flock of geese flying overhead.",
#   "A cow drinking from a river that winds through a valley.",
#   "A cow standing near a wooden fence with a field of blooming tulips.",
#   "A cow lying in a field of daisies on a sunny day.",
#   "A cow standing in a pasture as a herd of deer graze nearby.",
#   "A cow grazing near a field of bright red poppies.",
#   "A cow standing in a field with a weathered barn in the background.",
#   "A cow lying in a field under a clear blue sky.",
#   "A cow grazing near a field of colorful zinnias.",
#   "A cow standing next to a rusty old tractor in a field.",
#   "A cow resting in a meadow with a stone cottage in the background.",
#   "A cow grazing near a field of blooming lupines.",
#   "A cow standing in a field with a flock of birds flying above.",
#   "A cow lying in a field of tall, green grass.",
#   "A cow grazing near a field of marigolds.",
#   "A cow standing in a pasture with a clear blue sky overhead.",
#   "A cow drinking from a stream in a meadow.",
#   "A cow standing near a wooden fence with a view of the countryside.",
#   "A cow lying in a field of pink clover.",
#   "A cow grazing near a field of yellow sunflowers.",
#   "A cow standing in a field with a distant farmhouse.",
#   "A cow resting under a tree with a red barn in the distance.",
#   "A cow grazing near a field of white daisies.",
#   "A cow standing next to a wooden cart in a field.",
#   "A cow lying in a field of wildflowers.",
#   "A cow grazing near a field of goldenrod.",
#   "A cow standing in a pasture with a view of rolling hills.",
#   "A cow drinking from a pond in a grassy meadow.",
#   "A cow standing near a stone wall with a field of blooming roses.",
#   "A cow lying in a field under a cloudy sky.",
#   "A cow grazing near a field of bright blue irises.",
#   "A cow standing in a field with a windmill in the background.",
#   "A cow resting in a meadow with a view of a distant mountain range.",
#   "A cow grazing near a field of pink cosmos flowers.",
#   "A cow standing in a pasture with a view of a forest.",
#   "A cow lying in a field of tall wildflowers.",
#   "A cow grazing near a field of purple coneflowers.",
#   "A cow standing in a field with a wooden fence in the background.",
#   "A cow lying in a meadow with a view of a distant lake.",
#   "A cow grazing near a field of golden sunflowers.",
  "An elephant walking through a busy marketplace, with stalls of fruits and vegetables around.",
  "An elephant bathing in a river while people wash clothes nearby.",
  "An elephant standing in a grassy field with a herd of cattle grazing.",
  "An elephant being painted for a cultural festival in a small village.",
  "An elephant crossing a dirt road in a rural area with farmers working in the fields.",
  "An elephant lifting logs with its trunk in a dense forest.",
  "An elephant eating leaves from a tall tree with birds perched on branches.",
  "An elephant playing with a group of children in a park.",
  "An elephant walking along a beach with tourists taking photos.",
  "An elephant carrying tourists on its back through a jungle trail.",
  "An elephant standing by a waterhole surrounded by other wildlife.",
  "An elephant walking through a residential neighborhood with houses in the background.",
  "An elephant walking down a city street with skyscrapers and traffic.",
  "An elephant participating in a religious procession with people in traditional attire.",
  "An elephant helping to pull a stuck vehicle out of the mud in a remote area.",
  "An elephant grazing in a meadow with mountains in the distance.",
  "An elephant spraying water from its trunk on a hot day in a savanna.",
  "An elephant standing near a campfire with campers setting up tents.",
  "An elephant walking through a zoo enclosure with visitors watching.",
  "An elephant playing with a soccer ball in an open field.",
#   "An elephant standing by a school playground with children running around.",
#   "An elephant helping to carry goods in a market with vendors selling various items.",
#   "An elephant walking along a riverbank with fishermen casting nets.",
#   "An elephant pulling a plow in a rice paddy with a farmer guiding it.",
#   "An elephant lying in the shade of a large tree with a family having a picnic nearby.",
#   "An elephant walking through a desert landscape with sand dunes.",
#   "An elephant standing near a train station with passengers boarding a train.",
#   "An elephant walking through a tea plantation with workers picking leaves.",
#   "An elephant standing in the rain with puddles forming around its feet.",
#   "An elephant helping to move heavy stones at a construction site.",
#   "An elephant standing on a grassy hill with a windmill in the background.",
#   "An elephant walking through a carnival with colorful rides and games.",
#   "An elephant standing by a waterfall with mist rising from the water.",
#   "An elephant being fed by a zookeeper in an enclosure.",
#   "An elephant walking along a dirt path with a herd of sheep following.",
#   "An elephant standing near a bus stop with people waiting for a bus.",
#   "An elephant playing in the snow with children building snowmen.",
#   "An elephant walking through a sunflower field with bees buzzing around.",
#   "An elephant standing near a lighthouse with waves crashing against the shore.",
#   "An elephant walking through a vineyard with grapevines laden with fruit.",
#   "An elephant standing in a botanical garden with exotic plants and flowers.",
#   "An elephant walking through a bustling fish market with vendors shouting prices.",
#   "An elephant standing near a bonfire on a beach with people roasting marshmallows.",
#   "An elephant walking through a bamboo forest with sunlight filtering through the leaves.",
#   "An elephant standing in a field of lavender with butterflies flying around.",
#   "An elephant walking through a pumpkin patch with families picking pumpkins.",
#   "An elephant standing near a bridge with a river flowing underneath.",
#   "An elephant walking through a winter market with stalls selling holiday items.",
#   "An elephant standing in a wheat field with a combine harvester in the background.",
#   "An elephant walking through a city park with people jogging and walking dogs.",
#   "An elephant standing near a ferris wheel at a fairground with bright lights.",
#   "An elephant walking through a crowded marketplace with colorful fabrics on display.",
#   "An elephant standing near a fountain in a town square with children playing.",
#   "An elephant walking along a hiking trail with people carrying backpacks.",
#   "An elephant standing near a fire station with firefighters washing a truck.",
#   "An elephant walking through an art festival with paintings and sculptures on display.",
#   "An elephant standing near a library with students reading books on benches.",
#   "An elephant walking through an amusement park with roller coasters and attractions.",
#   "An elephant standing near a dock with boats and fishermen preparing to sail.",
#   "An elephant walking through an apple orchard with workers picking apples.",
#   "An elephant standing near a clock tower in a busy town center.",
#   "An elephant walking through a music festival with bands performing on stage.",
#   "An elephant standing near a greenhouse with gardeners tending to plants.",
#   "An elephant walking through a flower market with bouquets of flowers for sale.",
#   "An elephant standing near a barn with farm animals and hay bales.",
#   "An elephant walking through a shopping district with stores and shoppers.",
#   "An elephant standing near a water fountain in a park with ducks swimming.",
#   "An elephant walking through a field of wildflowers with bees and butterflies.",
#   "An elephant standing near a sports stadium with fans entering for a game.",
#   "An elephant walking through a farmers' market with fresh produce on display.",
#   "An elephant standing near a monument with tourists taking pictures.",
#   "An elephant walking through a vineyard with workers harvesting grapes.",
#   "An elephant standing near a river with kayakers paddling downstream.",
#   "An elephant walking through a parade with colorful floats and performers.",
#   "An elephant standing near a bus depot with buses arriving and departing.",
#   "An elephant walking through a fairground with rides and games.",
#   "An elephant standing near a beach with surfers riding waves.",
#   "An elephant walking through a snow-covered village with people shoveling snow.",
#   "An elephant standing near a construction site with cranes and workers.",
#   "An elephant walking through a rural village with thatched-roof houses.",
#   "An elephant standing near a train track with a train approaching.",
#   "An elephant walking through a wildlife sanctuary with rangers observing.",
#   "An elephant standing near a circus tent with performers practicing.",
#   "An elephant walking through a garden with blooming flowers and trees.",
#   "An elephant standing near a farm with fields of crops and a barn.",
#   "An elephant walking through a crowded street market with vendors selling goods.",
#   "An elephant standing near a park bench with an elderly couple sitting.",
#   "An elephant walking through a forest with hikers on a trail.",
#   "An elephant standing near a playground with children on swings and slides.",
#   "An elephant walking through a vineyard with a winery in the background.",
#   "An elephant standing near a water park with slides and pools.",
#   "An elephant walking through a city square with a statue and fountain.",
#   "An elephant standing near a camping site with tents and a campfire.",
#   "An elephant walking through a field of sunflowers with a clear blue sky.",
#   "An elephant standing near a marina with boats docked.",
#   "An elephant walking through a zoo with families and children watching.",
#   "An elephant standing near a flower garden with various colorful blooms.",
  "A bird perched on a rustic wooden fence during a foggy morning in the countryside.",
  "A bird flying over a calm lake as the sun sets in the distance.",
  "A bird feeding its chicks in a nest high up in a pine tree.",
  "A bird taking a bath in a shallow garden birdbath.",
  "A bird singing from the top of a blooming cherry blossom tree.",
  "A bird swooping down to catch a fish from a clear mountain stream.",
  "A bird resting on the window sill of a quaint cottage.",
  "A bird gliding through the air over a sprawling meadow filled with wildflowers.",
  "A bird perched on the shoulder of a scarecrow in a cornfield.",
  "A bird building a nest in the eaves of a rustic barn.",
  "A bird drinking from a small puddle after a light rain shower.",
  "A bird perched on a signpost along a quiet country road.",
  "A bird flying alongside a sailboat in the open sea.",
  "A bird perched on a park bench in a busy urban park.",
  "A bird hopping along the sandy beach searching for food.",
  "A bird landing on a sunflower in a vast field of sunflowers.",
  "A bird resting on the handle of a well-used gardening tool.",
  "A bird pecking at crumbs on the outdoor table of a cafe.",
  "A bird chirping from the branches of a tree in a bustling city square.",
  "A bird sitting on a telephone wire as the sun rises behind it.",
#   "A bird catching insects in mid-air in a suburban backyard.",
#   "A bird perched on the hood of a parked car in a quiet neighborhood.",
#   "A bird flying through a forest during autumn with colorful leaves falling.",
#   "A bird standing on a rock in the middle of a flowing river.",
#   "A bird perched on the edge of a birdhouse in a well-tended garden.",
#   "A bird pecking at seeds on a snowy ground in the middle of winter.",
#   "A bird perched on the armrest of a wooden rocking chair on a porch.",
#   "A bird gliding over a field of tall grass swaying in the wind.",
#   "A bird landing on the edge of a rooftop during a light drizzle.",
#   "A bird sitting on the branch of a tree covered in frost.",
#   "A bird perched on a signpost at the entrance of a nature reserve.",
#   "A bird perched on the handlebar of a parked bicycle.",
#   "A bird flying through a narrow alleyway between tall buildings.",
#   "A bird sitting on the railing of a wooden footbridge over a stream.",
#   "A bird pecking at grains on a farmyard.",
#   "A bird perched on the top of a lighthouse overlooking the ocean.",
#   "A bird resting on the back of a grazing sheep in a pasture.",
#   "A bird singing in the early morning light on the windowsill of a high-rise apartment.",
#   "A bird perched on the stone ledge of an ancient ruin.",
#   "A bird flying over a field of freshly harvested crops.",
#   "A bird standing on a lily pad in a tranquil pond.",
#   "A bird perched on the edge of a stone fountain in a town square.",
#   "A bird hopping around on the cobblestone street of a quiet village.",
#   "A bird sitting on the handle of a colorful umbrella left open in a park.",
#   "A bird flying past a windmill in a rural landscape.",
#   "A bird perched on the edge of a wooden pier at dawn.",
#   "A bird resting on the top of a street lamp during dusk.",
#   "A bird pecking at the ground in a bustling marketplace.",
#   "A bird perched on the rim of a water barrel in a farmhouse yard.",
#   "A bird sitting on the edge of a bird feeder in a suburban backyard.",
#   "A bird flying through the smoke rising from a campfire in the woods.",
#   "A bird perched on the branch of a tree overlooking a serene lake.",
#   "A bird resting on the top of a snow-covered fence post.",
#   "A bird gliding over the surface of a calm river at sunset.",
#   "A bird perched on the windowsill of an old abandoned house.",
#   "A bird drinking water from a dew-covered leaf in a garden.",
#   "A bird sitting on a fence post surrounded by tall sunflowers.",
#   "A bird flying across a clear blue sky with fluffy white clouds.",
#   "A bird perched on the edge of a stone well in a meadow.",
#   "A bird hopping around the base of a large oak tree.",
#   "A bird perched on the railing of a wooden deck overlooking a forest.",
#   "A bird resting on the handle of a fishing rod leaning against a pier.",
#   "A bird sitting on the top of a street sign at a crossroads.",
#   "A bird flying through a field of tall wheat swaying in the wind.",
#   "A bird perched on the edge of a clay pot filled with colorful flowers.",
#   "A bird hopping along the gravel path of a botanical garden.",
#   "A bird resting on the rim of a rain barrel in a cottage garden.",
#   "A bird pecking at insects on the bark of a tall tree.",
#   "A bird sitting on the armrest of a park bench in the early morning light.",
#   "A bird flying over a field dotted with grazing cows.",
#   "A bird perched on the edge of a broken window in an old barn.",
#   "A bird resting on the branch of a tree during a light snowfall.",
#   "A bird hopping around on the moss-covered stones of a garden path.",
#   "A bird perched on the handle of an old wooden wagon in a field.",
#   "A bird gliding over a serene lake surrounded by mountains.",
#   "A bird sitting on the top of a street lamp at night.",
#   "A bird flying through a field of lavender in full bloom.",
#   "A bird perched on the edge of a stone birdbath in a garden.",
#   "A bird resting on the roof of a small cabin in the woods.",
#   "A bird pecking at crumbs on the steps of a library.",
#   "A bird perched on the top of a wind-blown tree on a cliff.",
#   "A bird sitting on the handle of a watering can in a garden.",
#   "A bird flying over a foggy meadow at dawn.",
#   "A bird perched on the rim of a clay pot filled with herbs.",
#   "A bird resting on the branch of a tree overlooking a river.",
#   "A bird sitting on the edge of a stone bench in a garden.",
#   "A bird perched on the handle of a shovel stuck in the ground.",
#   "A bird flying through a field of colorful wildflowers.",
#   "A bird resting on the top of a fence post at sunrise.",
#   "A bird perched on the edge of a metal bucket filled with water.",
#   "A bird sitting on the edge of a wooden deck overlooking a pond.",
#   "A bird flying over a field of ripe wheat ready for harvest.",
#   "A bird perched on the windowsill of a small village house.",
#   "A bird resting on the handle of a pitchfork in a barnyard.",
#   "A bird pecking at seeds scattered on the ground in a park.",
#   "A bird perched on the branch of a tree in full bloom.",
#   "A bird flying over a serene valley at sunset.",
#   "A bird resting on the top of a garden statue.",
#   "A bird perched on the edge of a stone wall surrounding a garden.",
#   "A bird sitting on the rim of a large terracotta pot filled with flowers.",
]

# Non-animal prompts

In [ ]:
"""
sources
- https://chatgpt.com/share/6d374bc1-afec-4128-b379-0eb55b608dd7
- https://chatgpt.com/share/b0998f61-43f8-4e2d-aafd-b819ca019699
"""

non_animal_prompts = [
  "a young boy holding a tennis racket in a park",
  "a tennis racket on a tennis court",
  "a close-up of a tennis racket",
  "a tennis racket with a colorful grip",
  "a tennis racket beside a tennis ball",
  "a tennis racket in a professional player's hands",
  "a tennis racket in a sports store",
  "a tennis racket with fresh strings",
  "a tennis racket and a tennis net",
  "a tennis racket on a clay court",
  "a tennis racket with a city's skyline in the background",
  "a vintage tennis racket",
  "a tennis racket covered in stickers",
  "a tennis racket used as a prop",
  "a tennis racket and a broken string",
  "a tennis racket autographed by a famous player",
  "a tennis racket in a display case",
  "a tennis racket being restrung",
  "a tennis racket with a custom design",
  "a tennis racket next to a water bottle",
  "a tennis racket in a child's play area",
  "a tennis racket and a pair of shoes",
  "a tennis racket in a locker room",
  "a tennis racket with a cracked frame",
  "a tennis racket used in a photoshoot",
  "a tennis racket in a modern apartment",
  "a tennis racket and a training manual",
  "a tennis racket in a film set",
  "a tennis racket on a rainy day",
  "a tennis racket and a sports bag",
  "a tennis racket being held by a silhouette",
  "a tennis racket in a dark room",
  "a tennis racket next to a towel",
  "a tennis racket lying on the ground",
  "a tennis racket and a sunset",
  "a tennis racket on a picnic blanket",
  "a tennis racket next to a stack of books",
  "a tennis racket in a field of wildflowers",
  "a tennis racket with a ribbon tied around it",
  "a tennis racket in a child's toy collection",
  "a tennis racket at the beach",
  "a tennis racket and a kite in the sky",
  "a tennis racket in a school gym",
  "a tennis racket with a superhero logo",
  "a tennis racket in a garden",
  "a tennis racket at a carnival",
  "a tennis racket with a ferris wheel in the background",
  "a tennis racket on a boat",
  "a tennis racket in a forest",
  "a tennis racket with a butterfly landing on it",
  "a tennis racket in the sand",
  "a tennis racket next to a campfire",
  "a tennis racket with fireworks in the background",
  "a tennis racket on a mountain trail",
  "a tennis racket in a music festival crowd",
  "a tennis racket at a construction site",
  "a tennis racket in a library",
  "a tennis racket in a science lab",
  "a tennis racket with a rainbow in the background",
  "a tennis racket in a desert",
  "a tennis racket at a wedding",
  "a tennis racket and a bouquet of flowers",
  "a tennis racket in an art studio",
  "a tennis racket at a dog park",
  "a tennis racket and a playground slide",
  "a tennis racket on a city street",
  "a tennis racket in a car trunk",
  "a tennis racket at a swimming pool",
  "a tennis racket next to a snowman",
  "a tennis racket at a ski resort",
  "a tennis racket in a coffee shop",
  "a tennis racket on a college campus",
  "a tennis racket with a graduation cap",
  "a tennis racket in a grocery store",
  "a tennis racket in an ice cream shop",
  "a tennis racket with a lemonade stand",
  "a tennis racket at a street fair",
  "a tennis racket in a museum",
  "a tennis racket at a train station",
  "a tennis racket in a hotel lobby",
  "a tennis racket on a rooftop",
  "a tennis racket with a telescope",
  "a tennis racket at a science fair",
  "a tennis racket in a hospital",
  "a tennis racket in a firefighter's break room",
  "a tennis racket at a police station",
  "a tennis racket at a city hall",
  "a tennis racket in a flower shop",
  "a tennis racket in a bakery",
  "a tennis racket in a barber shop",
  "a tennis racket in a pet store",
  "a tennis racket at an amusement park",
  "a tennis racket with a roller coaster",
  "a tennis racket at a halloween party",
  "a tennis racket in a haunted house",
  "a tennis racket at a holiday parade",
  "a tennis racket with christmas lights",
  "a tennis racket at a new year's eve party",
  "a tennis racket with confetti",
  "a tennis racket in a superhero-themed room",
  "a tennis racket with an astronaut's helmet",
  "a young boy holding skis in a park",
  "skis on a snowy mountain",
  "a close-up of a pair of skis",
  "skis with a retro design",
  "skis beside a ski lodge",
  "skis in a professional skier's hands",
  "skis in a rental shop",
  "skis with fresh snow on them",
  "skis and ski poles in the snow",
  "skis on a mountain slope",
  "skis with a city's skyline in the background",
  "vintage wooden skis",
  "skis covered in stickers",
  "skis used as a prop",
  "skis and a broken ski pole",
  "skis autographed by a famous skier",
  "skis in a display case",
  "skis being waxed",
  "skis with colorful designs",
  "skis next to a snowboard",
  "skis in a child's play area",
  "skis and a helmet",
  "skis in a sports store",
  "skis with a cracked tip",
  "skis used in a photoshoot",
  "skis in a modern apartment",
  "skis and a mountain trail map",
  "skis in a film set",
  "skis on a rainy day",
  "skis and a backpack",
  "skis being used by a silhouette",
  "skis in a dark room",
  "skis next to a pair of boots",
  "skis lying on the ground",
  "skis and a sunset",
  "skis on a picnic blanket",
  "skis next to a stack of books",
  "skis in a field of wildflowers",
  "skis with a ribbon tied around them",
  "skis in a child's toy collection",
  "skis at the beach",
  "skis and a kite in the sky",
  "skis in a school gym",
  "skis with a superhero logo",
  "skis in a garden",
  "skis at a carnival",
  "skis with a ferris wheel in the background",
  "skis on a boat",
  "skis in a forest",
  "skis with a butterfly landing on them",
  "skis in the sand",
  "skis next to a campfire",
  "skis with fireworks in the background",
  "skis on a mountain trail",
  "skis in a music festival crowd",
  "skis at a construction site",
  "skis in a library",
  "skis in a science lab",
  "skis with a rainbow in the background",
  "skis in a desert",
  "skis at a wedding",
  "skis and a bouquet of flowers",
  "skis in an art studio",
  "skis at a dog park",
  "skis and a playground slide",
  "skis on a city street",
  "skis in a car trunk",
  "skis at a swimming pool",
  "skis next to a snowman",
  "skis at a ski resort",
  "skis in a coffee shop",
  "skis on a college campus",
  "skis with a graduation cap",
  "skis in a grocery store",
  "skis in an ice cream shop",
  "skis with a lemonade stand",
  "skis at a street fair",
  "skis in a museum",
  "skis at a train station",
  "skis in a hotel lobby",
  "skis on a rooftop",
  "skis with a telescope",
  "skis at a science fair",
  "skis in a hospital",
  "skis in a firefighter's break room",
  "skis at a police station",
  "skis at a city hall",
  "skis in a flower shop",
  "skis in a bakery",
  "skis in a barber shop",
  "skis in a pet store",
  "skis at an amusement park",
  "skis with a roller coaster",
  "skis at a halloween party",
  "skis in a haunted house",
  "skis at a holiday parade",
  "skis with christmas lights",
  "skis at a new year's eve party",
  "skis with confetti",
  "skis in a superhero-themed room",
  "skis with an astronaut's helmet",
  "A young boy holding a toaster in a park.",
  "A toaster on a kitchen counter.",
  "A close-up of a toaster.",
  "A toaster with a retro design.",
  "A toaster beside a coffee maker.",
  "A toaster in a chef's kitchen.",
  "A toaster in a break room.",
  "A toaster with slices of bread inside.",
  "A toaster and a jar of jam.",
  "A toaster on a breakfast table.",
  "A toaster with a city's skyline in the background.",
  "A vintage toaster.",
  "A toaster covered in stickers.",
  "A toaster used as a prop.",
  "A toaster and a broken plate.",
  "A toaster autographed by a celebrity chef.",
  "A toaster in a display case.",
  "A toaster being cleaned.",
  "A toaster with colorful buttons.",
  "A toaster next to a refrigerator.",
  "A toaster in a child's play kitchen.",
  "A toaster and a kettle.",
  "A toaster in a store.",
  "A toaster with a cracked body.",
  "A toaster used in a photoshoot.",
  "A toaster in a modern apartment.",
  "A toaster and a cookbook.",
  "A toaster in a film set.",
  "A toaster on a rainy day.",
  "A toaster and a dish towel.",
  "A toaster being used by a silhouette.",
  "A toaster in a dark room.",
  "A toaster next to a bowl of fruit.",
  "A toaster lying on the floor.",
  "A toaster and a sunset.",
  "A toaster on a picnic blanket.",
  "A toaster next to a stack of books.",
  "A toaster in a field of wildflowers.",
  "A toaster with a ribbon tied around it.",
  "A toaster in a child's toy collection.",
  "A toaster at the beach.",
  "A toaster and a kite in the sky.",
  "A toaster in a school cafeteria.",
  "A toaster with a superhero logo.",
  "A toaster in a garden.",
  "A toaster at a carnival.",
  "A toaster with a Ferris wheel in the background.",
  "A toaster on a boat.",
  "A toaster in a forest.",
  "A toaster with a butterfly landing on it.",
  "A toaster in the sand.",
  "A toaster next to a campfire.",
  "A toaster with fireworks in the background.",
  "A toaster on a mountain trail.",
  "A toaster in a music festival crowd.",
  "A toaster at a construction site.",
  "A toaster in a library.",
  "A toaster in a science lab.",
  "A toaster with a rainbow in the background.",
  "A toaster in a desert.",
  "A toaster at a wedding.",
  "A toaster and a bouquet of flowers.",
  "A toaster in an art studio.",
  "A toaster at a dog park.",
  "A toaster and a playground slide.",
  "A toaster on a city street.",
  "A toaster in a car trunk.",
  "A toaster at a swimming pool.",
  "A toaster next to a snowman.",
  "A toaster at a ski resort.",
  "A toaster in a coffee shop.",
  "A toaster on a college campus.",
  "A toaster with a graduation cap.",
  "A toaster in a grocery store.",
  "A toaster in an ice cream shop.",
  "A toaster with a lemonade stand.",
  "A toaster at a street fair.",
  "A toaster in a museum.",
  "A toaster at a train station.",
  "A toaster in a hotel lobby.",
  "A toaster on a rooftop.",
  "A toaster with a telescope.",
  "A toaster at a science fair.",
  "A toaster in a hospital.",
  "A toaster in a firefighter's break room.",
  "A toaster at a police station.",
  "A toaster at a city hall.",
  "A toaster in a flower shop.",
  "A toaster in a bakery.",
  "A toaster in a barber shop.",
  "A toaster in a pet store.",
  "A toaster at an amusement park.",
  "A toaster with a roller coaster.",
  "A toaster at a Halloween party.",
  "A toaster in a haunted house.",
  "A toaster at a holiday parade.",
  "A toaster with Christmas lights.",
  "A toaster at a New Year's Eve party.",
  "A toaster with confetti.",
  "A toaster in a superhero-themed room.",
  "A toaster with an astronaut's helmet.",
  "A young boy holding a computer mouse in a park.",
  "A computer mouse on a desk.",
  "A close-up of a computer mouse.",
  "A computer mouse with a gaming logo.",
  "A computer mouse beside a keyboard.",
  "A computer mouse in a programmer's hands.",
  "A computer mouse in a home office.",
  "A computer mouse with colorful LEDs.",
  "A computer mouse and a mouse pad.",
  "A computer mouse on a workstation.",
  "A computer mouse with a city's skyline in the background.",
  "A vintage computer mouse.",
  "A computer mouse covered in stickers.",
  "A computer mouse used as a prop.",
  "A computer mouse and a broken screen.",
  "A computer mouse autographed by a famous gamer.",
  "A computer mouse in a display case.",
  "A computer mouse being cleaned.",
  "A computer mouse with customizable buttons.",
  "A computer mouse next to a laptop.",
  "A computer mouse in a child's study area.",
  "A computer mouse and a gaming console.",
  "A computer mouse in a store.",
  "A computer mouse with a cracked shell.",
  "A computer mouse used in a photoshoot.",
  "A computer mouse in a modern office.",
  "A computer mouse and a monitor.",
  "A computer mouse in a film set.",
  "A computer mouse on a rainy day.",
  "A computer mouse and a notepad.",
  "A computer mouse being held by a silhouette.",
  "A computer mouse in a dark room.",
  "A computer mouse next to a cup of coffee.",
  "A computer mouse lying on the floor.",
  "A computer mouse and a sunset.",
  "A computer mouse on a picnic blanket.",
  "A computer mouse next to a stack of books.",
  "A computer mouse in a field of wildflowers.",
  "A computer mouse with a ribbon tied around it.",
  "A computer mouse in a child's toy collection.",
  "A computer mouse at the beach.",
  "A computer mouse and a kite in the sky.",
  "A computer mouse in a school computer lab.",
  "A computer mouse with a superhero logo.",
  "A computer mouse in a garden.",
  "A computer mouse at a carnival.",
  "A computer mouse with a Ferris wheel in the background.",
  "A computer mouse on a boat.",
  "A computer mouse in a forest.",
  "A computer mouse with a butterfly landing on it.",
  "A computer mouse in the sand.",
  "A computer mouse next to a campfire.",
  "A computer mouse with fireworks in the background.",
  "A computer mouse on a mountain trail.",
  "A computer mouse in a music festival crowd.",
  "A computer mouse at a construction site.",
  "A computer mouse in a library.",
  "A computer mouse in a science lab.",
  "A computer mouse with a rainbow in the background.",
  "A computer mouse in a desert.",
  "A computer mouse at a wedding.",
  "A computer mouse and a bouquet of flowers.",
  "A computer mouse in an art studio.",
  "A computer mouse at a dog park.",
  "A computer mouse and a playground slide.",
  "A computer mouse on a city street.",
  "A computer mouse in a car trunk.",
  "A computer mouse at a swimming pool.",
  "A computer mouse next to a snowman.",
  "A computer mouse at a ski resort.",
  "A computer mouse in a coffee shop.",
  "A computer mouse on a college campus.",
  "A computer mouse with a graduation cap.",
  "A computer mouse in a grocery store.",
  "A computer mouse in an ice cream shop.",
  "A computer mouse with a lemonade stand.",
  "A computer mouse at a street fair.",
  "A computer mouse in a museum.",
  "A computer mouse at a train station.",
  "A computer mouse in a hotel lobby.",
  "A computer mouse on a rooftop.",
  "A computer mouse with a telescope.",
  "A computer mouse at a science fair.",
  "A computer mouse in a hospital.",
  "A computer mouse in a firefighter's break room.",
  "A computer mouse at a police station.",
  "A computer mouse at a city hall.",
  "A computer mouse in a flower shop.",
  "A computer mouse in a bakery.",
  "A computer mouse in a barber shop.",
  "A computer mouse in a pet store.",
  "A computer mouse at an amusement park.",
  "A computer mouse with a roller coaster.",
  "A computer mouse at a Halloween party.",
  "A computer mouse in a haunted house.",
  "A computer mouse at a holiday parade.",
  "A computer mouse with Christmas lights.",
  "A computer mouse at a New Year's Eve party.",
  "A computer mouse with confetti.",
  "A computer mouse in a superhero-themed room.",
  "A computer mouse with an astronaut's helmet.",
  "A young boy holding a microwave in a park.",
  "A microwave on a kitchen counter.",
  "A close-up of a microwave.",
  "A microwave with a digital display.",
  "A microwave beside a toaster.",
  "A microwave in a chef's kitchen.",
  "A microwave in a break room.",
  "A microwave with a bowl of popcorn inside.",
  "A microwave and a mug.",
  "A microwave on a dining table.",
  "A microwave with a city's skyline in the background.",
  "A vintage microwave.",
  "A microwave covered in stickers.",
  "A microwave used as a prop.",
  "A microwave and a broken plate.",
  "A microwave autographed by a celebrity chef.",
  "A microwave in a display case.",
  "A microwave being cleaned.",
  "A microwave with colorful buttons.",
  "A microwave next to a refrigerator.",
  "A microwave in a child's play kitchen.",
  "A microwave and a kettle.",
  "A microwave in a store.",
  "A microwave with a cracked door.",
  "A microwave used in a photoshoot.",
  "A microwave in a modern apartment.",
  "A microwave and a cookbook.",
  "A microwave in a film set.",
  "A microwave on a rainy day.",
  "A microwave and a dish towel.",
  "A microwave being used by a silhouette.",
  "A microwave in a dark room.",
  "A microwave next to a bowl of fruit.",
  "A microwave lying on the floor.",
  "A microwave and a sunset.",
  "A microwave on a picnic blanket.",
  "A microwave next to a stack of books.",
  "A microwave in a field of wildflowers.",
  "A microwave with a ribbon tied around it.",
  "A microwave in a child's toy collection.",
  "A microwave at the beach.",
  "A microwave and a kite in the sky.",
  "A microwave in a school cafeteria.",
  "A microwave with a superhero logo.",
  "A microwave in a garden.",
  "A microwave at a carnival.",
  "A microwave with a Ferris wheel in the background.",
  "A microwave on a boat.",
  "A microwave in a forest.",
  "A microwave with a butterfly landing on it.",
  "A microwave in the sand.",
  "A microwave next to a campfire.",
  "A microwave with fireworks in the background.",
  "A microwave on a mountain trail.",
  "A microwave in a music festival crowd.",
  "A microwave at a construction site.",
  "A microwave in a library.",
  "A microwave in a science lab.",
  "A microwave with a rainbow in the background.",
  "A microwave in a desert.",
  "A microwave at a wedding.",
  "A microwave and a bouquet of flowers.",
  "A microwave in an art studio.",
  "A microwave at a dog park.",
  "A microwave and a playground slide.",
  "A microwave on a city street.",
  "A microwave in a car trunk.",
  "A microwave at a swimming pool.",
  "A microwave next to a snowman.",
  "A microwave at a ski resort.",
  "A microwave in a coffee shop.",
  "A microwave on a college campus.",
  "A microwave with a graduation cap.",
  "A microwave in a grocery store.",
  "A microwave in an ice cream shop.",
  "A microwave with a lemonade stand.",
  "A microwave at a street fair.",
  "A microwave in a museum.",
  "A microwave at a train station.",
  "A microwave in a hotel lobby.",
  "A microwave on a rooftop.",
  "A microwave with a telescope.",
  "A microwave at a science fair.",
  "A microwave in a hospital.",
  "A microwave in a firefighter's break room.",
  "A microwave at a police station.",
  "A microwave at a city hall.",
  "A microwave in a flower shop.",
  "A microwave in a bakery.",
  "A microwave in a barber shop.",
  "A microwave in a pet store.",
  "A microwave at an amusement park.",
  "A microwave with a roller coaster.",
  "A microwave at a Halloween party.",
  "A microwave in a haunted house.",
  "A microwave at a holiday parade.",
  "A microwave with Christmas lights.",
  "A microwave at a New Year's Eve party.",
  "A microwave with confetti.",
  "A microwave in a superhero-themed room.",
  "A microwave with an astronaut's helmet.",
  "A young boy holding a hair drier in a park.",
  "A hair drier on a bathroom counter.",
  "A close-up of a hair drier.",
  "A hair drier with a salon logo.",
  "A hair drier beside a brush.",
  "A hair drier in a stylist's hands.",
  "A hair drier in a salon.",
  "A hair drier blowing in the wind.",
  "A hair drier and a mirror.",
  "A hair drier on a vanity table.",
  "A hair drier with a city's skyline in the background.",
  "A vintage hair drier.",
  "A hair drier covered in stickers.",
  "A hair drier used as a prop.",
  "A hair drier and a broken mirror.",
  "A hair drier autographed by a celebrity.",
  "A hair drier in a display case.",
  "A hair drier being cleaned.",
  "A hair drier with colorful decorations.",
  "A hair drier next to a hair straightener.",
  "A hair drier in a child's bedroom.",
  "A hair drier and a wig stand.",
  "A hair drier in a store.",
  "A hair drier with a cracked nozzle.",
  "A hair drier used in a photoshoot.",
  "A hair drier held by a person in a uniform.",
  "A hair drier and a makeup station.",
  "A hair drier in a film set.",
  "A hair drier on a rainy day.",
  "A hair drier and a towel.",
  "A hair drier being held by a silhouette.",
  "A hair drier in a dark room.",
  "A hair drier next to a shampoo bottle.",
  "A hair drier lying on the floor.",
  "A hair drier and a sunset.",
  "A hair drier on a picnic blanket.",
  "A hair drier next to a stack of books.",
  "A hair drier in a field of wildflowers.",
  "A hair drier with a ribbon tied around it.",
  "A hair drier in a child's toy collection.",
  "A hair drier at the beach.",
  "A hair drier and a kite in the sky.",
  "A hair drier in a school locker.",
  "A hair drier with a superhero logo.",
  "A hair drier in a garden.",
  "A hair drier at a carnival.",
  "A hair drier with a Ferris wheel in the background.",
  "A hair drier on a boat.",
  "A hair drier in a forest.",
  "A hair drier with a butterfly landing on it.",
  "A hair drier in the sand.",
  "A hair drier next to a campfire.",
  "A hair drier with fireworks in the background.",
  "A hair drier on a mountain trail.",
  "A hair drier in a music festival crowd.",
  "A hair drier at a construction site.",
  "A hair drier in a library.",
  "A hair drier in a science lab.",
  "A hair drier with a rainbow in the background.",
  "A hair drier in a desert.",
  "A hair drier at a wedding.",
  "A hair drier and a bouquet of flowers.",
  "A hair drier in an art studio.",
  "A hair drier at a dog park.",
  "A hair drier and a playground slide.",
  "A hair drier on a city street.",
  "A hair drier in a car trunk.",
  "A hair drier at a swimming pool.",
  "A hair drier next to a snowman.",
  "A hair drier at a ski resort.",
  "A hair drier in a coffee shop.",
  "A hair drier on a college campus.",
  "A hair drier with a graduation cap.",
  "A hair drier in a grocery store.",
  "A hair drier in an ice cream shop.",
  "A hair drier with a lemonade stand.",
  "A hair drier at a street fair.",
  "A hair drier in a museum.",
  "A hair drier at a train station.",
  "A hair drier in a hotel lobby.",
  "A hair drier on a rooftop.",
  "A hair drier with a telescope.",
  "A hair drier at a science fair.",
  "A hair drier in a hospital.",
  "A hair drier in a firefighter's hands.",
  "A hair drier at a police station.",
  "A hair drier at a city hall.",
  "A hair drier in a flower shop.",
  "A hair drier in a bakery.",
  "A hair drier in a barber shop.",
  "A hair drier in a pet store.",
  "A hair drier at an amusement park.",
  "A hair drier with a roller coaster.",
  "A hair drier at a Halloween party.",
  "A hair drier in a haunted house.",
  "A hair drier at a holiday parade.",
  "A hair drier with Christmas lights.",
  "A hair drier at a New Year's Eve party.",
  "A hair drier with confetti.",
  "A hair drier in a superhero-themed room.",
  "A hair drier with an astronaut's helmet.",
  "A young boy sitting at a dining table in a park.",
  "A dining table set for dinner.",
  "A close-up of a dining table's surface.",
  "A dining table with a floral centerpiece.",
  "A dining table beside a window.",
  "A dining table with a team logo tablecloth.",
  "A dining table in a cozy kitchen.",
  "A dining table with a family gathered around.",
  "A dining table with a holiday feast.",
  "A dining table under a chandelier.",
  "A dining table with a city skyline in the background.",
  "A vintage wooden dining table.",
  "A dining table covered in art supplies.",
  "A dining table used as a workbench.",
  "A dining table and a broken vase.",
  "A dining table with an autographed table runner.",
  "A dining table in a formal dining room.",
  "A dining table being set by a waiter.",
  "A dining table with colorful place settings.",
  "A dining table next to a fireplace.",
  "A dining table in a child's playroom.",
  "A dining table and a high chair.",
  "A dining table in a restaurant.",
  "A dining table with a cracked leg.",
  "A dining table used as a prop in a movie scene.",
  "A dining table in a modern apartment.",
  "A dining table and a large mirror in the background.",
  "A dining table surrounded by friends.",
  "A dining table on a rainy day.",
  "A dining table with a serving platter.",
  "A dining table with a silhouette of people.",
  "A dining table in a dimly lit room.",
  "A dining table next to a kitchen island.",
  "A dining table with a vase of wildflowers.",
  "A dining table and a sunset view.",
  "A dining table on a patio.",
  "A dining table with a stack of books.",
  "A dining table with a field of wildflowers outside.",
  "A dining table with a ribbon tied around a chair.",
  "A dining table in a child's toy collection.",
  "A dining table at the beach.",
  "A dining table and a kite in the sky.",
  "A dining table in a school cafeteria.",
  "A dining table with a superhero theme.",
  "A dining table in a garden.",
  "A dining table at a carnival.",
  "A dining table with a Ferris wheel view.",
  "A dining table on a fishing pier.",
  "A dining table in a forest clearing.",
  "A dining table with a butterfly decoration.",
  "A dining table in the sand.",
  "A dining table next to a campfire.",
  "A dining table with fireworks overhead.",
  "A dining table on a mountain trail.",
  "A dining table at a music festival.",
  "A dining table at a construction site.",
  "A dining table in a library.",
  "A dining table in a science lab.",
  "A dining table with a rainbow view.",
  "A dining table in a desert.",
  "A dining table at a wedding.",
  "A dining table with a bouquet of flowers.",
  "A dining table in an art studio.",
  "A dining table at a dog park.",
  "A dining table and a playground slide.",
  "A dining table on a city street.",
  "A dining table in a car trunk.",
  "A dining table at a swimming pool.",
  "A dining table next to a snowman.",
  "A dining table at a ski resort.",
  "A dining table in a coffee shop.",
  "A dining table on a college campus.",
  "A dining table with a graduation cap centerpiece.",
  "A dining table in a grocery store.",
  "A dining table in an ice cream shop.",
  "A dining table with a lemonade pitcher.",
  "A dining table at a street fair.",
  "A dining table in a museum.",
  "A dining table at a train station.",
  "A dining table in a hotel lobby.",
  "A dining table on a rooftop.",
  "A dining table with a telescope.",
  "A dining table at a science fair.",
  "A dining table in a hospital.",
  "A dining table in a firefighter's break room.",
  "A dining table at a police station.",
  "A dining table at a city hall.",
  "A dining table in a flower shop.",
  "A dining table in a bakery.",
  "A dining table in a barber shop.",
  "A dining table in a pet store.",
  "A dining table at an amusement park.",
  "A dining table with a roller coaster view.",
  "A dining table at a Halloween party.",
  "A dining table in a haunted house.",
  "A dining table at a holiday parade.",
  "A dining table with Christmas lights.",
  "A dining table at a New Year's Eve party.",
  "A dining table with confetti.",
  "A dining table in a superhero-themed room.",
  "A dining table with an astronaut centerpiece.",
  "A young boy holding a bowl in a park.",
  "A bowl on a kitchen counter.",
  "A close-up of a ceramic bowl.",
  "A bowl with a team logo.",
  "A bowl beside a spoon.",
  "A bowl in a chef's hands.",
  "A bowl in a cupboard.",
  "A bowl filled with soup.",
  "A bowl and a pair of chopsticks.",
  "A bowl on a dining table.",
  "A bowl with fruit inside.",
  "A vintage wooden bowl.",
  "A bowl covered in paint.",
  "A bowl used as a planter.",
  "A bowl and a broken glass.",
  "A bowl autographed by a famous chef.",
  "A bowl in a display case.",
  "A bowl being crafted in a pottery workshop.",
  "A bowl with colorful patterns.",
  "A bowl next to a plate.",
  "A bowl in a child's play kitchen.",
  "A bowl and a pot on a stove.",
  "A bowl in a grocery store.",
  "A bowl with a cracked rim.",
  "A bowl used as a prop in a movie scene.",
  "A bowl held by a person in an apron.",
  "A bowl and a cutting board in the background.",
  "A bowl and a group of people sharing a meal.",
  "A bowl on a rainy windowsill.",
  "A bowl and a serving ladle.",
  "A bowl being washed in a sink.",
  "A bowl in a dimly lit room.",
  "A bowl next to a mug.",
  "A bowl lying in the grass.",
  "A bowl and a sunset.",
  "A bowl on a picnic blanket.",
  "A bowl next to a stack of books.",
  "A bowl and a field of wildflowers.",
  "A bowl with a ribbon tied around it.",
  "A bowl in a child's toy collection.",
  "A bowl at the beach.",
  "A bowl and a kite in the sky.",
  "A bowl on a school cafeteria table.",
  "A bowl with a superhero design.",
  "A bowl in a garden.",
  "A bowl at a carnival.",
  "A bowl and a Ferris wheel.",
  "A bowl on a fishing boat.",
  "A bowl in a forest.",
  "A bowl with a butterfly landing on it.",
  "A bowl in the sand.",
  "A bowl next to a campfire.",
  "A bowl with fireworks in the background.",
  "A bowl on a mountain trail.",
  "A bowl in a music festival crowd.",
  "A bowl at a construction site.",
  "A bowl in a library.",
  "A bowl in a science lab.",
  "A bowl with a rainbow in the background.",
  "A bowl in a desert.",
  "A bowl at a wedding.",
  "A bowl and a bouquet of flowers.",
  "A bowl in an art studio.",
  "A bowl at a dog park.",
  "A bowl and a playground slide.",
  "A bowl on a city street.",
  "A bowl in a car trunk.",
  "A bowl at a swimming pool.",
  "A bowl next to a snowman.",
  "A bowl at a ski resort.",
  "A bowl in a coffee shop.",
  "A bowl on a college campus.",
  "A bowl with a graduation cap.",
  "A bowl in a grocery store.",
  "A bowl in an ice cream shop.",
  "A bowl with a lemonade stand.",
  "A bowl at a street fair.",
  "A bowl in a museum.",
  "A bowl at a train station.",
  "A bowl in a hotel lobby.",
  "A bowl on a rooftop.",
  "A bowl with a telescope.",
  "A bowl at a science fair.",
  "A bowl in a hospital.",
  "A bowl in a firefighter's hands.",
  "A bowl at a police station.",
  "A bowl at a city hall.",
  "A bowl in a flower shop.",
  "A bowl in a bakery.",
  "A bowl in a barber shop.",
  "A bowl in a pet store.",
  "A bowl at an amusement park.",
  "A bowl with a roller coaster.",
  "A bowl at a Halloween party.",
  "A bowl in a haunted house.",
  "A bowl at a holiday parade.",
  "A bowl with Christmas lights.",
  "A bowl at a New Year's Eve party.",
  "A bowl with confetti.",
  "A bowl in a superhero costume.",
  "A bowl with an astronaut's helmet.",
  "A young boy holding a baseball bat in a park.",
  "A baseball bat leaning against a fence.",
  "A close-up of a worn-out baseball bat.",
  "A baseball bat with a team logo.",
  "A baseball bat beside a baseball glove.",
  "A baseball bat in a player's hands ready to swing.",
  "A baseball bat in a dugout.",
  "A baseball bat in the middle of a swing.",
  "A baseball bat and ball mid-air.",
  "A baseball bat on a baseball diamond.",
  "A baseball bat with a city's skyline in the background.",
  "A vintage wooden baseball bat.",
  "A baseball bat covered in dirt and grass.",
  "A baseball bat used as a walking stick.",
  "A baseball bat and a broken window.",
  "A baseball bat autographed by a famous player.",
  "A baseball bat in a trophy case.",
  "A baseball bat being crafted in a workshop.",
  "A baseball bat with colorful graffiti.",
  "A baseball bat next to a home plate.",
  "A baseball bat in a child's bedroom.",
  "A baseball bat and a pitcher's mound.",
  "A baseball bat in a sports store.",
  "A baseball bat with a cracked handle.",
  "A baseball bat used as a prop in a movie scene.",
  "A baseball bat held by a player in a uniform.",
  "A baseball bat and a scoreboard in the background.",
  "A baseball bat and a cheering crowd.",
  "A baseball bat on a rainy baseball field.",
  "A baseball bat and a catcher's mitt.",
  "A baseball bat being swung by a silhouette.",
  "A baseball bat in a shadowy alley.",
  "A baseball bat next to a baseball helmet.",
  "A baseball bat lying in the grass.",
  "A baseball bat and a sunset.",
  "A baseball bat on a picnic table.",
  "A baseball bat next to a sports bag.",
  "A baseball bat and a field of wildflowers.",
  "A baseball bat with a ribbon tied around it.",
  "A baseball bat in a child's toy collection.",
  "A baseball bat at the beach.",
  "A baseball bat and a kite in the sky.",
  "A baseball bat on a school playground.",
  "A baseball bat with a superhero logo.",
  "A baseball bat in a garden.",
  "A baseball bat at a carnival.",
  "A baseball bat and a Ferris wheel.",
  "A baseball bat on a fishing boat.",
  "A baseball bat in a forest.",
  "A baseball bat with a butterfly landing on it.",
  "A baseball bat in the sand.",
  "A baseball bat next to a campfire.",
  "A baseball bat with fireworks in the background.",
  "A baseball bat on a mountain trail.",
  "A baseball bat in a music festival crowd.",
  "A baseball bat at a construction site.",
  "A baseball bat in a library.",
  "A baseball bat in a science lab.",
  "A baseball bat with a rainbow in the background.",
  "A baseball bat in a desert.",
  "A baseball bat at a wedding.",
  "A baseball bat and a bouquet of flowers.",
  "A baseball bat in an art studio.",
  "A baseball bat at a dog park.",
  "A baseball bat and a playground slide.",
  "A baseball bat on a city street.",
  "A baseball bat in a car trunk.",
  "A baseball bat at a swimming pool.",
  "A baseball bat next to a snowman.",
  "A baseball bat at a ski resort.",
  "A baseball bat in a coffee shop.",
  "A baseball bat on a college campus.",
  "A baseball bat with a graduation cap.",
  "A baseball bat in a grocery store.",
  "A baseball bat in an ice cream shop.",
  "A baseball bat with a lemonade stand.",
  "A baseball bat at a street fair.",
  "A baseball bat in a museum.",
  "A baseball bat at a train station.",
  "A baseball bat in a hotel lobby.",
  "A baseball bat on a rooftop.",
  "A baseball bat with a telescope.",
  "A baseball bat at a science fair.",
  "A baseball bat in a hospital.",
  "A baseball bat in a firefighter's hands.",
  "A baseball bat at a police station.",
  "A baseball bat at a city hall.",
  "A baseball bat in a flower shop.",
  "A baseball bat in a bakery.",
  "A baseball bat in a barber shop.",
  "A baseball bat in a pet store.",
  "A baseball bat at an amusement park.",
  "A baseball bat with a roller coaster.",
  "A baseball bat at a Halloween party.",
  "A baseball bat in a haunted house.",
  "A baseball bat at a holiday parade.",
  "A baseball bat with Christmas lights.",
  "A baseball bat at a New Year's Eve party.",
  "A baseball bat with confetti.",
  "A baseball bat in a superhero costume.",
  "A baseball bat with an astronaut's helmet.",
  "A chef's knife on a wooden cutting board.",
  "A knife slicing through a ripe tomato.",
  "A knife set displayed on a kitchen counter.",
  "A hunting knife in a leather sheath.",
  "A butter knife next to a bread basket.",
  "A knife with a serrated edge.",
  "A pocket knife with multiple tools.",
  "A knife being sharpened on a whetstone.",
  "A steak knife at a dinner table.",
  "A knife carving a roast chicken.",
  "A knife with a sleek, modern design.",
  "A knife with a bone handle.",
  "A paring knife peeling an apple.",
  "A knife block holding various knives.",
  "A bread knife with a long blade.",
  "A fillet knife next to a fresh fish.",
  "A kitchen knife with a wooden handle.",
  "A chef holding a knife in a professional kitchen.",
  "A knife with a Damascus steel blade.",
  "A cheese knife on a charcuterie board.",
  "A knife chopping herbs on a cutting board.",
  "A knife with a brightly colored handle.",
  "A throwing knife stuck in a target.",
  "A utility knife in a toolbox.",
  "A tactical knife with a black blade.",
  "A silk tie draped over a chair.",
  "A red tie on a white dress shirt.",
  "A man adjusting his tie in a mirror.",
  "A collection of ties in a wooden drawer.",
  "A patterned tie on a dark suit.",
  "A tie with a paisley design.",
  "A bow tie at a black-tie event.",
  "A blue tie with polka dots.",
  "A tie being knotted for a wedding.",
  "A striped tie on a businessman's desk.",
  "A tie hanging on a coat rack.",
  "A tie with a subtle geometric pattern.",
  "A tie with a Windsor knot.",
  "A tie displayed in a shop window.",
  "A vintage tie in a thrift store.",
  "A tie with a floral print.",
  "A school uniform with a tie.",
  "A tie being straightened before an interview.",
  "A skinny tie with a modern look.",
  "A novelty tie with cartoon characters.",
  "A tie clip holding a tie in place.",
  "A silk tie in a gift box.",
  "A man loosening his tie after work.",
  "A tie with a diamond pattern.",
  "A tie worn at a graduation ceremony.",
  "A father teaching his son to tie a tie.",
  "A tie draped over a suitcase.",
  "A black tie for a formal occasion.",
  "A tie with a monogrammed initial.",
  "A tie in a corporate office setting.",
  "A red backpack on a wooden bench.",
  "A child wearing a yellow backpack.",
  "A hiker's backpack in the mountains.",
  "A futuristic backpack with glowing lights.",
  "A backpack lying on a sandy beach.",
  "A student carrying a heavy backpack.",
  "A backpacker standing on a mountain peak.",
  "A child with a backpack waiting for the bus.",
  "An artist's backpack with supplies.",
  "A stylish backpack in a fashion store.",
  "A rugged backpack for outdoor adventures.",
  "A vintage leather backpack.",
  "A backpack covered in travel patches.",
  "A colorful backpack in a playground.",
  "A tiny backpack for a pet.",
  "A backpack filled with camping gear.",
  "A military-style backpack.",
  "A backpack hanging from a tree.",
  "A backpack with a solar panel.",
  "A backpack left in a park.",
  "A futuristic city with sleek backpacks.",
  "A backpacker resting by a lake.",
  "A glowing backpack in a dark room.",
  "A backpack next to a campfire.",
  "A backpack full of books.",
  "A neon-colored backpack.",
  "A backpack covered in graffiti.",
  "A backpack on a bicycle.",
  "A backpack on a train seat.",
  "A backpack with a teddy bear.",
  "A backpack in a school hallway.",
  "A backpack on a hiking trail.",
  "A backpack in an airport.",
  "A backpack on a snowy path.",
  "A backpack under a tree.",
  "A backpack next to a tent.",
  "A backpack with musical instruments.",
  "A backpack in a busy market.",
  "A backpack in a desert.",
  "A backpack on a city street.",
  "A stylish handbag on a display shelf.",
  "A leather handbag on a wooden table.",
  "A woman carrying a designer handbag.",
  "A colorful handbag in a boutique.",
  "A handbag with a floral pattern.",
  "A black handbag with gold accents.",
  "A handbag lying on a park bench.",
  "A vintage handbag with a clasp.",
  "A handbag filled with essentials.",
  "A handbag next to a pair of sunglasses.",
  "A large handbag on a beach towel.",
  "A small handbag at a dinner table.",
  "A handbag in a busy market.",
  "A handbag with a chain strap.",
  "A handbag with a scarf tied to it.",
  "A handbag in an airport lounge.",
  "A handbag on a car seat.",
  "A handbag next to a coffee cup.",
  "A handbag in a shopping cart.",
  "A handbag hanging on a coat rack.",
  "A handbag with a book peeking out.",
  "A handbag on a café table.",
  "A handbag in an office.",
  "A handbag with a phone sticking out.",
  "A handbag next to a pair of shoes.",
  "A handbag with a makeup pouch inside.",
  "A handbag on a train seat.",
  "A handbag in a hotel room.",
  "A handbag on a nightstand.",
  "A handbag in a restaurant.",
  "A handbag on a picnic blanket.",
  "A handbag with a keychain.",
  "A handbag on a bicycle basket.",
  "A handbag on a kitchen counter.",
  "A handbag in a dressing room.",
  "A handbag on a bar stool.",
  "A handbag in a garden.",
  "A handbag at a fashion show.",
  "A handbag on a swing.",
  "A handbag in a library.",
  "A stainless steel toaster on a kitchen counter.",
  "A toaster with two slices of bread.",
  "A retro-style toaster in a vintage kitchen.",
  "A red toaster next to a coffee maker.",
  "A toaster with a bagel setting.",
  "A toaster with crumbs underneath.",
  "A toaster in a modern kitchen.",
  "A toaster with toast popping up.",
  "A toaster with a waffle inside.",
  "A toaster with a defrost button.",
  "A toaster next to a butter dish.",
  "A toaster with four slots.",
  "A toaster with a removable crumb tray.",
  "A toaster with toast tongs.",
  "A toaster with a timer dial.",
  "A toaster in a breakfast nook.",
  "A toaster with a heating rack.",
  "A toaster with an egg poacher.",
  "A toaster with a warm pastry.",
  "A toaster with a reheat function.",
  "A toaster in a college dorm.",
  "A toaster with a glass window.",
  "A toaster on a marble countertop.",
  "A toaster with a cancel button.",
  "A toaster next to a cutting board.",
  "A toaster in a hotel room.",
  "A toaster with toast shaped like a heart.",
  "A toaster next to a fruit bowl.",
  "A toaster with a sleek design.",
  "A toaster with a cord wrap.",
  "A toaster on a wooden table.",
  "A toaster with a keep warm feature.",
  "A toaster with LED indicators.",
  "A toaster with a stainless steel finish.",
  "A toaster in an office break room.",
  "A toaster with a wide slot.",
  "A toaster with a lift lever.",
  "A toaster with a cool-touch exterior.",
  "A toaster with a digital display.",
  "A toaster on a breakfast tray.",
  "A stainless steel toaster on a kitchen counter.",
  "A toaster with two slices of bread.",
  "A retro-style toaster in a vintage kitchen.",
  "A red toaster next to a coffee maker.",
  "A toaster with a bagel setting.",
  "A toaster with crumbs underneath.",
  "A toaster in a modern kitchen.",
  "A toaster with toast popping up.",
  "A toaster with a waffle inside.",
  "A toaster with a defrost button.",
  "A toaster next to a butter dish.",
  "A toaster with four slots.",
  "A toaster with a removable crumb tray.",
  "A toaster with toast tongs.",
  "A toaster with a timer dial.",
  "A toaster in a breakfast nook.",
  "A toaster with a heating rack.",
  "A toaster with an egg poacher.",
  "A toaster with a warm pastry.",
  "A toaster with a reheat function.",
  "A toaster in a college dorm.",
  "A toaster with a glass window.",
  "A toaster on a marble countertop.",
  "A toaster with a cancel button.",
  "A toaster next to a cutting board.",
  "A toaster in a hotel room.",
  "A toaster with toast shaped like a heart.",
  "A toaster next to a fruit bowl.",
  "A toaster with a sleek design.",
  "A toaster with a cord wrap.",
  "A toaster with toast popping up.",
  "A toaster with a bagel in one slot.",
  "A sleek, black toaster in a modern kitchen.",
  "A toaster with an LCD display.",
  "A toaster with a crumb tray being emptied.",
  "A family gathered around the toaster for breakfast.",
  "A toaster with a single slice of toast.",
  "A toaster next to a jar of jam.",
  "A toaster with a retro design.",
  "A toaster with a custom logo.",
  "A toaster with bread just inserted.",
  "A toaster beside a stack of pancakes.",
  "A toaster next to a glass of orange juice.",
  "A toaster with steam rising from it.",
  "A toaster on a picnic table.",
  "A toaster with a burn mark on the counter.",
  "A toaster with a timer set to brown bread.",
  "A toaster surrounded by breakfast ingredients.",
  "A toaster with an artisanal loaf of bread.",
  "A toaster with a reflection of a sunny window.",
  "A red apple on a wooden table.",
  "A green apple in a fruit bowl.",
  "A basket of apples in an orchard.",
  "A sliced apple on a cutting board.",
  "A caramel apple at a fair.",
  "A child holding a shiny apple.",
  "A teacher's desk with an apple.",
  "A pile of apples at a farmer's market.",
  "A bitten apple on a picnic blanket.",
  "A bowl of apples on a kitchen counter.",
  "A fresh apple in a lunchbox.",
  "A golden apple in a mythological setting.",
  "A basket of apples in autumn.",
  "A single apple on a white plate.",
  "A woman picking apples from a tree.",
  "A recipe book next to an apple pie.",
  "A bunch of apples in a grocery store.",
  "A painter's still life with apples.",
  "A red apple on a classroom desk.",
  "A crate of apples being delivered.",
  "A worm peeking out of an apple.",
  "A polished apple reflecting light.",
  "A green apple in a child's hand.",
  "A baked apple dessert on a plate.",
  "A shiny apple in a supermarket display.",
  "A pile of apples next to a cider press.",
  "A bowl of apples on a dining table.",
  "A ripe apple hanging from a tree.",
  "A basket of apples on a porch.",
  "A single apple with a leaf attached.",
  "A rustic wooden dining table",
  "A modern glass dining table",
  "A dining table in a farmhouse kitchen",
  "A dining table with a holiday feast",
  "A dining table with candlelight dinner",
  "A dining table with mismatched vintage chairs",
  "A dining table for a romantic dinner",
  "A large dining table in a grand hall",
  "A dining table with fresh flowers",
  "A dining table with international dishes",
  "A dining table under a pergola",
  "A dining table with a bohemian tablecloth",
  "A dining table for a children's party",
  "A dining table with a luxurious setup",
  "A dining table in a beach cabana",
  "A dining table with a rustic breakfast",
  "A dining table in an urban apartment",
  "A dining table with afternoon tea",
  "A dining table in a mountain cabin",
  "A dining table for Thanksgiving dinner",
  "A dining table with a charcuterie board",
  "A dining table with Scandinavian design",
  "A dining table in a sunlit room",
  "A dining table with contemporary and antique elements",
  "A dining table for an outdoor barbecue",
  "A dining table with an Alice in Wonderland theme",
  "A dining table in a modern kitchen",
  "A dining table with an organic meal",
  "A dining table in a tropical setting",
  "A dining table for a wedding reception",
  "A dining table with a rustic brunch",
  "A futuristic glowing keyboard",
  "A vintage typewriter-style keyboard",
  "A mechanical keyboard with RGB lighting",
  "A colorful painted keyboard",
  "A steampunk-themed keyboard",
  "A sleek wireless keyboard",
  "A magical self-typing keyboard",
  "A musician's hands playing a keyboard",
  "A cat sitting on a keyboard",
  "A keyboard underwater with fish",
  "A retro computer keyboard",
  "A transparent futuristic keyboard",
  "A keyboard with gemstone keys",
  "A holographic keyboard in a lab",
  "A coffee-stained keyboard on a messy desk",
  "A keyboard covered in leaves and vines",
  "A keyboard made of ice",
  "A keyboard with tiny LED screens",
  "A high-tech spaceship keyboard",
  "A keyboard made of wood and stone",
  "A robot typing on a keyboard",
  "A keyboard in a candlelit library",
  "A keyboard with animal-shaped keys",
  "An ancient mystical keyboard",
  "A colorful toy keyboard",
  "A keyboard in a high-rise office",
  "A keyboard with flowers between the keys",
  "A virtual reality keyboard",
  "A broken keyboard with scattered keys",
  "A holographic projection of a keyboard",
  "A silver antique spoon",
  "A wooden spoon with carved designs",
  "A spoon with colorful enamel",
  "A futuristic metallic spoon",
  "A spoon with an intricate handle",
  "A spoon in a bowl of soup",
  "A spoon with a dollop of ice cream",
  "A spoon in a cup of coffee",
  "A spoon with a floral pattern",
  "A spoon on a rustic table",
  "A spoon with a gold finish",
  "A spoon resting on a napkin",
  "A spoon with a modern minimalist design",
  "A spoon with a ceramic handle",
  "A spoon in a jar of honey",
  "A spoon with a twisted handle",
  "A spoon with an animal motif",
  "A spoon in a mixing bowl",
  "A spoon with a geometric design",
  "A spoon with a mother-of-pearl handle",
  "A spoon in a glass of tea",
  "A spoon in a dessert bowl",
  "A spoon with an engraved name",
  "A spoon in a fruit salad",
  "A spoon with a seashell design",
  "A spoon with a colorful mosaic handle",
  "A spoon in a fancy place setting",
  "A spoon with a delicate lace pattern",
  "A spoon in a pot of stew",
  "A spoon with a whimsical shape",
  "A spoon with a classic vintage style",
  "A blue toothbrush with soft bristles",
  "A bamboo toothbrush with natural fibers",
  "A futuristic electric toothbrush",
  "A toothbrush with a rainbow handle",
  "A toothbrush in a glass holder",
  "A toothbrush with a built-in timer",
  "A child’s toothbrush with cartoon characters",
  "A toothbrush with eco-friendly packaging",
  "A travel toothbrush with a protective case",
  "A toothbrush with replaceable heads",
  "A toothbrush with a suction cup base",
  "A toothbrush with a built-in water flosser",
  "A toothbrush with a wooden handle",
  "A toothbrush with a curved design",
  "A toothbrush with a bright LED light",
  "A toothbrush with silicone bristles",
  "A toothbrush with a textured grip",
  "A toothbrush with a sleek, modern look",
  "A toothbrush in a bathroom organizer",
  "A toothbrush with a colorful pattern",
  "A toothbrush with extra-soft bristles for sensitive teeth",
  "A toothbrush with a tongue cleaner",
  "A toothbrush in a travel bag",
  "A toothbrush with a built-in toothpaste dispenser",
  "A toothbrush with antimicrobial bristles",
  "A toothbrush in a kid’s bathroom",
  "A toothbrush with a rechargeable battery",
  "A toothbrush with an ergonomic handle",
  "A toothbrush with dual action bristles",
  "A toothbrush in a minimalist bathroom",
  "A toothbrush with a fun, quirky design",
  "A silver dining fork",
  "A wooden-handled fork",
  "A gold-plated fork",
  "A plastic disposable fork",
  "A vintage ornate fork",
  "A fork with a floral pattern",
  "A stainless steel dessert fork",
  "A salad fork with a short handle",
  "A fork with a curved design",
  "A fork with a black handle",
  "A picnic fork in a basket",
  "A fork with a textured grip",
  "A fork with a colorful handle",
  "A camping fork with a folding design",
  "A fork with an ergonomic handle",
  "A child's fork with animal designs",
  "A seafood fork with narrow tines",
  "A fork with a long handle",
  "A fork with a decorative handle",
  "A travel fork in a case",
  "A fork with a straight design",
  "A fork in a cutlery drawer",
  "A fork with a unique shape",
  "A fork with a marble handle",
  "A fork with a hammered finish",
  "A fork in a place setting",
  "A fork with a mother-of-pearl handle",
  "A fork with a sleek, modern design",
  "A fork with a personalized engraving",
  "A fork with a simple, minimalist look",
  "A sharp chef's knife",
  "A serrated bread knife",
  "A hunting knife with a leather sheath",
  "A pocket knife with multiple tools",
  "A ceramic paring knife",
  "A knife with a wooden handle",
  "A decorative ceremonial knife",
  "A knife with a stainless steel blade",
  "A folding knife with a clip",
  "A knife set in a wooden block",
  "A butter knife with a curved edge",
  "A steak knife with a serrated edge",
  "A knife with an ornate handle",
  "A survival knife with a compass",
  "A knife on a cutting board",
  "A knife with a Damascus steel blade",
  "A knife with a colorful handle",
  "A knife with a built-in sharpener",
  "A knife with a plastic handle",
  "A knife with a safety cover",
  "A knife with a black blade",
  "A knife with a grip handle",
  "A knife in a chef's roll",
  "A knife in a kitchen drawer",
  "A knife with a straight edge",
  "A knife with a hollow handle",
  "A knife with an ergonomic design",
  "A knife with a non-stick coating",
  "A knife with a finger guard",
  "A knife with a custom engraving",
  "A compact microwave oven",
  "A stainless steel microwave",
  "A retro-style microwave",
  "A microwave with a digital display",
  "A microwave with convection cooking",
  "A microwave with a turntable",
  "A microwave with a black finish",
  "A microwave with a touchpad control",
  "A microwave with a child lock feature",
  "A microwave with a quick-start button",
  "A microwave with a defrost setting",
  "A microwave with a grilling function",
  "A microwave with a built-in timer",
  "A microwave with a sensor cooking",
  "A microwave with a white finish",
  "A microwave with a popcorn button",
  "A microwave with an eco mode",
  "A microwave with a glass door",
  "A microwave with a rotating plate",
  "A microwave with a compact design",
  "A microwave with a powerful wattage",
  "A microwave with a sleek, modern look",
  "A microwave with an auto-cook menu",
  "A microwave with a quiet operation",
  "A microwave with a stainless interior",
  "A microwave with a push-button door",
  "A microwave with a keep-warm function",
  "A microwave with a smart control",
  "A microwave with a soft-close door",
  "A microwave with a stylish design",
  "A ceramic coffee cup",
  "A glass tea cup",
  "A travel mug with a lid",
  "A stainless steel tumbler",
  "A plastic water cup",
  "A vintage porcelain cup",
  "A cup with a floral pattern",
  "A cup with a handle",
  "A cup with a silicone sleeve",
  "A cup with a straw",
  "A cup with a funny quote",
  "A cup with a lid and straw",
  "A cup with a double wall",
  "A cup with a gradient color",
  "A cup with a minimalist design",
  "A cup with a custom print",
  "A cup with a marble finish",
  "A cup with a gold rim",
  "A cup with an infuser",
  "A cup with a matching saucer",
  "A cup with a geometric pattern",
  "A cup with a whimsical design",
  "A cup with a motivational message",
  "A cup with a sleek, modern look",
  "A cup with a thermal insulation",
  "A cup with a cork bottom",
  "A cup with a wide rim",
  "A cup with an ergonomic handle",
  "A cup with a spill-proof lid",
  "A cup with a unique shape",
  "A small brown field mouse",
  "A white lab mouse",
  "A mouse nibbling on cheese",
  "A mouse peeking out of a hole",
  "A mouse with big ears",
  "A mouse with a long tail",
  "A mouse in a cage",
  "A mouse in a field of grass",
  "A mouse climbing a stalk of wheat",
  "A mouse curled up and sleeping",
  "A mouse with whiskers twitching",
  "A mouse with a piece of bread",
  "A mouse in a pile of leaves",
  "A mouse in a cozy nest",
  "A mouse exploring a kitchen",
  "A mouse in a dollhouse",
  "A mouse on a piece of wood",
  "A mouse hiding in a boot",
  "A mouse with shiny eyes",
  "A mouse scurrying across the floor",
  "A mouse in a meadow of flowers",
  "A mouse under the moonlight",
  "A mouse with a pink nose",
  "A mouse climbing a rock",
  "A mouse in a burrow",
  "A mouse in the snow",
  "A mouse in a pile of seeds",
  "A mouse with tiny paws",
  "A mouse in a basket",
  "A mouse on a windowsill",
  "A mouse next to a mushroom",
  "A glass water bottle",
  "A stainless steel sports bottle",
  "A plastic soda bottle",
  "A vintage milk bottle",
  "A wine bottle with a cork",
  "A bottle with a flip-top lid",
  "A colorful reusable bottle",
  "A clear bottle with a fruit infuser",
  "A baby bottle with a nipple",
  "A perfume bottle with an atomizer",
  "A bottle with a straw",
  "A medicine bottle with a child-proof cap",
  "A bottle with a pump dispenser",
  "A travel-sized shampoo bottle",
  "A bottle with a wide mouth",
  "A bottle with a handle",
  "A bottle with a silicone sleeve",
  "A bottle with measurement markings",
  "A bottle with a unique shape",
  "A bottle with a decorative label",
  "A bottle in a neoprene sleeve",
  "A bottle with a carabiner clip",
  "A collapsible water bottle",
  "A bottle with an inspirational quote",
  "A bottle with a sports logo",
  "A thermal insulated bottle",
  "A UV sterilizing water bottle",
  "A bottle with a filter",
  "A bottle with a bamboo cap",
  "A bottle with a gradient color",
  "A stainless steel convection oven",
  "A vintage gas oven",
  "A compact toaster oven",
  "A double-wall oven",
  "A microwave oven with grill",
  "A wood-fired pizza oven",
  "A portable camping oven",
  "A solar-powered oven",
  "A built-in wall oven",
  "A countertop convection oven",
  "A smart oven with Wi-Fi",
  "A commercial bakery oven",
  "A traditional clay oven",
  "A rotisserie oven",
  "A self-cleaning oven",
  "A steam oven",
  "A freestanding electric oven",
  "A mini oven for small spaces",
  "A combination microwave and oven",
  "A glass door oven",
  "A Dutch oven",
  "A steam-assisted cooking oven",
  "A professional chef’s oven",
  "A colorful retro oven",
  "A propane outdoor oven",
  "A brick pizza oven",
  "A French door oven",
  "A slow cooker oven",
  "A portable electric oven",
  "A multifunctional air fryer oven",
  "A large flat-screen tv",
  "A vintage CRT tv",
  "A curved OLED tv",
  "A smart tv with built-in apps",
  "A wall-mounted tv",
  "A tv with a soundbar",
  "A portable mini tv",
  "A 4K ultra HD tv",
  "A tv with a DVD player",
  "A tv with a gaming console",
  "A tv on a stand",
  "A projector tv",
  "A waterproof outdoor tv",
  "A frameless tv",
  "A tv with a swivel mount",
  "A tv with built-in speakers",
  "A tv with a streaming stick",
  "A tv with an HDMI input",
  "A tv with a remote control",
  "A tv in a living room setup",
  "A tv with a cable box",
  "A tv with a USB port",
  "A tv with a VCR",
  "A tv with a picture-in-picture feature",
  "A tv with a smart assistant",
  "A tv in a bedroom",
  "A tv with a curved screen",
  "A tv with a flat base",
  "A tv with multiple HDMI ports",
  "A tv in a home theat",
  "A tall stemmed wine glass",
  "A stemless wine glass",
  "A crystal wine glass",
  "A wine glass with a gold rim",
  "A wine glass with a colored base",
  "A wine glass with etched designs",
  "A wine glass with a wide bowl",
  "A wine glass with a narrow stem",
  "A wine glass with a decorative pattern",
  "A wine glass with a unique shape",
  "A wine glass with a personalized engraving",
  "A wine glass with a thick base",
  "A wine glass with a swirled design",
  "A wine glass with a sleek modern look",
  "A wine glass with a rustic finish",
  "A wine glass in a gift box",
  "A wine glass with a smooth rim",
  "A wine glass with a heavy base",
  "A wine glass with a fancy logo",
  "A wine glass with a frosted finish",
  "A wine glass with a twisted stem",
  "A wine glass with a hand-painted design",
  "A wine glass with a long stem",
  "A wine glass with a short stem",
  "A wine glass with a vibrant color",
  "A wine glass with a metallic finish",
  "A wine glass with a matching decanter",
  "A wine glass with a thin lip",
  "A wine glass with a classic shape",
  "A wine glass with a playful design",
  "A pair of stainless steel scissors",
  "A pair of pinking shears",
  "A pair of children's safety scissors",
  "A pair of heavy-duty kitchen scissors",
  "A pair of craft scissors with patterned blades",
  "A pair of sewing scissors",
  "A pair of ergonomic scissors",
  "A pair of hair cutting scissors",
  "A pair of left-handed scissors",
  "A pair of garden pruning scissors",
  "A pair of folding travel scissors",
  "A pair of vintage scissors",
  "A pair of office scissors",
  "A pair of medical bandage scissors",
  "A pair of scissors with a soft grip",
  "A pair of decorative scissors",
  "A pair of bonsai scissors",
  "A pair of multi-purpose scissors",
  "A pair of precision nail scissors",
  "A pair of large industrial scissors",
  "A pair of stainless steel embroidery scissors",
  "A pair of non-stick scissors",
  "A pair of lightweight plastic scissors",
  "A pair of ergonomic handle scissors",
  "A pair of pocket scissors",
  "A pair of detachable blade scissors",
  "A pair of double-bladed scissors",
  "A pair of bent handle scissors",
  "A pair of utility scissors",
  "A pair of wire-cutting scissors",
  "A mountain bicycle",
  "A road racing bicycle",
  "A vintage cruiser bicycle",
  "A folding commuter bicycle",
  "A tandem bicycle for two",
  "A BMX trick bicycle",
  "A children's training bicycle",
  "A carbon fiber racing bicycle",
  "A bicycle with a basket",
  "A hybrid city bicycle",
  "A touring bicycle with panniers",
  "A fixed gear bicycle",
  "A beach cruiser bicycle",
  "A recumbent bicycle",
  "A fat tire mountain bicycle",
  "A bicycle with disc brakes",
  "A bicycle with a rear rack",
  "A bicycle with a child seat",
  "An electric assist bicycle",
  "A gravel bike",
  "A hardcover novel",
  "A leather-bound book",
  "A paperback thriller",
  "A children's picture book",
  "A cookbook with recipes",
  "A science fiction novel",
  "A historical biography",
  "A travel guidebook",
  "A poetry anthology",
  "A graphic novel",
  "A self-help book",
  "A fantasy epic",
  "A mystery detective book",
  "A classic literature book",
  "A comic book",
  "A coffee table photo book",
  "A DIY craft book",
  "A romance novel",
  "A non-fiction history book",
  "A gardening book",
  "A head of broccoli",
  "Steamed broccoli florets",
  "broccoli cheddar soup",
  "broccoli and cheese casserole",
  "broccoli stir-fry",
  "Roasted broccoli with garlic",
  "broccoli salad with cranberries",
  "broccoli and beef",
  "broccoli with lemon zest",
  "broccoli and rice",
  "broccoli pasta",
  "broccoli in a veggie platter",
  "broccoli soup",
  "broccoli cheese bites",
  "broccoli and quinoa",
  "broccoli and tofu",
  "broccoli gratin",
  "broccoli tempura",
  "broccoli with almonds",
  "broccoli pesto pasta",
  "A leather sectional couch",
  "A modern fabric sofa",
  "A vintage velvet couch",
  "A reclining couch",
  "A sleeper sofa",
  "A loveseat couch",
  "A modular couch",
  "A Chesterfield sofa",
  "A tufted couch",
  "A minimalist couch",
  "A mid-century modern couch",
  "A L-shaped couch",
  "A pull-out sofa bed",
  "A convertible futon couch",
  "A corner sofa",
  "A three-seater couch",
  "A couch with throw pillows",
  "A chaise lounge couch",
  "A couch with a slipcover",
  "A patterned fabric couch",
  "A fresh orange",
  "A peeled orange",
  "A glass of orange juice",
  "A bowl of orange slices",
  "An orange tree",
  "A basket of oranges",
  "An orange smoothie",
  "Orange zest",
  "Orange marmalade",
  "A cut orange half",
  "A segment of orange",
  "A blood orange",
  "A mandarin orange",
  "A Valencia orange",
  "Orange wedges in a salad",
  "A juicing orange",
  "A navel orange",
  "Orange sorbet",
  "Orange and clove pomander",
  "Orange peel garnish",
  "A classic parking meter",
  "A digital parking meter",
  "A parking meter with a credit card slot",
  "A parking meter with a broken sign",
  "A solar-powered parking meter",
  "A parking meter with a yellow hood",
  "A parking meter next to a fire hydrant",
  "A parking meter with a time limit",
  "A double parking meter",
  "A parking meter on a busy street",
  "A parking meter with a coin slot",
  "A parking meter with graffiti",
  "A parking meter in a parking lot",
  "A parking meter with an expired sign",
  "A parking meter in a downtown area",
  "A parking meter with a blue cover",
  "A parking meter with a sticker",
  "A parking meter with an out of order sign",
  "A parking meter with a digital screen",
  "A parking meter at a beachside",
  "A TV remote control",
  "A universal remote",
  "A remote with a touchscreen",
  "A remote with large buttons",
  "A voice-activated remote",
  "A backlit remote",
  "A remote with a numeric keypad",
  "A gaming console remote",
  "A DVD player remote",
  "A streaming device remote",
  "A ceiling fan remote",
  "An air conditioner remote",
  "A projector remote",
  "A smart home remote",
  "A remote with a scroll wheel",
  "A simple remote with few buttons",
  "A Bluetooth remote",
  "A remote with a headphone jack",
  "A remote with a built-in keyboard",
  "A remote with a rechargeable battery",
  "A snowboard on fresh powder",
  "A snowboard with bindings",
  "A freestyle snowboard",
  "A snowboarder in mid-air",
  "A snowboard with a colorful design",
  "A snowboard leaning against a cabin",
  "A snowboard with a mountain backdrop",
  "A snowboard on a ski lift",
  "A snowboarder carving down a slope",
  "A snowboard with a winter landscape",
  "A snowboard with a custom graphic",
  "A snowboard on a snowy trail",
  "A snowboard with a camber profile",
  "A snowboard with a flat profile",
  "A snowboarder in the park",
  "A splitboard snowboard",
  "A snowboarder doing a trick",
  "A snowboard with a stomp pad",
  "A snowboard with a travel bag",
  "A snowboarder in deep powder",
  "A hard-shell suitcase",
  "A soft-sided suitcase",
  "A carry-on suitcase",
  "A suitcase with wheels",
  "A suitcase with a telescoping handle",
  "A vintage leather suitcase",
  "A suitcase with a combination lock",
  "A suitcase with compartments",
  "A suitcase with a floral pattern",
  "A suitcase with a built-in charger",
  "A suitcase with a TSA-approved lock",
  "A lightweight suitcase",
  "A suitcase with an expandable section",
  "A suitcase with a bright color",
  "A suitcase in an airport",
  "A suitcase with travel stickers",
  "A suitcase with a broken handle",
  "A suitcase with a camouflage design",
  "A suitcase with a detachable strap",
  "A suitcase with packing cubes",
  "A red traffic light",
  "A green traffic light",
  "A yellow traffic light",
  "A traffic light with a countdown timer",
  "A pedestrian traffic light",
  "A traffic light at a busy intersection",
  "A traffic light with a left turn signal",
  "A traffic light with a camera",
  "A traffic light on a rural road",
  "A traffic light in the rain",
  "A traffic light with a flashing yellow",
  "A traffic light at a crosswalk",
  "A traffic light with a sunshade",
  "A traffic light at a construction zone",
  "A traffic light with a right turn arrow",
  "A traffic light with a bike signal",
  "A traffic light on a snowy day",
  "A traffic light with a malfunction",
  "A traffic light in a city",
  "A traffic light with a green arrow",
  "hot dog with mustard",
  "hot dog with ketchup",
  "hot dog with relish",
  "hot dog with onions",
  "hot dog with sauerkraut",
  "hot dog with cheese",
  "hot dog with bacon bits",
  "hot dog with jalapenos",
  "hot dog with coleslaw",
  "hot dog with BBQ sauce",
  "hot dog with pickles",
  "hot dog with spicy mustard",
  "hot dog with avocado",
  "hot dog with crispy onions",
  "hot dog with horseradish sauce",
  "hot dog with mac and cheese",
  "hot dog with Sriracha mayo",
  "hot dog with pulled pork",
  "hot dog with fried egg",
  "hot dog with tofu",
  "hot dog with teriyaki sauce",
  "hot dog with wasabi mayo",
  "hot dog with poutine",
  "hot dog with marinara",
  "hot dog with kimchi",
  "hot dog with pesto",
  "hot dog with tzatziki",
  "hot dog with mango salsa",
  "hot dog with curry sauce",
  "hot dog with spinach dip",
  "hot dog with buffalo chicken",
  "hot dog with hummus",
  "hot dog with peanut butter",
  "hot dog with sweet and sour",
  "hot dog with ranch dressing",
  "hair drier with diffuser",
  "hair drier with ionic technology",
  "hair drier with cool shot",
  "hair drier with heat settings",
  "hair drier with concentrator",
  "hair drier with lightweight design",
  "hair drier with foldable handle",
  "hair drier with cordless feature",
  "hair drier with retractable cord",
  "hair drier with speed settings",
  "hair drier with ceramic technology",
  "hair drier with tourmaline",
  "hair drier with quiet motor",
  "hair drier with sleek design",
  "hair drier with removable filter",
  "hair drier with ergonomic handle",
  "hair drier with salon-grade motor",
  "hair drier with dual voltage",
  "hair drier with long cord",
  "hair drier with digital display",
  "hair drier with cool-touch",
  "hair drier with infrared heat",
  "hair drier with storage bag",
  "hair drier with hanging loop",
  "hair drier with anti-frizz",
  "hair drier with brush attachment",
  "hair drier with soft-grip",
  "hair drier with turbo boost",
  "hair drier with precision nozzle",
  "hair drier with ceramic finish",
  "hair drier with travel case",
  "hair drier with detachable filter",
  "hair drier with lightweight body",
  "hair drier with multiple attachments",
  "hair drier with fast drying",
  "baseball glove with leather",
  "baseball glove for youth",
  "baseball glove for catcher",
  "baseball glove for first baseman",
  "baseball glove for softball",
  "baseball glove for left-handed",
  "baseball glove for right-handed",
  "baseball glove with vintage look",
  "baseball glove with padded palm",
  "baseball glove with adjustable strap",
  "baseball glove with broken-in feel",
  "baseball glove with finger protection",
  "baseball glove with deep pocket",
  "baseball glove with open web",
  "baseball glove with closed web",
  "baseball glove with breathable mesh",
  "baseball glove with durable leather",
  "baseball glove with soft lining",
  "baseball glove with reinforced thumb",
  "baseball glove with wrist strap",
  "baseball glove with dual post web",
  "baseball glove with basket web",
  "baseball glove with h-web",
  "baseball glove with trapeze web",
  "baseball glove with modified trapeze web",
  "baseball glove with pro-I web",
  "baseball glove with I-web",
  "baseball glove with T-web",
  "baseball glove with solid web",
  "baseball glove with split hinge web",
  "baseball glove with single post web",
  "baseball glove with spiral lock web",
  "baseball glove with v-web",
  "baseball glove with hybrid web",
  "baseball glove with checkerboard web",
  "potted plant with succulents",
  "potted plant with fern",
  "potted plant with cactus",
  "potted plant with spider plant",
  "potted plant with snake plant",
  "potted plant with peace lily",
  "potted plant with bonsai tree",
  "potted plant with aloe vera",
  "potted plant with fiddle leaf fig",
  "potted plant with pothos",
  "potted plant with philodendron",
  "potted plant with jade plant",
  "potted plant with rubber plant",
  "potted plant with ZZ plant",
  "potted plant with palm tree",
  "potted plant with bamboo",
  "potted plant with orchid",
  "potted plant with lavender",
  "potted plant with rosemary",
  "potted plant with mint",
  "potted plant with basil",
  "potted plant with sage",
  "potted plant with thyme",
  "potted plant with parsley",
  "potted plant with cilantro",
  "potted plant with hibiscus",
  "potted plant with bougainvillea",
  "potted plant with geranium",
  "potted plant with hydrangea",
  "potted plant with begonia",
  "potted plant with marigold",
  "potted plant with chrysanthemum",
  "potted plant with petunia",
  "potted plant with pansy",
  "potted plant with coleus",
  "A frisbee flying through the air.",
  "A dog catching a frisbee in the park.",
  "A colorful frisbee on the grass.",
  "A frisbee game on the beach.",
  "A frisbee in mid-air during a game of Ultimate.",
  "A frisbee lying next to a picnic blanket.",
  "A frisbee stuck in a tree.",
  "A group of friends throwing a frisbee.",
  "A frisbee with a sunset backdrop.",
  "A frisbee gliding over a lake.",
  "A child reaching to catch a frisbee.",
  "A frisbee with a custom design.",
  "A frisbee in a backyard barbecue.",
  "A frisbee with a city skyline in the background.",
  "A frisbee on a sandy beach.",
  "A frisbee being thrown in a field.",
  "A frisbee next to a pair of sunglasses.",
  "A frisbee landing on a roof.",
  "A frisbee on a hiking trail.",
  "A frisbee in a college campus quad.",
  "A frisbee in a sports store display.",
  "A frisbee with a team logo.",
  "A frisbee in a park on a sunny day.",
  "A frisbee being caught by an athlete.",
  "A frisbee in a family's yard.",
  "A frisbee with a mountain view.",
  "A frisbee in a school playground.",
  "A frisbee game at a summer camp.",
  "A frisbee with a floral pattern.",
  "A frisbee in a crowded park.",
  "A stainless steel sink in a modern kitchen.",
  "A farmhouse sink filled with dishes.",
  "A bathroom sink with a marble countertop.",
  "A sink with water running from the faucet.",
  "A sink with a colorful backsplash.",
  "A kitchen sink with a window view.",
  "A sink with a soap dispenser and sponge.",
  "A sink full of bubbles and soapy water.",
  "A double-basin sink with a cutting board.",
  "A vintage porcelain sink in a cottage.",
  "A sink with fresh vegetables being washed.",
  "A sink with a dripping faucet.",
  "A sink with a toothbrush holder and toothpaste.",
  "A sink with a pile of dirty dishes.",
  "A sink with a drying rack next to it.",
  "A sink with a shiny chrome faucet.",
  "A sink with flowers in a vase beside it.",
  "A sink in a laundry room.",
  "A sink with a child washing their hands.",
  "A sink with a stack of clean plates.",
]

sports_ball_prompts = [
  "A soccer ball on a grassy field.",
  "A basketball on a wooden court.",
  "A beach ball on the sand.",
  "A tennis ball on a racket.",
  "A baseball in a glove.",
  "A golf ball on a tee.",
  "A volleyball at the net.",
  "A bowling ball on the lane.",
  "A football on a turf field.",
  "A ping pong ball on a table.",
  "A dodgeball in a gymnasium.",
  "A rugby ball in mid-air.",
  "A ball of yarn in a knitting basket.",
  "A crystal ball on a stand.",
  "A rubber ball bouncing on the sidewalk.",
  "A cue ball on a pool table.",
  "A ball of twine in a drawer.",
  "A medicine ball in a gym.",
  "A juggling ball in motion.",
  "A bocce ball on a lawn.",
  "A soccer ball on a grassy field.",
  "A basketball on a wooden court.",
  "A beach ball on the sand.",
  "A tennis ball on a racket.",
  "A baseball in a glove.",
  "A golf ball on a tee.",
  "A volleyball at the net.",
  "A bowling ball on the lane.",
  "A football on a turf field.",
  "A ping pong ball on a table.",
  "A dodgeball in a gymnasium.",
  "A rugby ball in mid-air.",
  "A cricket ball in a batsman's hand.",
  "A handball on a court.",
  "A polo ball in a game.",
  "A lacrosse ball in a stick's net.",
  "A squash ball on a court.",
  "A field hockey ball in motion.",
  "A softball on the pitcher's mound.",
  "A bocce ball on a lawn.",
  "A polo ball rolling on grass.",
  "A water polo ball in the pool.",
  "A paddle ball in a beach game.",
  "A billiard ball on a pool table.",
  "A hurling ball in play.",
  "A fustal ball on an indoor court.",
  "A tetherball swinging around a pole.",
  "A racquetball in a court.",
  "A wiffle ball in a backyard game.",
  "A kabbadi ball on a field.",
  "A sepaktakraw ball in mid-air.",
  "A pelota ball in a jai alai game.",
  "A petanque ball on gravel.",
  "A gaelic football in a game.",
  "A roller hockey ball on a rink.",
  "A tchoukball on a rebounder.",
  "A broomball in a match.",
  "A carrot in a vegetable garden.",
  "A bunch of fresh carrots on a wooden table.",
  "A rabbit holding a carrot.",
  "A carrot cake with cream cheese frosting.",
  "A carrot-shaped pencil case.",
  "A bowl of carrot soup.",
  "A baby eating a carrot.",
  "A carrot growing in the soil.",
  "A carrot in a shopping basket.",
  "A carrot being sliced on a cutting board.",
  "A carrot in a glass of juice.",
  "A snowman with a carrot nose.",
  "A basket filled with carrots.",
  "A carrot being harvested.",
  "A carrot in a salad.",
  "A carrot in a rabbit's mouth.",
  "A plate of roasted carrots.",
  "A carrot-shaped cookie.",
  "A pile of carrots at a farmer's market.",
  "A carrot in a blender.",
  "A carrot in a child's lunchbox.",
  "A carrot next to a bunch of parsley.",
  "A carrot in a bowl of stew.",
  "A carrot on a kitchen counter.",
  "A carrot in a vegetable stir-fry.",
  "A carrot cake being decorated.",
  "A bowl of grated carrots.",
  "A carrot in a smoothie.",
  "A carrot in a vegetable soup.",
  "A carrot in a rabbit hutch.",
  "A carrot being peeled.",
  "A carrot in a grocery store.",
  "A carrot in a picnic basket.",
  "A carrot being fed to a horse.",
  "A carrot in a garden bed.",
  "A carrot next to a radish.",
  "A carrot in a lunch salad.",
  "A carrot being dipped in hummus.",
  "A carrot on a cutting board.",
  "A carrot in a basket with other vegetables.",
]

# Merge prompts

In [ ]:
animal_prompts = unrealistic_prompts + realistic_prompts
raw_prompts = animal_prompts + captions + non_animal_prompts
# print(len(raw_prompts))

# remove duplicates
raw_prompts = list(set(raw_prompts))
# print(len(raw_prompts))

# remove prompts with multiple animal species
animals = ['giraffe', 'zebra', 'bear', 'sheep', 'cat', 'dog', 'horse', 'cow', 'elephant', 'bird']
counts = [0] * len(labels)

prompts = []

for prompt in raw_prompts:
    count = 0
    for label in labels:
        if label in prompt:
            if label != 'hot dog' and label != 'carrot':
                count += 1
    if count == 1:
        for label in labels:
            if label in prompt:
                # if label == 'bicycle':
                #     print(prompt)
                i = labels.index(label)
                if (counts[i] < 100) or (label == 'giraffe' or label == 'zebra'):
                    prompts.append((prompt, label))
                    counts[i] += 1

#sports ball prompts
for prompt in sports_ball_prompts:
    prompts.append((prompt, 'sports ball'))
    counts[labels.index('sports ball')] += 1

for i in range(len(counts)):
    if labels[i] in animals:
        print(f'{labels[i]}, {counts[i]}')

print(labels)
print(f'counts: {counts}') # counts: [519, 694, 145, 155, 133, 135, 150, 138, 130, 187]
print(len(prompts))

bird, 89
cat, 100
cow, 76
dog, 100
elephant, 98
giraffe, 587
horse, 100
sheep, 75
zebra, 747
['airplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'clock', 'couch', 'cow', 'cup', 'dining table', 'dog', 'donut', 'elephant', 'fire hydrant', 'fork', 'frisbee', 'giraffe', 'hair drier', 'handbag', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorcycle', 'mouse', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'potted plant', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'spoon', 'sports ball', 'stop sign', 'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'tie', 'toaster', 'toilet', 'toothbrush', 'traffic light', 'train', 'truck', 'tv', 'umbrella', 'vase', 'wine glass', 'zebra']
counts: [28, 33, 32, 43, 100, 35, 56, 60, 32, 89, 52,

# Test

In [ ]:
# YOLOで検出

import torch
from transformers import YolosImageProcessor, YolosForObjectDetection
import numpy as np

torch.set_grad_enabled(False)

device = 'cuda'
seed = 42

model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')
image_processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")
model.to(device)

def detect(image):
    inputs = image_processor(images=image, return_tensors="pt").to(device)
    outputs = model(**inputs)
    target_sizes = torch.tensor([image.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.6, target_sizes=target_sizes)[0]
    objects = [model.config.id2label[idx.item()] for idx in results['labels']]
    return objects

def is_correct(objects, name):
    # name = classes[label]
    return set(objects).intersection(set(labels)) == {name}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/26.0M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

# Creating images

In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

base_dir = "data/train"

In [ ]:
final_counts = [0] * len(labels)

def make_data():
    pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
    pipe.to("cuda")
    pipe.set_progress_bar_config(disable=True)
    def generate(prompt):
        image = pipe(prompt=prompt, num_inference_steps=1, guidance_scale=.0).images[0]
        return image

    ids = []
    paths = []
    final_prompts = []

    count = 0

    for i in tqdm(range(len(prompts))):
        prompt = prompts[i]
        # if prompt in final_prompts:
        #     continue
        path = os.path.join(base_dir, f"{i}.png")
        image = generate(prompt[0])

        # YOLOチェック
        objects = detect(image)
        if is_correct(objects, prompt[1]):
            count += 1
            final_counts[labels.index(prompt[1])] += 1
            final_prompts.append(prompt)
            image.save(path)
            # ids.append(i)
            paths.append(path)

    del pipe
    df = pd.DataFrame(
        {
            "prompts": final_prompts,
            "paths": paths,
        }
    )

    print()
    print(f'count: {count}')
    print(final_counts)
    for i in range(len(final_counts)):
        if final_counts[i] < 20:
            print(f'{labels[i]}, {final_counts[i]}')

    return df

In [ ]:
df = make_data()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 5638/5638 [25:10<00:00,  3.73it/s]


count: 2092
[18, 12, 5, 18, 13, 3, 14, 15, 16, 67, 23, 2, 16, 7, 3, 9, 6, 4, 3, 30, 1, 3, 64, 1, 47, 10, 1, 20, 10, 60, 11, 2, 12, 407, 3, 12, 37, 5, 17, 8, 8, 1, 21, 19, 23, 14, 5, 2, 11, 19, 4, 4, 5, 3, 23, 53, 3, 2, 32, 8, 2, 20, 14, 15, 1, 11, 39, 6, 2, 29, 9, 29, 62, 15, 4, 5, 4, 8, 542]
airplane, 18
apple, 12
backpack, 5
banana, 18
baseball bat, 13
baseball glove, 3
bed, 14
bench, 15
bicycle, 16
book, 2
bottle, 16
bowl, 7
broccoli, 3
bus, 9
cake, 6
car, 4
carrot, 3
cell phone, 1
chair, 3
couch, 1
cup, 10
dining table, 1
donut, 10
fire hydrant, 11
fork, 2
frisbee, 12
hair drier, 3
handbag, 12
hot dog, 5
keyboard, 17
kite, 8
knife, 8
laptop, 1
motorcycle, 19
orange, 14
oven, 5
parking meter, 2
person, 11
pizza, 19
potted plant, 4
refrigerator, 4
remote, 5
sandwich, 3
sink, 3
skateboard, 2
snowboard, 8
spoon, 2
stop sign, 14
suitcase, 15
surfboard, 1
teddy bear, 11
tie, 6
toaster, 2
toothbrush, 9
truck, 15
tv, 4
umbrella, 5
vase, 4
wine glass, 8


In [ ]:
# print(f'count: {count}')
print(final_counts)
for i in range(len(final_counts)):
    if final_counts[i] == 0:
        print(f'{labels[i]}, {final_counts[i]}')

count: 0
[18, 12, 5, 18, 13, 3, 14, 15, 16, 67, 23, 2, 16, 7, 3, 9, 6, 4, 3, 30, 1, 3, 64, 1, 47, 10, 1, 20, 10, 60, 11, 2, 12, 407, 3, 12, 37, 5, 17, 8, 8, 1, 21, 19, 23, 14, 5, 2, 11, 19, 4, 4, 5, 3, 23, 53, 3, 2, 32, 8, 2, 20, 14, 15, 1, 11, 39, 6, 2, 29, 9, 29, 62, 15, 4, 5, 4, 8, 542]


In [ ]:
len(captions)

3365

In [ ]:
df.to_csv("data/data.csv")

In [ ]:
!zip -q data_018.zip -r data

In [ ]:
!cp data_018.zip "/content/drive/MyDrive/dataset/data_018.zip"

In [ ]:
def terminate_session():
    # Terminate this session

    from google.colab import runtime
    runtime.unassign()

terminate_session()